In [ ]:
import sys
sys.path.insert(0, "..")
from src.config import PROJECT_ROOT, MODEL_SEED, LABELS
from src.variables import LLM_COLS, LAB_COLS, CODE_COLS, CATEGORY_COLS

import os
import re
import joblib, json
import glob
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple
from itertools import combinations

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    brier_score_loss,
    precision_recall_fscore_support,
    accuracy_score,
    roc_curve,
    confusion_matrix,
)
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Tree models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import shap
import matplotlib.pyplot as plt
from upsetplot import from_contents, plot
from sklearn.manifold import TSNE
import umap.umap_ as umap

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update(
    {"figure.dpi": 130, "axes.spines.top": False, "axes.spines.right": False}
)

In [ ]:
# ---------------------------
# 0) 경로/조합 설정
# ---------------------------
OUT_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
)
FIG_DIR = OUT_DIR / "figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

IMP_DIR = Path(
    str(PROJECT_ROOT / "data/processed_imp/260114_split_corr_LLM_ADER/imputation/simple_imput")
)
FS_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/Feature_Selection/simple_20/step2_FS")
)

# Train/Test 파일 이름
DATA_FILES = {
    "label_30d": {
        "train": IMP_DIR / "simple_label_30d_train.csv",
        "test": IMP_DIR / "simple_label_30d_test.csv",
    },
    "label_60d": {
        "train": IMP_DIR / "simple_label_60d_train.csv",
        "test": IMP_DIR / "simple_label_60d_test.csv",
    },
    "label_90d": {
        "train": IMP_DIR / "simple_label_90d_train.csv",
        "test": IMP_DIR / "simple_label_90d_test.csv",
    },
    "label_180d": {
        "train": IMP_DIR / "simple_label_180d_train.csv",
        "test": IMP_DIR / "simple_label_180d_test.csv",
    },
    "label_365d": {
        "train": IMP_DIR / "simple_label_365d_train.csv",
        "test": IMP_DIR / "simple_label_365d_test.csv",
    },
}

# 당신의 탐색 결과를 반영한 권장조합 (필요시 수정)
BEST_COMBOS = {
    "label_30d": {"model": "rf", "feature_set": "All_Features"},
    "label_60d": {"model": "rf", "feature_set": "All_Features"},
    "label_90d": {"model": "rf", "feature_set": "All_Features"},
    "label_180d": {"model": "rf", "feature_set": "All_Features"},
    "label_365d": {"model": "rf", "feature_set": "All_Features"},
}

TARGET_COL_FALLBACK = "label"

In [ ]:
MODELS_TO_RUN = ["LR", "RF", "XGB", "LGBM", "CATBOOST"]
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# 변수 그룹 정의
LLM_COLS = [
    "Impaired_Social_Function",  # 사회기능 저하
    "Religious_Affiliation",  # 종교적 소속
    "Violence_and_Impulsivity",  # 폭력성 및 충동성
    "Domestic_Violence",  # 가정폭력
    "Physical_Abuse",  # 신체적 학대
    "Divorce",  # 이혼 경험
    "Death_of_Family_Member",  # 가족의 죽음
    "Emotional_Abuse",  # 정서적 학대
    "Lack_of_Family_Support",  # 가족 지지 부족
    "Social_Isolation_and_Lack_of_Support",  # 사회적 지지 부족 및 사회적 고립
    "Psychotic_Symptoms",  # 환청/망상/피해사고
    "Interpersonal_Conflict",  # 대인관계 갈등
    "Exposure_to_Suicide",  # 타인의 자살 목격 및 노출
    "Alcohol_Use_Problems",  # 알코올 사용 문제
    "Sexual_Abuse",  # 성적피해
    "Physical_and_Emotional_Neglect",  # 신체적 및 정서적 방임
]

# Lab 변수 목록
LAB_COLS = [
    "BL3125",
    "BL3137",
    "BL3140",
    "BL3141",
    "BL3142",
    "BL314201",
    "BL3603",
    "NR4303",
    "BL2011",
    "BL2012",
    "BL2013",
    "BL2014",
    "BL201401",
    "BL201402",
    "BL201403",
    "BL2016",
    "BL201801",
    "BL201802",
    "BL201803",
    "BL201804",
    "BL201805",
    "BL201806",
    "BL201807",
    "BL201808",
    "BL201809",
    "BL201810",
    "BL201811",
    "BL201812",
    "BL201813",
    "BL201814",
    "BL201815",
    "BL201816",
    "BL201818",
    "BL3111",
    "BL3112",
    "BL3131",
    "BL3132",
    "BL3133",
    "BL311201",
    "BL311202",
    "BL3113",
    "BL3114",
    "BL3115",
    "BL3116",
    "BL3117",
    "BL3118",
    "BL3119",
    "BL3120",
    "BL312001",
    "BL312002",
    "BL3121",
    "BL3122",
    "BL3123",
]

CODE_COLS = ["sleep", "appetite", "weight"]

CATEGORY_COLS = [
    "sex",
    "edu",
    "job",
    "marry",
    "drink",
    "smoke",
    "substance_abuse",
    "psy_family",
    "AD_more_three",
    "ER_more_two",
    "Suicidalidea",
    "Suicidalplan",
    "Suicidalattempt",
    "benzodiazepine",
    "quetiapine",
    "aripiprazole",
    "lithium",
    "divalproex",
    "olanzapine",
    "bipolar",
    "depression",
    "schizophrenia",
    "anxiety",
    "trauma_stressor_related",
    "somatic_symptom_disorder",
    "psychotic_other",
    # LLM 변수도 범주형으로 취급
    "Impaired_Social_Function",  # 사회기능 저하
    "Religious_Affiliation",  # 종교적 소속
    "Violence_and_Impulsivity",  # 폭력성 및 충동성
    "Domestic_Violence",  # 가정폭력
    "Physical_Abuse",  # 신체적 학대
    "Divorce",  # 이혼 경험
    "Death_of_Family_Member",  # 가족의 죽음
    "Emotional_Abuse",  # 정서적 학대
    "Lack_of_Family_Support",  # 가족 지지 부족
    "Social_Isolation_and_Lack_of_Support",  # 사회적 지지 부족 및 사회적 고립
    "Psychotic_Symptoms",  # 환청/망상/피해사고
    "Interpersonal_Conflict",  # 대인관계 갈등
    "Exposure_to_Suicide",  # 타인의 자살 목격 및 노출
    "Alcohol_Use_Problems",  # 알코올 사용 문제
    "Sexual_Abuse",  # 성적피해
    "Physical_and_Emotional_Neglect",  # 신체적 및 정서적 방임
]

In [ ]:
# ---------------------------
# 1) 유틸: FS/데이터 로드
# ---------------------------
def find_feature_list_file(label: str) -> Path:
    # step2_FS 폴더 내 label 키워드 포함 & 35개 선정 파일을 우선 탐색
    # (환경에 맞게 패턴 필요시 조정)
    pats = [f"final_features_{label}.csv"]
    cands = []
    for p in pats:
        cands += list(FS_DIR.glob(p))
    if not cands:
        raise FileNotFoundError(f"[FS] No feature list file for {label} in {FS_DIR}")
    return sorted(cands)[-1]


def read_feature_list(fpath: Path) -> List[str]:
    df = pd.read_csv(fpath)
    cols_lower = [c.lower() for c in df.columns]
    if "feature" in cols_lower:
        col = df.columns[cols_lower.index("feature")]
    elif "variable" in cols_lower:
        col = df.columns[cols_lower.index("variable")]
    else:
        col = df.columns[0]
    feats = df[col].dropna().astype(str).tolist()
    # 중복 제거 & 공백 정리
    feats = list(dict.fromkeys([f.strip() for f in feats]))
    return feats


def load_xy(
    path: Path, label: str, features: List[str]
) -> Tuple[pd.DataFrame, pd.Series]:
    df = pd.read_csv(path)
    y_col = (
        label
        if label in df.columns
        else (TARGET_COL_FALLBACK if TARGET_COL_FALLBACK in df.columns else None)
    )
    if y_col is None:
        raise KeyError(
            f"Target column not found in {path.name}. Expected '{label}' or '{TARGET_COL_FALLBACK}'."
        )
    use_cols = [c for c in features if c in df.columns]
    missing = sorted(set(features) - set(use_cols))
    if missing:
        print(
            f"[{label}] Missing {len(missing)} FS features in {path.name}: {missing[:5]}{'...' if len(missing)>5 else ''}"
        )
    X = df[use_cols].copy()
    y = df[y_col].astype(int).copy()
    return X, y


def youden_threshold(y_true: np.ndarray, y_prob: np.ndarray):
    """
    ROC기반 Youden index 최대가 되는 threshold 반환.
    반환: thr*, tpr_at_thr, fpr_at_thr
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    youden = tpr - fpr
    idx = int(np.nanargmax(youden))
    return float(thresholds[idx]), float(tpr[idx]), float(fpr[idx])


def metrics_at_threshold(y_true: np.ndarray, y_prob: np.ndarray, thr: float) -> dict:
    """
    주어진 threshold에서의 성능 지표 계산: Sens, Spec, PPV, NPV, Acc, F1, TP/TN/FP/FN
    """
    y_pred = (y_prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    sens = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # Recall
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0.0  # Precision
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0
    acc = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0.0
    # F1 (positive class)
    f1 = (2 * ppv * sens) / (ppv + sens) if (ppv + sens) > 0 else 0.0
    return {
        "threshold": thr,
        "sensitivity": sens,
        "specificity": spec,
        "ppv": ppv,
        "npv": npv,
        "accuracy": acc,
        "f1": f1,
        "tp": int(tp),
        "tn": int(tn),
        "fp": int(fp),
        "fn": int(fn),
    }


def to_bayes_space(grid_dict):
    """
    Grid(dict of lists)를 BayesSearchCV의 search_spaces(dict of skopt spaces)로 변환.
    리스트는 Categorical로 감쌉니다.
    """
    space = {}
    for k, v in grid_dict.items():
        # v가 (low, high, type) 같은 튜플이 아니라면 전부 Categorical 처리
        # (원하시면 Real/Integer 범위로 바꾸세요)
        space[k] = Categorical(v)  # v가 list라고 가정
    return space

In [ ]:
# 0) 전역 상수로 이동 (또는 파일 상단에 정의)
FRIENDLY_OVERRIDES = {
    "age": "Age",
    "sex": "Sex (M=1)",
    "edu": "Education Level",
    "job": "Employment",
    "marry": "Marital Status",
    "smoke": "Smoking",
    "drink": "Drinking",
    "benzodiazepine": "Benzodiazepine",
    "quetiapine": "Quetiapine",
    "lithium": "Lithium",
    "divalproex": "Divalproex",
    "substance_abuse": "Substance Abuse",
    "Suicidalattempt": "Suicidal Attempt",
    "Suicidalplan": "Suicidal Plan",
    "trauma_stressor_related": "Trauma Stress",
    "WorkingMemoryIndex-Compositescore": "Working Memory",
    "PerceptualReasoningIndex-Compositescore": "Perceptual Reasoning",
    "ProcessingSpeedIndex-Compositescore": "Processing Speed",
    "psy_family": "Psychiatric family history",
    "stay_day": "Hospitalization period",
    "AD_more_three": "≥3 Admissions",
    "ER_more_two": "≥2 ER Visits",
    "psychotic_other": "Other Psychotic",
    "somatic_symptom_disorder": "Somatic Symptom Disorder",
    "anxiety": "Anxiety",
    # LLM
    "Impaired_Social_Function": "Social Function Impairment (LLM)",
    "Religious_Affiliation": "Religious Affiliation (LLM)",
    "Violence_and_Impulsivity": "Aggression/Impulsivity (LLM)",
    "Domestic_Violence": "Domestic Violence (LLM)",
    "Physical_Abuse": "Physical Abuse (LLM)",
    "Divorce": "Divorce Experience (LLM)",
    "Death_of_Family_Member": "Family Loss (LLM)",
    "Emotional_Abuse": "Emotional Abuse (LLM)",
    "Lack_of_Family_Support": "Lack of Family Support (LLM)",
    "Social_Isolation_and_Lack_of_Support": "Social Isolation (LLM)",
    "Psychotic_Symptoms": "Halluc/Delusion/Paranoia (LLM)",
    "Interpersonal_Conflict": "Interpersonal Conflict (LLM)",
    "Exposure_to_Suicide": "Suicide Exposure (LLM)",
    "Alcohol_Use_Problems": "Alcohol Use Issues (LLM)",
    "Sexual_Abuse": "Sexual Victimization (LLM)",
    "Physical_and_Emotional_Neglect": "Neglect (LLM)",
    # CODE_COLS
    "sleep": "Sleep",
    "appetite": "Appetite",
    "weight": "Weight Change",
}

# 1) LEVEL_MAP 정리: 중복되는 변수명 제거 + 레벨 키 정규화에 대비
LEVEL_MAP = {
    "sleep": {
        "0": "Normal sleep",
        "1": "Insomnia",
        "2": "Hypersomnia",
    },
    "appetite": {
        "0": "No change",
        "1": "Decreased appetite",
        "2": "Increased appetite",
    },
    "weight": {
        "0": "No change",
        "1": "Weight loss",
        "2": "Weight gain",
    },
}


def _norm_level(level: str) -> str:
    try:
        return str(int(float(level)))
    except Exception:
        return str(level).strip()


def _safe_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(f"Columns not found: {candidates}")


def load_feature_name_map(csv_path: str | Path) -> dict[str, str]:
    df = pd.read_csv(csv_path)
    code_col = _safe_col(df, ["Variabel Mapping", "Variable"])
    name_col = _safe_col(df, ["Unnamed: 9", "Term"])
    sub = df[[code_col, name_col]].dropna()
    sub = sub.rename(columns={code_col: "code", name_col: "name"})
    sub["code"] = sub["code"].astype(str).str.strip()
    sub["name"] = sub["name"].astype(str).str.strip()
    sub = sub[sub["code"].str.match(r"^BL\d+")]
    sub["pretty"] = sub["name"] + " (" + sub["code"] + ")"
    mapping = dict(zip(sub["code"], sub["pretty"]))
    # 전역 오버라이드도 함께 병합해 한 딕셔너리로 사용
    mapping.update(FRIENDLY_OVERRIDES)
    return mapping


def clean_ct_feature_name(raw_name: str) -> str:
    name = re.sub(r"^[^_]+__", "", raw_name)  # 'num__', 'cat__', 'code__'
    name = re.sub(r"^(num_|cat_|code_)", "", name)
    return name


def prettify_name(raw_name: str, mapping: dict[str, str] | None = None) -> str:
    """
    'sleep_1.0' → 'Sleep: Insomnia'
    'appetite_2' → 'Appetite: Increased appetite'
    'weight_0' → 'Weight Change: No change'
    BL코드/일반 변수는 mapping(+FRIENDLY_OVERRIDES)에 따라 변환.
    """
    base = clean_ct_feature_name(raw_name)
    mapping = mapping or {}

    m = re.match(r"^(.*?)[_](.+)$", base)
    if m:
        var, level = m.group(1), _norm_level(m.group(2))
        var_friendly = mapping.get(var, FRIENDLY_OVERRIDES.get(var, var))
        level_friendly = LEVEL_MAP.get(var, {}).get(level)
        if level_friendly is not None:
            return f"{var_friendly}: {level_friendly}"
        # 원-핫이지만 레벨 매핑이 없으면 원문 유지
        return f"{var_friendly}: {level}"

    # 원-핫이 아닌 일반 변수(BL코드 포함)
    return mapping.get(base, FRIENDLY_OVERRIDES.get(base, base))

In [ ]:
def inject_psych_scale_aliases(mapping: dict[str, str]) -> dict[str, str]:
    """
    L, Pd, Vr, mf 등 심리검사 약어 → 사람이 읽기 좋은 라벨로 보강.
    - 대소문자 무시
    - 접미어(_t, _raw 등)가 있어도 동작
    """
    # 기본 약어 사전 (필요시 여기 계속 추가)
    base_alias = {
        # MMPI Validity & Clinical
        "L": "MMPI Validity: Lie (L)",
        "Pd": "MMPI Clinical: Psychopathic Deviate (Pd)",
        "Mf": "MMPI Clinical: Masculinity–Femininity (Mf)",
        "mf": "MMPI Clinical: Masculinity–Femininity (Mf)",
        "TR": "MMPI Validity: TRIN",  # (True response inconsistency)
        "Vr": "MMPI Validity: VRIN",  # (Variable Response Inconsistency)
        "F": "MMPI Validity: Infrequency (F)",
        "K": "MMPI Validity: Defensiveness (K)",
        "Hs": "MMPI Clinical: Hypochondriasis (Hs)",
        "D": "MMPI Clinical: Depression (D)",
        "Hy": "MMPI Clinical: Hysteria (Hy)",
        "Pa": "MMPI Clinical: Paranoia (Pa)",
        "Sc": "MMPI Clinical: Schizophrenia (Sc)",
        "Ma": "MMPI Clinical: Hypomania (Ma)",
        "Si": "MMPI Clinical: Social Introversion (Si)",
    }

    # 대소문자 무시 매칭을 위해 보조 룰 추가:
    # - 'pd', 'PD', 'Pd_t', 'mf_raw'처럼 변형된 컬럼명에도 적용
    def add_case_insensitive(alias_key: str, pretty: str):
        # 정확히 일치하는 키도 덮어씀
        mapping[alias_key] = pretty
        # 자주 보이는 변형 패턴 → 정규식 룰
        # 1) 소문자/대문자 변형
        mapping[alias_key.lower()] = pretty
        mapping[alias_key.upper()] = pretty
        # 2) 접미어(_t, _raw, _score, _z 등)
        #   → 정규식으로 처리할 수 있도록, 나중 단계 prettify에서 사용
        #   여기선 원키만 심어두고, 정규식은 prettify 단계에서 적용
        #   (아래 prettify_psych_name에서 처리)
        return

    for k, v in base_alias.items():
        add_case_insensitive(k, v)

    return mapping


def prettify_psych_name(clean_name: str, mapping: dict[str, str]) -> str:
    """
    clean_name(접두사 제거된 원 컬럼명)에 심리검사 스케일 약어 매핑 적용.
    - 대소문자 무시
    - 접미어(_t/_raw 등)는 유지하되, 앞의 약어만 사람이 읽는 표기로 교체
    """
    # 원본 보존
    name = clean_name

    # 약어 + 선택적 접미어 패턴: 예) 'Pd', 'pd_t', 'MF_raw', 'Vr_score'
    m = re.match(r"^([A-Za-z]{1,3})(?:[_\-](.+))?$", name)
    if m:
        key = m.group(1)  # Pd / pd / MF / Vr ...
        suffix = m.group(2)  # t / raw / z 등 (없을 수도)
        # 매핑에 있으면 교체
        pretty_core = mapping.get(
            key, mapping.get(key.lower(), mapping.get(key.upper(), None))
        )
        if pretty_core:
            if suffix:
                # 접미어는 괄호 뒤에 그대로 덧붙여서 정보 보존
                return f"{pretty_core} [{suffix}]"
            else:
                return pretty_core
    # 매칭 안 되면 원래 이름 반환
    return name


def prettify_names(raw_names: list[str], mapping: dict[str, str]) -> list[str]:
    # (1) ColumnTransformer 접두사 제거
    cleaned = [clean_ct_feature_name(n) for n in raw_names]

    # ★ (2) 심리검사 약어 보정 먼저 처리
    mapping = inject_psych_scale_aliases(mapping)
    cleaned2 = [prettify_psych_name(n, mapping) for n in cleaned]

    # (3) BL코드 / LLM 등 일반 매핑 적용
    pretty = [mapping.get(n, n) for n in cleaned2]
    return pretty

In [ ]:
FEATURE_MAP_CSV = str(PROJECT_ROOT / "results/new_analysis/260106/Feature Selection/simple_20/feature_summary.csv")
NAME_MAP = load_feature_name_map(FEATURE_MAP_CSV)

# 시각화 코드

In [ ]:
# 경로
OUT_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
)
FIG_DIR = OUT_DIR / "figures/clinic_interpretation"
FIG_DIR.mkdir(parents=True, exist_ok=True)

LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]


# 모델/데이터 로드 + SHAP 계산
def load_best_pipeline(label: str):
    """run_for_label에서 저장한 통합 파이프라인(.pkl) 로드"""
    path = OUT_DIR / f"{label}__best_model.pkl"
    if not path.exists():
        raise FileNotFoundError(path)
    return joblib.load(path)


def load_feature_names_transformed(label: str):
    """전처리 후 (원핫 포함) 최종 입력 피처명 (예쁘게 정리된)"""
    p = OUT_DIR / f"{label}__transformed_feature_names.json"
    if p.exists():
        return json.load(open(p, "r"))
    # 없으면 파이프라인에서 추출
    pipe = load_best_pipeline(label)
    preproc = pipe.named_steps["preprocessor"]
    try:
        names = list(preproc.get_feature_names_out())
    except Exception:
        names = [
            f"f{i}"
            for i in range(
                preproc.transform(
                    pd.DataFrame([0], columns=pipe.feature_names_in_)
                ).shape[1]
            )
        ]
    return names


def load_train_test(label: str):
    """최종 코드에서 저장한 test_predictions.csv(확률/정답) + 모델로부터 X_te 변환"""
    # (1) 확률/정답
    pred_csv = OUT_DIR / f"{label}__test_predictions.csv"
    df_pred = pd.read_csv(pred_csv)  # columns: [label, y_true, y_prob, y_pred]
    # (2) 파이프라인과 함께 Test 원본 X를 얻으려면, 모델링 단계에서 X_te를 저장해두는 게 베스트.
    # 여기서는 파이프라인에 들어가기 전 원본 X_te를 아직 안 저장했다고 가정 → 아래 방식 사용:
    #   - best_model에서 preprocessor 입력 칼럼 순서를 알아내고
    #   - 같은 순서로 test csv에서 읽어서 transform
    # 원본 test csv 경로는 기존 dict와 동일 규칙을 그대로 활용하면 가장 안전.
    # (필요시 알맞게 수정)
    IMP_DIR = Path(
        str(PROJECT_ROOT / "data/processed_imp/260106_split_corr_LLM_ADER/imputation/simple_imput")
    )
    test_path = IMP_DIR / f"simple_{label}_test.csv"
    raw = pd.read_csv(test_path)

    # 타겟 컬럼 제거
    y = raw[label].astype(int)
    # run_for_label에서 사용된 “원본 피처 리스트” 저장 파일을 불러옴
    feats_json = OUT_DIR / f"{label}__features_final.json"
    feats = (
        json.load(open(feats_json, "r"))
        if feats_json.exists()
        else [c for c in raw.columns if c != label]
    )
    X = raw[[c for c in feats if c in raw.columns]].copy()

    return X, y, df_pred


def shap_on_test(label: str, max_bg=200, force_tree_interventional=True):
    """Test셋 SHAP (확률기준) 계산해서 (Explanation, 변환후X, 피처명) 반환"""
    pipe = load_best_pipeline(label)
    preproc = pipe.named_steps["preprocessor"]
    clf = pipe.named_steps["clf"]

    X_raw, y_true, df_pred = load_train_test(label)
    # transform
    X_te_t = preproc.transform(X_raw)
    feat_names = load_feature_names_transformed(label)

    # background (train 일부 저장 안했으면 test 일부 사용 – OK)
    rng = np.random.RandomState(42)
    bg_idx = rng.choice(
        X_te_t.shape[0], size=min(max_bg, X_te_t.shape[0]), replace=False
    )
    X_bg_t = X_te_t[bg_idx]

    # explainer
    try:
        if force_tree_interventional:
            explainer = shap.TreeExplainer(
                clf,
                data=X_bg_t,
                feature_perturbation="interventional",
                model_output="probability",
            )
        else:
            explainer = shap.Explainer(clf, X_bg_t)
        expl = explainer(X_te_t)
    except Exception:
        expl = shap.Explainer(clf, X_bg_t)(X_te_t)

    # 다중출력 (n,p,2)인 경우 양성클래스만 취함
    vals = np.array(expl.values)
    if vals.ndim == 3 and vals.shape[2] == 2:
        expl = expl[:, :, 1]

    # feature names 보정
    try:
        expl.feature_names = feat_names
    except Exception:
        pass

    return expl, X_te_t, y_true, df_pred

# Feature Upset Plot

In [ ]:
# UpSet Plot (윈도우별 Top-K SHAP 교집합/합집합)
from collections import defaultdict

try:
    from upsetplot import UpSet, from_contents

    HAS_UPSETPLOT = True
except Exception:
    HAS_UPSETPLOT = False
    print("upsetplot 미설치. pip install upsetplot 권장합니다.")


def topk_features_by_label(label: str, K=10):
    expl, *_ = shap_on_test(label)
    mean_abs = np.abs(np.array(expl.values)).mean(axis=0)
    feat = np.array(expl.feature_names)
    order = np.argsort(mean_abs)[::-1][:K]
    return list(feat[order])


def collect_topk_across_labels(labels, K=10):
    topk_map = {}
    for lb in labels:
        try:
            topk_map[lb] = set(topk_features_by_label(lb, K=K))
        except Exception as e:
            print(f"[WARN] {lb}: {e}")
    return topk_map  # {label: set(features)}


def plot_upset_across_labels(labels, K=10, out_png=None):
    if not HAS_UPSETPLOT:
        print("upsetplot이 없어서 스킵합니다.")
        return
    contents = collect_topk_across_labels(labels, K=K)
    upset_data = from_contents(contents)
    plt.figure(figsize=(8, 6))
    UpSet(upset_data, subset_size="count").plot()
    plt.suptitle(f"Top-{K} features overlap across outcomes")
    if out_png:
        plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
# # 사용
# plot_upset_across_labels(
#     LABELS,
#     K=10,
#     out_png=str(PROJECT_ROOT / "results/step2_modeling/Final_modeling/figures/Feature/UpSet_top10_across_windows.png"),
# )

# SHAP t-SNE

In [ ]:
# SHAP t-SNE (피처 중요도 공간 → 환자별 패턴 시각화)
import seaborn as sns


def shap_tsne(label: str, perplexity=30, random_state=42, out_png=None):
    expl, X_te_t, y_true, df_pred = shap_on_test(label)
    vals = np.array(expl.values)  # (n, p)
    # 스케일링은 보통 불필요. 값의 스케일이 크게 다르면 정규화 추가 고려.
    tsne = TSNE(
        n_components=2,
        random_state=random_state,
        perplexity=min(perplexity, max(5, vals.shape[0] // 4)),
    )
    Z = tsne.fit_transform(vals)

    dfz = pd.DataFrame(
        {
            "tsne1": Z[:, 0],
            "tsne2": Z[:, 1],
            "y_true": y_true.values if hasattr(y_true, "values") else y_true,
            "y_prob": df_pred["y_prob"].values,
        }
    )

    plt.figure(figsize=(6, 5))
    sc = plt.scatter(dfz.tsne1, dfz.tsne2, c=dfz.y_prob, s=18, alpha=0.8)
    plt.colorbar(sc, label="Predicted risk (prob.)")
    sns.kdeplot(
        x=dfz.tsne1, y=dfz.tsne2, levels=3, color="k", linewidths=0.6, alpha=0.3
    )
    plt.title(f"SHAP t-SNE (patient-level) – {label}")
    plt.xlabel("t-SNE 1")
    plt.ylabel("t-SNE 2")
    if out_png:
        plt.savefig(out_png, dpi=200, bbox_inches="tight")
    plt.show()

In [ ]:
# # 사용
# for lb in LABELS:
#     shap_tsne(lb, perplexity=25, out_png=FIG_DIR / f"{lb}__SHAP_tSNE.png")

# individual Force plot

In [ ]:
# SHAP Individual Force Plot (개별 환자)
def save_force_plots(label: str, indices="topk", k=5, out_dir=None):
    expl, X_te_t, y_true, df_pred = shap_on_test(label)
    base = np.mean(getattr(expl, "base_values", 0.0))
    # Force plot은 1개 샘플 단위로 쓰는게 안정적
    # 인덱스 선택
    if indices == "topk":
        order = np.argsort(-df_pred["y_prob"].values)[:k]
        idx_list = list(order)
    elif isinstance(indices, (list, tuple, np.ndarray)):
        idx_list = list(indices)
    else:
        idx_list = [indices]  # 단일 int

    out_dir = Path(out_dir or FIG_DIR / f"{label}__forceplots")
    out_dir.mkdir(parents=True, exist_ok=True)

    # 간단한 small summary bar(전체 개요)도 함께 저장 (선택)
    plt.figure()
    shap.plots.bar(expl, max_display=20, show=False)
    plt.tight_layout()
    plt.savefig(out_dir / f"{label}__SHAP_bar_top20.png", dpi=300)
    plt.close()

    for i in idx_list:
        # 한 샘플짜리 Explanation 만들기
        single = expl[i : i + 1]
        # plotly 기반 force plot (HTML 저장 권장)
        fp = shap.plots.force(
            single.base_values,
            single.values,
            feature_names=single.feature_names,
            matplotlib=False,
        )
        # 파일 저장
        html_path = out_dir / f"force_patient_{i}.html"
        shap.save_html(str(html_path), fp)
        print(f"[saved] {html_path}")

In [ ]:
# # 사용: 각 아웃컴 별 상위 위험 5명 force plot 저장
# for lb in LABELS:
#     save_force_plots(lb, indices="topk", k=5)

In [ ]:
# def save_force_plots_static(
#     label: str,
#     indices="topk",
#     k=5,
#     out_dir=None,
#     figsize=(10, 5),
#     dpi=300,
#     as_pdf=False,
# ):
#     # expl: shap.Explanation (이미 2D, 양성클래스 슬라이싱은 shap_on_test에서 처리된 상태라고 가정)
#     expl, X_te_t, y_true, df_pred = shap_on_test(label)

#     # 저장 폴더
#     out_dir = Path(out_dir or FIG_DIR / f"{label}__forceplots")
#     out_dir.mkdir(parents=True, exist_ok=True)

#     # 인덱스 선택
#     if indices == "topk":
#         order = np.argsort(-df_pred["y_prob"].values)[:k]
#         idx_list = list(order)
#     elif isinstance(indices, (list, tuple, np.ndarray)):
#         idx_list = list(indices)
#     else:
#         idx_list = [int(indices)]

#     # 각 환자 Force plot 저장 (Matplotlib backend → PNG/PDF)
#     for i in idx_list:
#         single = expl[i]  # Explanation의 i번째 샘플

#         plt.figure(figsize=figsize)
#         try:
#             shap.plots.force(
#                 single, matplotlib=True
#             )  # SHAP가 현재 figure/axes에 그립니다
#         except TypeError:
#             # 일부 버전은 show 인자를 요구할 수 있음
#             shap.plots.force(single, matplotlib=True, show=True)
#         except Exception as e:
#             print(f"[WARN] force plot failed at index {i}: {e}")
#             plt.close()
#             continue

#         stem = f"force_patient_{i}"
#         if as_pdf:
#             plt.savefig(out_dir / f"{stem}.pdf", bbox_inches="tight")
#         else:
#             plt.savefig(out_dir / f"{stem}.png", dpi=dpi, bbox_inches="tight")
#         plt.close()

#     print(f"[OK] Force plots saved to: {out_dir}")


# for lb in LABELS:
#     save_force_plots_static(lb, indices="topk", k=5)

# Umap clustering

In [ ]:
# Patient-level SHAP Clustering (t-SNE / UMAP on SHAP values)
def shap_umap(label: str, n_neighbors=15, min_dist=0.1, random_state=42, out_png=None):
    try:
        import umap.umap_ as UMAP
    except Exception as e:
        print("UMAP 미설치 (pip install umap-learn). t-SNE 결과를 사용하세요.")
        return
    expl, X_te_t, y_true, df_pred = shap_on_test(label)
    vals = np.array(expl.values)
    reducer = UMAP.UMAP(
        n_components=2,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        random_state=random_state,
    )
    Z = reducer.fit_transform(vals)

    dfz = pd.DataFrame(
        {
            "u1": Z[:, 0],
            "u2": Z[:, 1],
            "y_true": y_true.values if hasattr(y_true, "values") else y_true,
            "y_prob": df_pred["y_prob"].values,
        }
    )

    plt.figure(figsize=(6, 5))
    sc = plt.scatter(dfz.u1, dfz.u2, c=dfz.y_prob, s=18, alpha=0.8)
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"SHAP UMAP (patient-level) – {label}")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    if out_png:
        plt.savefig(out_png, dpi=200, bbox_inches="tight")
    plt.show()

In [ ]:
# # 사용
# for lb in LABELS:
#     shap_umap(lb, out_png=FIG_DIR / f"{lb}__SHAP_UMAP.png")

# 개별 WaterFall plot

In [ ]:
def save_waterfall_plots(
    label: str,
    indices="topk",
    k=5,
    out_dir=None,
    max_display=20,
    figsize=(8, 6),
    dpi=300,
    as_pdf=False,
):
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    out_dir = Path(out_dir or FIG_DIR / f"{label}__waterfalls")
    out_dir.mkdir(parents=True, exist_ok=True)

    # 인덱스 선택
    if indices == "topk":
        order = np.argsort(-df_pred["y_prob"].values)[:k]
        idx_list = list(order)
    elif isinstance(indices, (list, tuple, np.ndarray)):
        idx_list = list(indices)
    else:
        idx_list = [int(indices)]

    for i in idx_list:
        single = expl[i]

        plt.figure(figsize=figsize)
        try:
            shap.plots.waterfall(single, max_display=max_display, show=False)
        except TypeError:
            shap.plots.waterfall(single, max_display=max_display)
        except Exception as e:
            print(f"[WARN] waterfall plot failed at index {i}: {e}")
            plt.close()
            continue

        stem = f"waterfall_patient_{i}"
        if as_pdf:
            plt.savefig(out_dir / f"{stem}.pdf", bbox_inches="tight")
        else:
            plt.savefig(out_dir / f"{stem}.png", dpi=dpi, bbox_inches="tight")
        plt.close()

    print(f"[OK] Waterfall plots saved to: {out_dir}")

In [ ]:
# for lb in LABELS:
#     save_waterfall_plots(lb, indices="topk", k=5, max_display=20)

In [ ]:
def save_waterfall_plots(
    label: str,
    indices="topk_prob",  # topk_prob / bottomk_prob / topk_shap / bottomk_shap / fp / fn / list(int)
    k=5,
    out_dir=None,
    max_display=20,
    figsize=(8, 6),
    dpi=300,
    as_pdf=False,
):
    # expl: shap.Explanation (n x p), df_pred: ["y_prob","y_true","y_pred"] 등
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    out_dir = Path(out_dir or FIG_DIR / f"{label}__waterfalls")
    out_dir.mkdir(parents=True, exist_ok=True)

    # --- 인덱스 선택 로직 ---
    if isinstance(indices, (list, tuple, np.ndarray)):  # 직접 인덱스 리스트 제공
        idx_list = list(map(int, indices))

    else:
        mode = str(indices).lower()

        if mode == "topk_prob":  # 예측확률 상위 k
            idx_list = np.argsort(-df_pred["y_prob"].values)[:k].tolist()

        elif mode == "bottomk_prob":  # 예측확률 하위 k
            idx_list = np.argsort(df_pred["y_prob"].values)[:k].tolist()

        elif mode in {"topk_shap", "bottomk_shap"}:  # |SHAP| 합 기준 상/하위 k
            # 각 환자별 총 영향력(절댓값 합)
            shap_mag = np.abs(np.array(expl.values)).sum(axis=1)
            order = (
                np.argsort(-shap_mag) if mode == "topk_shap" else np.argsort(shap_mag)
            )
            idx_list = order[:k].tolist()

        elif mode == "fp":  # False Positive 상위 k (확률 기준)
            cand = df_pred[(df_pred["y_true"] == 0) & (df_pred["y_pred"] == 1)].copy()
            cand = cand.sort_values("y_prob", ascending=False).head(k)
            idx_list = cand.index.tolist()

        elif mode == "fn":  # False Negative 하위 k (확률 기준)
            cand = df_pred[(df_pred["y_true"] == 1) & (df_pred["y_pred"] == 0)].copy()
            cand = cand.sort_values("y_prob", ascending=True).head(k)
            idx_list = cand.index.tolist()

        elif mode.isdigit():  # 단일 인덱스 문자열
            idx_list = [int(mode)]

        else:
            # 기존 호환: "topk" -> topk_prob
            idx_list = np.argsort(-df_pred["y_prob"].values)[:k].tolist()

    # --- 저장 ---
    for i in idx_list:
        single = expl[i]
        plt.figure(figsize=figsize)
        try:
            shap.plots.waterfall(single, max_display=max_display, show=False)
        except TypeError:
            shap.plots.waterfall(single, max_display=max_display)
        stem = f"waterfall_patient_{i}"
        path = out_dir / (f"{stem}.pdf" if as_pdf else f"{stem}.png")
        plt.tight_layout()
        plt.savefig(path, dpi=None if as_pdf else dpi, bbox_inches="tight")
        plt.close()

    print(f"[OK] Waterfall plots saved to: {out_dir}")

In [ ]:
for lb in LABELS:
    save_waterfall_plots(lb, indices="topk_prob", k=5)

In [ ]:
for lb in LABELS:
    save_waterfall_plots(lb, indices="bottomk_prob", k=5)

In [ ]:
# save_waterfall_plots(lb, indices="topk_prob", k=5)
# save_waterfall_plots(lb, indices="bottomk_prob", k=5)
# save_waterfall_plots(lb, indices="topk_shap", k=5)
# save_waterfall_plots(lb, indices="bottomk_shap", k=5)

## ⭐️version 2⭐️

In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable, List, Tuple, Optional
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import roc_curve

In [ ]:
# ====== Paths (EDIT if needed) ======
BASE_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
)
FIG_DIR = BASE_DIR / "figures/clinic_interpretation/personal_waterfall_figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# ====== Helpers ======


def _try_read_youden_threshold(
    label: str, base_dir: Path = BASE_DIR
) -> Optional[float]:
    # support both __ and ___ prior to 'youden'
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


def _top_abs_shap_indices(
    expl: shap.Explanation, k: int, reverse: bool = True
) -> List[int]:
    vals = np.array(expl.values)
    mag = np.abs(vals).sum(axis=1)
    order = np.argsort(-mag if reverse else mag)
    return order[:k].tolist()


def _pick_indices(
    mode: str,
    k: int,
    expl: shap.Explanation,
    df_pred: pd.DataFrame,
    youden_thr: Optional[float],
) -> List[int]:
    mode = str(mode).lower()
    if isinstance(k, (list, tuple, np.ndarray)):
        return list(map(int, k))

    if mode == "topk_prob":
        return np.argsort(-df_pred["y_prob"].values)[:k].tolist()
    if mode == "bottomk_prob":
        return np.argsort(df_pred["y_prob"].values)[:k].tolist()
    if mode == "topk_shap":
        return _top_abs_shap_indices(expl, k, reverse=True)
    if mode == "bottomk_shap":
        return _top_abs_shap_indices(expl, k, reverse=False)
    if mode == "fp":
        cand = df_pred[(df_pred["y_true"] == 0) & (df_pred["y_pred"] == 1)]
        return cand.sort_values("y_prob", ascending=False).head(k).index.tolist()
    if mode == "fn":
        cand = df_pred[(df_pred["y_true"] == 1) & (df_pred["y_pred"] == 0)]
        return cand.sort_values("y_prob", ascending=True).head(k).index.tolist()
    if mode == "tp":
        cand = df_pred[(df_pred["y_true"] == 1) & (df_pred["y_pred"] == 1)]
        return cand.sort_values("y_prob", ascending=False).head(k).index.tolist()
    if mode == "tn":
        cand = df_pred[(df_pred["y_true"] == 0) & (df_pred["y_pred"] == 0)]
        return cand.sort_values("y_prob", ascending=True).head(k).index.tolist()
    if mode == "borderline_k":
        thr = (
            youden_thr
            if youden_thr is not None
            else _compute_youden_from_preds(
                df_pred["y_true"].values, df_pred["y_prob"].values
            )
        )
        df_tmp = df_pred.copy()
        df_tmp["dist"] = (df_tmp["y_prob"] - thr).abs()
        # pick closest k, but keep balance across sides if possible
        topk = df_tmp.sort_values("dist").head(max(k, 2))
        # try to split by side
        above = topk[topk["y_prob"] >= thr].head((k + 1) // 2)
        below = topk[topk["y_prob"] < thr].head(k // 2)
        idx_list = list(pd.concat([above, below]).index.unique())
        # if not enough both sides, just fill with nearest
        if len(idx_list) < k:
            idx_list = df_tmp.sort_values("dist").head(k).index.tolist()
        return idx_list
    if mode == "top_change":
        # large SHAP magnitude but mid-range probability → clear structure with non-extreme risk
        vals = np.array(expl.values)
        mag = np.abs(vals).sum(axis=1)
        q1, q2 = 0.35, 0.65
        mid = df_pred[(df_pred["y_prob"] >= q1) & (df_pred["y_prob"] <= q2)]
        if len(mid) < k:
            mid = df_pred.copy()  # fallback
        order = mid.assign(_mag=mag[mid.index]).sort_values("_mag", ascending=False)
        return order.head(k).index.tolist()

    if mode.isdigit():
        return [int(mode)]

    # default: topk_prob
    return np.argsort(-df_pred["y_prob"].values)[:k].tolist()


def _extract_top_features_for_row(
    expl_row: shap.Explanation, max_display: int, feature_names: List[str]
) -> pd.DataFrame:
    vals = np.array(expl_row.values).reshape(-1)
    order = np.argsort(-np.abs(vals))[:max_display]
    feats = [feature_names[i] for i in order]
    shap_vals = vals[order]
    # Try to fetch original feature values if present (can be None)
    try:
        data_vals = np.array(expl_row.data).reshape(-1)[order]
    except Exception:
        data_vals = [np.nan] * len(order)
    return pd.DataFrame(
        {
            "rank": np.arange(1, len(order) + 1),
            "feature": feats,
            "feature_value": data_vals,
            "shap_value": shap_vals,
        }
    )

In [ ]:
# ====== Main function ======
def save_waterfall_plots_pro(
    label: str,
    indices: "str|Iterable[int]" = "topk_prob",
    k: int = 6,
    out_dir: Optional[str | Path] = None,
    max_display: int = 20,
    figsize: Tuple[float, float] = (8, 6),
    dpi: int = 300,
    as_pdf: bool = False,
    xlim_mode: str = "per_plot",  # 'fixed' or 'per_plot'
    fixed_xlim: Optional[Tuple[float, float]] = None,
):
    """Save a batch of individual waterfall plots with robust case selection.
    Returns a summary DataFrame of saved cases.
    """
    # ---- fetch SHAP & predictions
    expl, X_te_t, y_true, df_pred = shap_on_test(label)
    out_dir = Path(out_dir or (FIG_DIR / f"{label}__waterfalls_pro"))
    out_dir.mkdir(parents=True, exist_ok=True)

    # Ensure prediction columns present
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= 0.5).astype(int))

    # Youden threshold (optional for modes)
    ythr = _try_read_youden_threshold(label)

    # pick index list
    idx_list = _pick_indices(indices, k, expl, df_pred, ythr)

    # feature names
    if getattr(expl, "feature_names", None) is not None:
        feat_names = list(expl.feature_names)
    else:
        feat_names = [f"f{i}" for i in range(np.array(expl.values).shape[1])]

    # Determine fixed xlim if requested
    global_xlim = None
    if xlim_mode == "fixed" and fixed_xlim is None:
        # use 95th percentile of |SHAP| sum as symmetric bound
        mags = np.abs(np.array(expl.values)).sum(axis=1)
        L = float(np.percentile(mags, 95))
        global_xlim = (-L, +L)
    elif fixed_xlim is not None:
        global_xlim = fixed_xlim

    rows = []

    for i in idx_list:
        row = expl[i]
        y_t = (
            int(df_pred.loc[i, "y_true"])
            if "y_true" in df_pred.columns
            else int(np.asarray(y_true)[i])
        )
        y_p = (
            int(df_pred.loc[i, "y_pred"])
            if "y_pred" in df_pred.columns
            else int((df_pred.loc[i, "y_prob"]) >= 0.5)
        )
        y_prob = (
            float(df_pred.loc[i, "y_prob"]) if "y_prob" in df_pred.columns else np.nan
        )

        # --- plot
        plt.figure(figsize=figsize)
        try:
            ax = shap.plots.waterfall(row, max_display=max_display, show=False)
        except Exception:
            ax = shap.plots.waterfall(row, max_display=max_display)
        ax = plt.gca()

        # title
        thr_txt = f", thr={ythr:.3f}" if ythr is not None else ""
        plt.title(
            f"{label} | id={i} | y_true={y_t}, y_pred={y_p}, y_prob={y_prob:.3f}{thr_txt}"
        )

        # optional fixed xlim for comparability
        if global_xlim is not None:
            ax.set_xlim(global_xlim)

        stem = f"waterfall_patient_{i}"
        fpath = out_dir / (f"{stem}.pdf" if as_pdf else f"{stem}.png")
        plt.tight_layout()
        plt.savefig(fpath, dpi=None if as_pdf else dpi, bbox_inches="tight")
        plt.close()

        # extract top contributors CSV
        df_top = _extract_top_features_for_row(
            row, max_display=max_display, feature_names=feat_names
        )
        df_top.to_csv(out_dir / f"{stem}__top{max_display}.csv", index=False)

        # keep a summary row
        rows.append(
            {
                "idx": i,
                "file": str(fpath),
                "y_true": y_t,
                "y_pred": y_p,
                "y_prob": y_prob,
                "youden_thr": ythr,
            }
        )

    summary = pd.DataFrame(rows)
    summary.to_csv(out_dir / f"summary__{label}__{indices}.csv", index=False)
    print(f"[OK] saved {len(rows)} plots → {out_dir}")
    return summary

In [ ]:
# ====== Convenience batch runner ======


def run_personal_waterfalls_batch(
    label: str,
    base_outdir: str | Path,
    youden_cases_k: int = 6,
    tp_k: int = 5,
    fn_k: int = 5,
    max_display: int = 20,
):
    base_outdir = Path(base_outdir)
    # 1) Youden-near borderline
    out1 = base_outdir / label / "borderline"
    save_waterfall_plots_pro(
        label=label,
        indices="borderline_k",
        k=youden_cases_k,
        out_dir=out1,
        max_display=max_display,
        xlim_mode="fixed",
    )
    # 2) High-risk True Positives
    out2 = base_outdir / label / "true_positive"
    save_waterfall_plots_pro(
        label=label,
        indices="tp",
        k=tp_k,
        out_dir=out2,
        max_display=max_display,
        xlim_mode="per_plot",
    )
    # 3) False Negatives
    out3 = base_outdir / label / "false_negative"
    save_waterfall_plots_pro(
        label=label,
        indices="fn",
        k=fn_k,
        out_dir=out3,
        max_display=max_display,
        xlim_mode="per_plot",
    )

In [ ]:
# ====== Multi-outcome loop ======


def run_all_labels_waterfalls(
    labels: List[str],
    base_outdir: str | Path,
    youden_cases_k: int = 6,
    tp_k: int = 5,
    fn_k: int = 5,
    max_display: int = 20,
):
    base_outdir = Path(base_outdir)
    for lb in labels:
        print(f"\n>>> Running personal waterfall for {lb}")
        run_personal_waterfalls_batch(
            label=lb,
            base_outdir=base_outdir,
            youden_cases_k=youden_cases_k,
            tp_k=tp_k,
            fn_k=fn_k,
            max_display=max_display,
        )

In [ ]:
# labels_all = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]
# run_all_labels_waterfalls(
#     labels=labels_all,
#     base_outdir=str(PROJECT_ROOT / "results/step2_modeling/251127_Final_modeling_globalunion25/figures/clinic_interpretation/personal_waterfall_figures"),
#     youden_cases_k=6,
#     tp_k=5,
#     fn_k=5,
#     max_display=20,
# )

## UMAP + DBSCAN

## version 1

In [ ]:
# --- 추가 import ---
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
# ---- 경로
MODEL_DIR = str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
OUT_ROOT = Path(MODEL_DIR) / "figures/clinic_interpretation/PatientClusters"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# --------------------------
# ★ UMAP + DBSCAN (적응형 튜닝) ★
# --------------------------
def plot_cluster_heatmap(
    vals, feat_names, labels, topN, out_path, cmap: str = "rocket"
):
    from textwrap import fill

    # --- 유효 클러스터만 추출 ---
    clusters = [c for c in sorted(np.unique(labels)) if c != -1]
    if not clusters:
        return

    # --- 클러스터별 평균 |SHAP| ---
    mean_abs = []
    for c in clusters:
        mask = labels == c
        mean_abs.append(np.abs(vals[mask]).mean(axis=0))
    M = np.vstack(mean_abs)

    # --- 전체에서 상위 topN 피처 선택 ---
    top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    # --- 각 클러스터 행 정규화 (상대적 중요도) ---
    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max

    # --- plotting data 전치 (features → y축, clusters → x축) ---
    M_plot = M_disp.T
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]  # 자동 줄바꿈

    # --- figure 크기 계산 (적절히 압축) ---
    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        M_plot,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )

    # --- 스타일 조정 ---
    ax.set_title(
        "Cluster × Feature importance",  # (row-normalized)
        fontsize=13,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features by mean |SHAP|", fontsize=10, labelpad=6)

    # 🔹 x축: 글씨 겹침 방지 (회전 + 작은 폰트)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    # 🔹 y축: 글씨 크기 축소 및 패딩
    ax.tick_params(axis="y", labelsize=8, pad=4)

    # 🔹 컬러바 폰트도 줄이기
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=9)

    plt.tight_layout()
    plt.savefig(out_path, dpi=300, bbox_inches="tight", pad_inches=0.15)
    plt.close()


def _standardize(X: np.ndarray) -> np.ndarray:
    X = (X - X.mean(0)) / (X.std(0) + 1e-9)
    return X


def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    """UMAP latent space에서 k-distance의 pct-백분위를 eps 초기값으로 사용."""
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    # 각 점에서 k번째 이웃까지의 거리(자기 자신 제외)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps


def cluster_umap_dbscan_tuned(
    vals: np.ndarray,
    random_state: int = 42,
    # UMAP(클러스터용) 세팅
    umap_n_components: int = 12,
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "cosine",
    # DBSCAN 초기값
    min_samples: int = 6,
    eps: float | None = None,  # None이면 k-distance로 추정
    # 적응형 종료 조건
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    V = _standardize(vals)

    # 1) UMAP 고차원 임베딩(클러스터링용)
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # eps 초기값이 없으면 k-distance 기반으로 추정
    if eps is None:
        eps = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.85)

    trial = 0
    labels = None
    info_hist = []

    # 2) 적응형 루프
    while trial < max_trials:
        db = DBSCAN(eps=eps, min_samples=min_samples, metric="euclidean", n_jobs=-1)
        labels = db.fit_predict(X_hi)

        n = len(labels)
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial=trial,
                eps=eps,
                min_samples=min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        # 종료 조건
        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        # 노이즈가 너무 많다 → eps ↑ (이웃 더 많이 포함), min_samples ↓
        if noise_ratio > max_noise_ratio:
            eps *= 1.20
            if min_samples > 3:
                min_samples -= 1
        # 군집이 너무 적다(과병합) → eps ↓ (더 촘촘하게)
        elif n_clusters < target_min_clusters:
            eps *= 0.88
        else:
            # 애매하면 소폭 완화
            eps *= 1.05

        trial += 1

    # 3) 시각화용 2D UMAP
    um_2d = UMAP.UMAP(
        n_components=2,
        n_neighbors=max(umap_n_neighbors, 15),
        min_dist=max(umap_min_dist, 0.05),
        metric=umap_metric,
        random_state=random_state,
    )
    Z2 = um_2d.fit_transform(V)

    info = dict(
        final_eps=eps,
        final_min_samples=min_samples,
        trials=trial,
        history=info_hist,
        n_clusters=len(set(labels) - {-1}),
        n_noise=int(np.sum(labels == -1)),
    )
    return labels, Z2, info

In [ ]:
# def run_cluster_analysis_for_label(
#     label: str,
#     algo: str = "dbscan_tuned",
#     k=None,
#     topN: int = 10,
#     save_representatives: bool = True,
# ):
#     print(f"\n=== {label}: SHAP clustering & summaries (algo={algo}) ===")
#     out_dir = OUT_ROOT / label
#     out_dir.mkdir(parents=True, exist_ok=True)

#     expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

#     # Youden threshold 확보 및 df_pred 보정
#     ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
#     if ythr is None:
#         ythr = _compute_youden_from_preds(
#             np.asarray(y_true).astype(int), np.asarray(y_prob)
#         )
#     if "y_true" not in df_pred.columns:
#         df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
#     if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
#         df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

#     # --- 군집화 ---
#     if algo == "dbscan_tuned":
#         labels, Z, info = cluster_umap_dbscan_tuned(vals)
#         print(
#             f"[DBSCAN tuned] clusters={info['n_clusters']}, noise={info['n_noise']}, "
#             f"eps={info['final_eps']:.3f}, min_samples={info['final_min_samples']}, "
#             f"trials={info['trials']}"
#         )
#         # 2D 시각화 (cluster 색)
#         uniq = sorted(np.unique(labels))
#         cmap = plt.get_cmap("tab20")
#         color_map = {c: ("#bbbbbb" if c == -1 else cmap(c % 20)) for c in uniq}
#         colors = [color_map[c] for c in labels]
#         plt.figure(figsize=(7.2, 6.2))
#         plt.scatter(Z[:, 0], Z[:, 1], c=colors, s=18, alpha=0.9)
#         plt.title(
#             f"UMAP(2D) + DBSCAN — {info['n_clusters']} clusters, noise={info['n_noise']}"
#         )
#         plt.xlabel("UMAP-1")
#         plt.ylabel("UMAP-2")
#         plt.tight_layout()
#         plt.savefig(out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300)
#         plt.close()

#         # 위험도 색
#         plt.figure(figsize=(7.2, 6.2))
#         sc = plt.scatter(Z[:, 0], Z[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
#         plt.colorbar(sc, label="Predicted risk (prob.)")
#         plt.title(f"UMAP(2D) colored by risk — {label}")
#         plt.xlabel("UMAP-1")
#         plt.ylabel("UMAP-2")
#         plt.tight_layout()
#         plt.savefig(out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300)
#         plt.close()

#         cluster_ids = labels

#     elif algo == "hdbscan_tuned":
#         # ← 너가 이미 만든 HDBSCAN 버전 그대로 사용
#         labels, probs, Z, info = cluster_umap_hdbscan_tuned(vals)
#         cluster_ids = labels
#         # (플롯 저장은 기존 코드 그대로)

#     else:  # KMeans 대안
#         vals_std = (vals - vals.mean(0)) / (vals.std(0) + 1e-9)
#         kk = k or pick_k_by_silhouette(vals_std, k_min=2, k_max=6, random_state=42)
#         km = KMeans(n_clusters=kk, random_state=42, n_init=20)
#         cluster_ids = km.fit_predict(vals_std)
#         um = UMAP.UMAP(
#             n_components=2,
#             n_neighbors=30,
#             min_dist=0.1,
#             metric="cosine",
#             random_state=42,
#         )
#         Z = um.fit_transform(vals_std)
#         plt.figure(figsize=(7.2, 6.2))
#         sc = plt.scatter(Z[:, 0], Z[:, 1], c=cluster_ids, s=18, alpha=0.9, cmap="tab10")
#         plt.colorbar(sc, label="Cluster ID")
#         plt.title(f"UMAP(2D) + KMeans — k={kk}")
#         plt.tight_layout()
#         plt.savefig(out_dir / f"{label}__UMAP2D_KMeans.png", dpi=300)
#         plt.close()

#     # --- 요약/히트맵/대표 waterfall (그대로) ---
#     df_summary = summarize_clusters(
#         vals, feat_names, y_prob, y_true, cluster_ids, topN=topN
#     )
#     df_summary.to_csv(out_dir / f"{label}__cluster_summary_{algo}.csv", index=False)
#     plot_cluster_heatmap(
#         vals,
#         feat_names,
#         cluster_ids,
#         topN=topN,
#         out_path=out_dir / f"{label}__heatmap_top{topN}_{algo}.png",
#     )

#     if save_representatives:
#         reps = pick_representatives(y_prob, cluster_ids)
#         rep_dir = out_dir / f"representatives_{algo}"
#         for rep in reps:
#             c = rep["cluster"]
#             for tag in ["median_idx", "max_idx", "min_idx"]:
#                 i = rep[tag]
#                 name_tag = f"cluster{c}__{tag.replace('_idx','')}"
#                 save_local_plots(
#                     label,
#                     expl,
#                     i,
#                     rep_dir / f"cluster_{c}",
#                     feat_names=feat_names,
#                     max_display=15,
#                     df_pred=df_pred,
#                     youden_thr=ythr,
#                     name_tag=name_tag,
#                 )

#     print(f"-> saved: {out_dir}")
#     return df_summary, cluster_ids

In [ ]:
# all_cluster_summaries = []
# for lb in LABELS:
#     try:
#         df_sum, labels = run_cluster_analysis_for_label(
#             lb, algo="dbscan_tuned", topN=10, save_representatives=True
#         )
#         df_sum.insert(0, "label", lb)
#         all_cluster_summaries.append(df_sum)
#     except Exception as e:
#         print(f"[WARN] {lb}: {e}")

# if all_cluster_summaries:
#     all_summ = pd.concat(all_cluster_summaries, ignore_index=True)
#     all_summ.to_csv(
#         OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.csv", index=False
#     )
#     print("✅ saved:", OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.csv")

## version 2

In [ ]:
# ============================================
from __future__ import annotations

import re, json, time
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from scipy.stats import chi2_contingency, kruskal
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# --- UMAP
try:
    import umap.umap_ as UMAP

    HAS_UMAP = True
except Exception:
    HAS_UMAP = False

# =============== 경로/전역 ===============
MODEL_DIR = str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
OUT_ROOT = Path(MODEL_DIR) / "figures/clinic_interpretation/PatientClusters"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

# 없으면 기본 라벨
if "LABELS" not in globals():
    LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# ---- 가독명 매핑 (없을 때 fallback)
if "clean_name" not in globals():

    def clean_name(name: str) -> str:
        name = re.sub(r"^[^_]+__", "", str(name))
        name = re.sub(r"^(num_|cat_|code_)", "", name)
        return name

In [ ]:
# =============== 유틸 ===============
def _standardize(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    X = np.nan_to_num(
        X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
    )
    return (X - X.mean(0)) / (X.std(0) + 1e-9)


def save_png_svg(figpath: Path, **savefig_kwargs):
    plt.savefig(figpath, **savefig_kwargs)
    try:
        plt.savefig(
            figpath.with_suffix(".svg"),
            bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
        )
    except Exception:
        pass
    plt.close()


# =============== Youden 임계값 ===============
from sklearn.metrics import roc_curve


def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


# =============== SHAP 로딩 ===============
def get_shap_matrix(label: str):
    """
    Returns:
      expl: shap.Explanation (n, p)
      vals: ndarray (n, p)
      feat_names: list[str] length p (cleaned)
      y_prob: ndarray (n,)
      y_true: ndarray (n,)
      X_te_t: ndarray (n, p_after_preproc)
      df_pred: DataFrame with y_prob[, y_true, y_pred]
    """
    # ⚠️ 기존 프로젝트의 함수 가정
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    vals = np.asarray(expl.values, dtype=float)
    vals = np.nan_to_num(
        vals,
        nan=0.0,
        posinf=np.finfo(float).max / 1e6,
        neginf=-np.finfo(float).max / 1e6,
    )

    if getattr(expl, "feature_names", None) is not None:
        raw_names = list(expl.feature_names)
    else:
        raw_names = [f"f{i}" for i in range(vals.shape[1])]
    feat_names = [clean_name(n) for n in raw_names]

    y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
    y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
    return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# =============== k-distance & eps 추정 ===============
def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps, kth


def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
    plt.figure(figsize=(4, 3))
    plt.plot(np.arange(len(kth_sorted)), kth_sorted)
    plt.xlabel("Points (sorted)")
    plt.ylabel(f"{k}-NN distance")
    plt.title(f"k-distance curve (k={k})")
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 히트맵 ===============
def _agg_abs(values: np.ndarray, how="mean"):
    return (
        np.abs(values).mean(axis=0)
        if how == "mean"
        else np.mean(np.abs(values), axis=0)
    )


def plot_cluster_heatmap(
    vals,
    feat_names,
    labels,
    topN,
    out_path: Path,
    cmap: str = "rocket",
    agg: str = "mean",
):
    from textwrap import fill

    clusters = [c for c in sorted(np.unique(labels)) if c != -1]  # 노이즈 제외
    if not clusters:
        return

    rows = []
    for c in clusters:
        mask = labels == c
        rows.append(_agg_abs(vals[mask], how=agg))
    M = np.vstack(rows)

    # top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    # M_top = M[:, top_idx]
    # feat_top = [feat_names[i] for i in top_idx]

    global_imp = _agg_abs(vals, how=agg)  # shape (p,)  # median(|SHAP|)
    top_idx = np.argsort(global_imp)[::-1][:topN]

    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max

    M_plot = M_disp.T
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]

    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        M_plot,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",  # row-normalized
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 대표 케이스 ===============
def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:  # 노이즈 제외(원하면 포함 가능)
            continue
        idx = np.where(labels == c)[0]
        probs = y_prob[idx]
        med = np.mean(probs)
        med_idx = idx[np.argmin(np.abs(probs - med))]
        max_idx = idx[np.argmax(probs)]
        min_idx = idx[np.argmin(probs)]
        reps.append(
            {
                "cluster": int(c),
                "mean": int(med_idx),
                "max_idx": int(max_idx),
                "min_idx": int(min_idx),
            }
        )
    return reps


def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:
            continue
        idx = np.where(labels == c)[0]
        d = df_pred.iloc[idx]
        rec = {"cluster": int(c)}
        fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
        fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
        if len(fp):
            rec["fp_idx"] = int(fp["y_prob"].idxmax())
        if len(fn):
            rec["fn_idx"] = int(fn["y_prob"].idxmin())
        reps.append(rec)
    return reps


def save_local_plots(
    label: str,
    expl: shap.Explanation,
    idx: int,
    out_dir: Path,
    feat_names=None,
    max_display=15,
    df_pred: pd.DataFrame | None = None,
    youden_thr: float | None = None,
    name_tag: str | None = None,
):
    out_dir.mkdir(parents=True, exist_ok=True)
    y_t = y_p = None
    y_prob = None
    thr_txt = ""
    if df_pred is not None:
        if "y_true" in df_pred.columns:
            y_t = int(df_pred.loc[idx, "y_true"])
        if "y_prob" in df_pred.columns:
            y_prob = float(df_pred.loc[idx, "y_prob"])
        if "y_pred" in df_pred.columns:
            y_p = int(df_pred.loc[idx, "y_pred"])
        else:
            thr = youden_thr if youden_thr is not None else 0.5
            y_p = (
                int((df_pred.loc[idx, "y_prob"]) >= thr)
                if "y_prob" in df_pred.columns
                else None
            )
        if youden_thr is not None:
            thr_txt = f", thr={youden_thr:.3f}"

    # waterfall
    plt.figure(figsize=(10, 8))
    try:
        shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
    except Exception:
        row = expl[idx]
        shap.plots.waterfall(row, show=False, max_display=max_display)
    title_bits = [f"{label} | id={idx}"]
    if y_t is not None:
        title_bits.append(f"y_true={y_t}")
    if y_p is not None:
        title_bits.append(f"y_pred={y_p}")
    if y_prob is not None:
        title_bits.append(f"y_prob={y_prob:.3f}")
    if name_tag:
        title_bits.append(f"[{name_tag}]")
    ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
    plt.title(ttl)
    plt.tight_layout()
    save_png_svg(
        out_dir
        / (
            f"{label}__waterfall_id{idx}"
            + (f"__{name_tag}" if name_tag else "")
            + ".png"
        ),
        dpi=300,
        bbox_inches="tight",
    )

    # force (옵션)
    try:
        plt.figure(figsize=(7, 2.8))
        shap.plots.force(
            expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
        )
        plt.tight_layout()
        save_png_svg(
            out_dir
            / (
                f"{label}__force_id{idx}"
                + (f"__{name_tag}" if name_tag else "")
                + ".png"
            ),
            dpi=300,
        )
    except Exception:
        plt.close()


# =============== DBSCAN 통계/요약 ===============
def add_dbscan_stats(
    out_dir: Path,
    label: str,
    y_true: np.ndarray,
    y_prob: np.ndarray,
    labels: np.ndarray,
):
    lab = np.asarray(labels)
    clusters = [c for c in np.unique(lab) if c != -1]
    rows = []
    for c in clusters:
        m = lab == c
        rows.append(
            dict(
                cluster=int(c),
                n=int(m.sum()),
                event_rate=float(np.mean(y_true[m] == 1)),
                mean_y_prob=float(np.mean(y_prob[m])),
            )
        )
    df = pd.DataFrame(rows).sort_values("n", ascending=False)

    chi_p = np.nan
    kw_p = np.nan
    if len(clusters) >= 2:
        table = [
            [
                int(np.sum((lab == c) & (y_true == 1))),
                int(np.sum((lab == c) & (y_true == 0))),
            ]
            for c in clusters
        ]
        chi_p = chi2_contingency(table)[1]
        groups = [y_prob[lab == c] for c in clusters]
        kw_p = kruskal(*groups).pvalue

    df["p_event_rate(chi2)"] = chi_p
    df["p_yprob(kruskal)"] = kw_p
    df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
    return df


# =============== 메인 루틴 ===============
def run_cluster_analysis_for_label(
    label: str,
    umap_n_components: int = 12,  # 군집용 UMAP 차원
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "cosine",
    min_samples: int = 6,  # DBSCAN
    eps: float | None = None,
    dbscan_metric: str = "euclidean",
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
    topN: int = 10,
    agg_for_heatmap: str = "mean",
    save_representatives: bool = True,
    random_state: int = 42,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
    out_dir = OUT_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    # 0) 데이터 로드
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # Youden 임계값 & df_pred 보정
    ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
    if ythr is None:
        ythr = _compute_youden_from_preds(
            np.asarray(y_true).astype(int), np.asarray(y_prob)
        )
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

    # 1) 표준화 & UMAP(고차원; 군집용)
    V = _standardize(vals)
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # 2) eps 초기값 (k-distance) & 곡선 저장
    if eps is None:
        eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.85)
        eps = eps0
        save_kdist_plot(
            kth,
            k=min_samples,
            out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
        )

    # 3) DBSCAN 적응형 루프
    trial = 0
    labels = None
    info_hist = []
    cur_min_samples = int(min_samples)
    cur_eps = float(eps)

    while trial < max_trials:
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))

        info_hist.append(
            dict(
                trial=trial,
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        if noise_ratio > max_noise_ratio:
            cur_eps *= 1.20
            if cur_min_samples > 3:
                cur_min_samples -= 1
        elif n_clusters < target_min_clusters:
            cur_eps *= 0.88
        else:
            cur_eps *= 1.05

        trial += 1

    # all-noise 대비: 마지막 완화 시도 1회
    if (labels is None) or np.all(labels == -1):
        cur_eps *= 1.30
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial="final_relax",
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

    # 4) 시각화용 2D UMAP (시각화 전용)
    um_2d = UMAP.UMAP(
        n_components=2,
        n_neighbors=max(umap_n_neighbors, 15),
        min_dist=max(umap_min_dist, 0.05),
        metric=umap_metric,
        random_state=random_state,
    )
    Z2 = um_2d.fit_transform(V)

    # 5) 라벨 매핑 저장(노이즈 포함)
    m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # 6) 2D 플롯 (클러스터 색)
    uniq = sorted(np.unique(labels))
    cmap = plt.get_cmap("tab20")  # tab20
    color_map = {c: ("#bbbbbb" if c == -1 else cmap(c % 20)) for c in uniq}
    colors = [color_map[c] for c in labels]
    plt.figure(figsize=(7.2, 6.2))
    plt.scatter(Z2[:, 0], Z2[:, 1], c=colors, s=18, alpha=0.9)
    title = f"UMAP + DBSCAN ({umap_n_components}D)"
    plt.title(f"{title} — {len(set(labels)-{-1})} clusters, noise={(labels==-1).sum()}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
    )

    # 6-2) 2D 플롯 (위험도 색)
    plt.figure(figsize=(7.2, 6.2))
    sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"UMAP(2D) (colored by risk) — {label}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
    )

    # 7) 클러스터 통계/요약 + 히트맵
    df_stats = add_dbscan_stats(
        out_dir,
        label,
        y_true=np.asarray(y_true),
        y_prob=np.asarray(y_prob),
        labels=np.asarray(labels),
    )
    plot_cluster_heatmap(
        vals,
        feat_names,
        labels,
        topN=topN,
        out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
        cmap="rocket",
        agg=agg_for_heatmap,
    )

    # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
    if save_representatives:
        rep_dir = out_dir / "representatives_dbscan"
        # mean/max/min
        reps = pick_representatives(y_prob, np.asarray(labels))
        for rep in reps:
            c = rep["cluster"]
            for tag in ["mean", "max_idx", "min_idx"]:
                i = rep[tag]
                name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                save_local_plots(
                    label,
                    expl,
                    i,
                    rep_dir / f"cluster_{c}",
                    feat_names=feat_names,
                    max_display=15,
                    df_pred=df_pred,
                    youden_thr=ythr,
                    name_tag=name_tag,
                )
        # FP/FN
        err_reps = pick_error_representatives(df_pred, np.asarray(labels))
        for rep in err_reps:
            c = rep["cluster"]
            for tag in ["fp_idx", "fn_idx"]:
                if tag in rep:
                    i = rep[tag]
                    name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                    save_local_plots(
                        label,
                        expl,
                        i,
                        rep_dir / f"cluster_{c}",
                        feat_names=feat_names,
                        max_display=15,
                        df_pred=df_pred,
                        youden_thr=ythr,
                        name_tag=name_tag,
                    )

    # 9) 메타(JSON)
    meta = dict(
        timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
        algo="dbscan_tuned",
        random_state=random_state,
        umap=dict(
            n_components=umap_n_components,
            n_neighbors=umap_n_neighbors,
            min_dist=umap_min_dist,
            metric=umap_metric,
        ),
        final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
        trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
        n_clusters=int(len(set(labels) - {-1})),
        noise_ratio=float(np.mean(np.asarray(labels) == -1)),
        history=info_hist,
    )
    with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"-> saved: {out_dir}")
    return df_stats, labels

In [ ]:
# # =============== 전체 라벨 반복 실행 ===============
# all_cluster_summaries = []
# for lb in LABELS:
#     try:
#         df_sum, labels = run_cluster_analysis_for_label(
#             lb,
#             umap_n_components=12,
#             umap_n_neighbors=12,
#             umap_min_dist=0.01,
#             umap_metric="cosine",
#             min_samples=6,
#             eps=None,
#             dbscan_metric="euclidean",
#             target_min_clusters=2,
#             max_noise_ratio=0.45,
#             max_trials=5,
#             topN=10,
#             agg_for_heatmap="mean",
#             save_representatives=True,
#             random_state=42,
#         )
#         df_sum.insert(0, "label", lb)
#         all_cluster_summaries.append(df_sum)
#     except Exception as e:
#         print(f"[WARN] {lb}: {e}")

# if all_cluster_summaries:
#     all_summ = pd.concat(all_cluster_summaries, ignore_index=True)
#     out_csv = OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.csv"
#     all_summ.to_csv(out_csv, index=False)
#     print("✅ saved:", out_csv)
#     # Excel(라벨별 시트) 옵션
#     try:
#         with pd.ExcelWriter(
#             OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx"
#         ) as xw:
#             for lb, df_ in all_summ.groupby("label"):
#                 df_.to_excel(xw, sheet_name=lb, index=False)
#         print("✅ saved:", OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx")
#     except Exception as e:
#         print("[WARN] Excel save:", e)

In [ ]:
# ---------- 경로 설정 ----------
csv_path = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters/ALL_labels__cluster_summary_dbscan_tuned.csv")
)
out_path = csv_path.with_name("Table3_Cluster_Summary.xlsx")

# ---------- 데이터 로드 ----------
df = pd.read_csv(csv_path)

# ---------- 전처리 ----------
# (1) Δ 계산: 예측확률 - 실제 event rate 차이
df["Delta(abs)"] = (df["mean_y_prob"] - df["event_rate"]).abs().round(3)
df["Direction"] = df.apply(
    lambda x: (
        "Over (Pred>Event)"
        if x["mean_y_prob"] > x["event_rate"]
        else "Under (Pred<Event)"
    ),
    axis=1,
)

# (2) 도메인/해석 컬럼 자리 만들기 (수동 입력용)
df["Dominant domain"] = ""
df["Key top features"] = ""
df["Interpretation summary"] = ""

# (3) 정렬 및 보기 좋은 라벨링
df = df.rename(
    columns={
        "label": "Outcome (label)",
        "cluster": "Cluster",
        "n": "N",
        "event_rate": "Event rate",
        "mean_y_prob": "Mean predicted prob",
        "p_event_rate(chi2)": "p (event rate, χ²)",
        "p_yprob(kruskal)": "p (pred prob, KW)",
    }
)
df = df[
    [
        "Outcome (label)",
        "Cluster",
        "N",
        "Event rate",
        "Mean predicted prob",
        "Delta(abs)",
        "Direction",
        "Dominant domain",
        "Key top features",
        "Interpretation summary",
        "p (event rate, χ²)",
        "p (pred prob, KW)",
    ]
]

# ---------- Excel 저장 ----------
with pd.ExcelWriter(out_path) as writer:
    for lb, dsub in df.groupby("Outcome (label)"):
        dsub.to_excel(writer, sheet_name=lb, index=False)
    # 전체 통합 시트
    df.to_excel(writer, sheet_name="All_labels_combined", index=False)

print(f"✅ Saved Table 3 summary Excel → {out_path}")

## ⭐️⭐️⭐️ version 3 ⭐️⭐️⭐️

In [ ]:
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.stats import chi2_contingency, kruskal
from scipy.spatial import ConvexHull
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from adjustText import adjust_text

try:
    import umap.umap_ as UMAP

    HAS_UMAP = True
except Exception:
    HAS_UMAP = False

In [ ]:
# =============== 경로/전역 ===============
MODEL_DIR = str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
OUT_ROOT = Path(MODEL_DIR) / "figures/clinic_interpretation/PatientClusters"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
LLM_ALIAS = {
    "Impaired_Social_Function": "Social Function Impairment (LLM)",
    "Religious_Affiliation": "Religious Affiliation (LLM)",
    "Violence_and_Impulsivity": "Aggression/Impulsivity (LLM)",
    "Domestic_Violence": "Domestic Violence (LLM)",
    "Physical_Abuse": "Physical Abuse (LLM)",
    "Divorce": "Divorce Experience (LLM)",
    "Death_of_Family_Member": "Family Loss (LLM)",
    "Emotional_Abuse": "Emotional Abuse (LLM)",
    "Lack_of_Family_Support": "Lack of Family Support (LLM)",
    "Social_Isolation_and_Lack_of_Support": "Social Isolation (LLM)",
    "Psychotic_Symptoms": "Halluc/Delusion/Paranoia (LLM)",
    "Interpersonal_Conflict": "Interpersonal Conflict (LLM)",
    "Exposure_to_Suicide": "Suicide Exposure (LLM)",
    "Alcohol_Use_Problems": "Alcohol Use Issues (LLM)",
    "Sexual_Abuse": "Sexual Victimization (LLM)",
    "Physical_and_Emotional_Neglect": "Neglect (LLM)",
}

def harmonize_columns_for_pipeline(X, pipe):
    """
    파이프라인의 전처리기가 기대하는 컬럼 구성과 순서로 데이터프레임을 재정렬하는 함수
    """
    # 보통 파이프라인의 첫 번째 스텝(전처리기)에서 피처 이름을 가져옵니다.
    try:
        expected_cols = pipe.named_steps["preprocessor"].feature_names_in_
    except AttributeError:
        # 피처 이름이 저장되지 않은 경우 등에 대한 예외 처리
        return X

    # 누락된 컬럼은 0으로 채우고, 순서를 맞춤
    for col in expected_cols:
        if col not in X.columns:
            X[col] = 0

    return X[expected_cols]

def harmonize_with_alias(X, pipe, alias_map=LLM_ALIAS, fill_value=0):
    X = X.copy()

    # 1) alias 컬럼이 있으면, 파이프라인이 기대하는 "원본키" 컬럼로 복사 생성
    for raw_key, pretty_name in alias_map.items():
        if (raw_key not in X.columns) and (pretty_name in X.columns):
            X[raw_key] = X[pretty_name]

    # 2) 기존 harmonize (없는 컬럼 생성/정렬 등)
    X = harmonize_columns_for_pipeline(X, pipe)

    return X

def shap_on_test(label: str, max_bg=200, force_tree_interventional=True):
    pipe = load_best_pipeline(label)
    preproc = pipe.named_steps["preprocessor"]
    clf = pipe.named_steps["clf"]

    # ✅ 이거 반드시 필요
    X_raw, y_true, df_pred = load_train_test(label)

    # ✅ alias -> raw_key 복구 후 harmonize
    X_raw_h = harmonize_with_alias(X_raw, pipe)

    X_te_t = preproc.transform(X_raw_h)
    feat_names = load_feature_names_transformed(label)

    rng = np.random.RandomState(42)
    bg_idx = rng.choice(X_te_t.shape[0], size=min(max_bg, X_te_t.shape[0]), replace=False)
    X_bg_t = X_te_t[bg_idx]

    try:
        if force_tree_interventional:
            explainer = shap.TreeExplainer(
                clf,
                data=X_bg_t,
                feature_perturbation="interventional",
                model_output="probability",
            )
        else:
            explainer = shap.Explainer(clf, X_bg_t)
        expl = explainer(X_te_t)
    except Exception:
        expl = shap.Explainer(clf, X_bg_t)(X_te_t)

    vals = np.array(expl.values)
    if vals.ndim == 3 and vals.shape[2] == 2:
        expl = expl[:, :, 1]

    try:
        expl.feature_names = feat_names
    except Exception:
        pass

    return expl, X_te_t, y_true, df_pred

In [ ]:
def _standardize(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    X = np.nan_to_num(
        X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
    )
    return (X - X.mean(0)) / (X.std(0) + 1e-9)


def save_png_svg(figpath: Path, **savefig_kwargs):
    plt.savefig(figpath, **savefig_kwargs)
    try:
        plt.savefig(
            figpath.with_suffix(".svg"),
            bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
        )
    except Exception:
        pass
    plt.close()


def reorder_labels_by_risk(labels: np.ndarray, y_true: np.ndarray):
    """
    클러스터 라벨을 '실제 발생률(Event Rate)' 오름차순으로 재정렬합니다.
    - Cluster 0: 발생률이 가장 낮은 그룹
    - Cluster N: 발생률이 가장 높은 그룹
    - Cluster -1: 노이즈 (변경 없음)
    """
    # 노이즈 제외한 유니크 라벨
    unique_labels = [c for c in np.unique(labels) if c != -1]

    if not unique_labels:
        return labels

    # (구 라벨, Event Rate) 리스트 생성
    risk_scores = []
    print("\n[Event Rate Reordering Check]")
    for c in unique_labels:
        # 해당 클러스터의 실제 발생률 계산 (Mean of y_true)
        # y_true가 0/1 이진값이라 가정하면 평균이 곧 발생률입니다.
        event_rate = np.mean(y_true[labels == c])
        risk_scores.append((c, event_rate))
        print(f" - Original Cluster {c}: Event Rate = {event_rate:.4f}")

    # Event Rate 기준 오름차순 정렬 (낮음 -> 높음)
    risk_scores.sort(key=lambda x: x[1])

    # 매핑 딕셔너리 생성
    print("-> Mapping Result:")
    mapping = {-1: -1}
    for new_idx, (old_label, rate) in enumerate(risk_scores):
        mapping[old_label] = new_idx
        print(f"   Old {old_label} (Rate {rate:.4f}) -> New {new_idx} (Low to High)")

    # 새로운 라벨 적용
    new_labels = np.array([mapping[x] for x in labels])

    return new_labels


# =============== Youden 임계값 ===============
from sklearn.metrics import roc_curve


def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


# =============== SHAP 로딩 ===============
def get_shap_matrix(label: str):
    """
    Returns:
      expl: shap.Explanation (n, p)
      vals: ndarray (n, p)
      feat_names: list[str] length p (cleaned)
      y_prob: ndarray (n,)
      y_true: ndarray (n,)
      X_te_t: ndarray (n, p_after_preproc)
      df_pred: DataFrame with y_prob[, y_true, y_pred]
    """
    # ⚠️ 기존 프로젝트의 함수 가정
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    vals = np.asarray(expl.values, dtype=float)
    vals = np.nan_to_num(
        vals,
        nan=0.0,
        posinf=np.finfo(float).max / 1e6,
        neginf=-np.finfo(float).max / 1e6,
    )

    if getattr(expl, "feature_names", None) is not None:
        raw_names = list(expl.feature_names)
    else:
        raw_names = [f"f{i}" for i in range(vals.shape[1])]
    feat_names = [clean_name(n) for n in raw_names]

    y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
    y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
    return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# =============== k-distance & eps 추정 ===============
def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps, kth


def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
    plt.figure(figsize=(4, 3))
    plt.plot(np.arange(len(kth_sorted)), kth_sorted)
    plt.xlabel("Points (sorted)")
    plt.ylabel(f"{k}-NN distance")
    plt.title(f"k-distance curve (k={k})")
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 히트맵 ===============
def _agg_abs(values: np.ndarray, how="mean"):
    return (
        np.abs(values).mean(axis=0)
        if how == "mean"
        else np.mean(np.abs(values), axis=0)
    )


def plot_cluster_heatmap(
    vals,
    feat_names,
    labels,
    topN,
    out_path: Path,
    cmap: str = "rocket",
    agg: str = "mean",
):
    """
    반환:
      {
        "raw_all":  DataFrame (clusters × all_features) : 클러스터별 원시 집계 |SHAP|
        "raw_top":  DataFrame (clusters × topN)         : 히트맵에 사용된 TopN의 원시 |SHAP|
        "norm_top": DataFrame (clusters × topN)         : TopN의 행 정규화(상대값)
        "feat_top": List[str]                           : TopN 피처명
        "clusters": np.ndarray                          : 사용된 클러스터 ID (노이즈 -1 제외)
        "fig_raw_path": Path                            : raw heatmap PNG 경로
        "fig_norm_path": Path                           : normalized heatmap PNG 경로(= out_path)
      }
    """
    from textwrap import fill

    # 노이즈(-1) 제외
    clusters = [c for c in sorted(np.unique(labels)) if c != -1]
    if not clusters:
        return None

    # --- (A) 클러스터별 원시 집계 |SHAP|
    rows = []
    for c in clusters:
        mask = labels == c
        rows.append(_agg_abs(vals[mask], how=agg))  # shape: (p,)
    M = np.vstack(rows)  # (n_clusters, n_features)

    # --- (B) 히트맵에서 쓸 TopN 선택 (클러스터 max 기준)
    global_imp = _agg_abs(vals, how=agg)  # 전반적 중요도 (median|SHAP| 또는 mean|SHAP|)
    hetero = M.max(axis=0) - M.min(axis=0)  # 간단한 이질성 점수 (max - min)

    topK_imp = max(topN * 2, 30)  # 후보 풀(전반적 중요도 상위 K)
    cand = np.argsort(global_imp)[::-1][:topK_imp]
    top_idx = cand[
        np.argsort(hetero[cand])[::-1][:topN]
    ]  # 후보 안에서 이질성 큰 순 TopN

    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    ###########################################
    # global_imp = _agg_abs(vals, how=agg)  # shape (p,)  # median(|SHAP|)
    # top_idx = np.argsort(global_imp)[::-1][:topN]

    # M_top = M[:, top_idx]
    # feat_top = [feat_names[i] for i in top_idx]
    ###########################################
    # top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    # M_top = M[:, top_idx]  # raw topN
    # feat_top = [feat_names[i] for i in top_idx]

    # --- (C) 행 정규화(클러스터 내부 상대값)
    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max  # normalized topN

    # --- (D) DataFrame 구성
    idx_names = [f"cluster_{c}" for c in clusters]
    df_raw_all = pd.DataFrame(M, index=idx_names, columns=feat_names)
    df_raw_top = pd.DataFrame(M_top, index=idx_names, columns=feat_top)
    df_norm_top = pd.DataFrame(M_disp, index=idx_names, columns=feat_top)

    # ---------- (E) Normalized Heatmap (기존 파일명 유지) ----------
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]
    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_norm_top.T.values,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",  #  – Row-normalized
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")

    # ---------- (F) Raw Heatmap (새 파일) ----------
    out_path_raw = out_path.with_name(out_path.stem + "__RAW.png")
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_raw_top.T.values,
        cmap=cmap,
        cbar_kws={"label": f"{agg} |SHAP| (raw)"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|) – Raw",
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path_raw, dpi=300, bbox_inches="tight")

    return {
        "raw_all": df_raw_all,
        "raw_top": df_raw_top,
        "norm_top": df_norm_top,
        "feat_top": feat_top,
        "clusters": np.array(clusters),
        "fig_raw_path": out_path_raw,
        "fig_norm_path": out_path,
    }


# =============== 대표 케이스 ===============
def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:  # 노이즈 제외(원하면 포함 가능)
            continue
        idx = np.where(labels == c)[0]
        probs = y_prob[idx]
        med = np.mean(probs)
        med_idx = idx[np.argmin(np.abs(probs - med))]
        max_idx = idx[np.argmax(probs)]
        min_idx = idx[np.argmin(probs)]
        reps.append(
            {
                "cluster": int(c),
                "mean_idx": int(med_idx),
                "max_idx": int(max_idx),
                "min_idx": int(min_idx),
            }
        )
    return reps


def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:
            continue
        idx = np.where(labels == c)[0]
        d = df_pred.iloc[idx]
        rec = {"cluster": int(c)}
        fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
        fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
        if len(fp):
            rec["fp_idx"] = int(fp["y_prob"].idxmax())
        if len(fn):
            rec["fn_idx"] = int(fn["y_prob"].idxmin())
        reps.append(rec)
    return reps


def save_local_plots(
    label: str,
    expl: shap.Explanation,
    idx: int,
    out_dir: Path,
    feat_names=None,
    max_display=15,
    df_pred: pd.DataFrame | None = None,
    youden_thr: float | None = None,
    name_tag: str | None = None,
):
    out_dir.mkdir(parents=True, exist_ok=True)
    y_t = y_p = None
    y_prob = None
    thr_txt = ""
    if df_pred is not None:
        if "y_true" in df_pred.columns:
            y_t = int(df_pred.loc[idx, "y_true"])
        if "y_prob" in df_pred.columns:
            y_prob = float(df_pred.loc[idx, "y_prob"])
        if "y_pred" in df_pred.columns:
            y_p = int(df_pred.loc[idx, "y_pred"])
        else:
            thr = youden_thr if youden_thr is not None else 0.5
            y_p = (
                int((df_pred.loc[idx, "y_prob"]) >= thr)
                if "y_prob" in df_pred.columns
                else None
            )
        if youden_thr is not None:
            thr_txt = f", thr={youden_thr:.3f}"

    # waterfall
    plt.figure(figsize=(10, 8))
    try:
        shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
    except Exception:
        row = expl[idx]
        shap.plots.waterfall(row, show=False, max_display=max_display)
    title_bits = [f"{label} | id={idx}"]
    if y_t is not None:
        title_bits.append(f"y_true={y_t}")
    if y_p is not None:
        title_bits.append(f"y_pred={y_p}")
    if y_prob is not None:
        title_bits.append(f"y_prob={y_prob:.3f}")
    if name_tag:
        title_bits.append(f"[{name_tag}]")
    ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
    plt.title(ttl)
    plt.tight_layout()
    save_png_svg(
        out_dir
        / (
            f"{label}__waterfall_id{idx}"
            + (f"__{name_tag}" if name_tag else "")
            + ".png"
        ),
        dpi=300,
        bbox_inches="tight",
    )

    # force (옵션)
    try:
        plt.figure(figsize=(7, 2.8))
        shap.plots.force(
            expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
        )
        plt.tight_layout()
        save_png_svg(
            out_dir
            / (
                f"{label}__force_id{idx}"
                + (f"__{name_tag}" if name_tag else "")
                + ".png"
            ),
            dpi=300,
        )
    except Exception:
        plt.close()


# =============== DBSCAN 통계/요약 ===============
def add_dbscan_stats(
    out_dir: Path,
    label: str,
    y_true: np.ndarray,
    y_prob: np.ndarray,
    labels: np.ndarray,
):
    lab = np.asarray(labels)
    clusters = [c for c in np.unique(lab) if c != -1]
    rows = []
    for c in clusters:
        m = lab == c
        rows.append(
            dict(
                cluster=int(c),
                n=int(m.sum()),
                event_rate=float(np.mean(y_true[m] == 1)),
                mean_y_prob=float(np.mean(y_prob[m])),
            )
        )
    df = pd.DataFrame(rows).sort_values("n", ascending=False)

    chi_p = np.nan
    kw_p = np.nan
    if len(clusters) >= 2:
        table = [
            [
                int(np.sum((lab == c) & (y_true == 1))),
                int(np.sum((lab == c) & (y_true == 0))),
            ]
            for c in clusters
        ]
        chi_p = chi2_contingency(table)[1]
        groups = [y_prob[lab == c] for c in clusters]
        kw_p = kruskal(*groups).pvalue

    df["p_event_rate(chi2)"] = chi_p
    df["p_yprob(kruskal)"] = kw_p
    df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
    return df


# =============== 메인 루틴 ===============
def run_cluster_analysis_for_label(
    label: str,
    umap_n_components: int = 12,  # 군집용 UMAP 차원
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "euclidean",  # cosine
    min_samples: int = 6,  # DBSCAN
    eps: float | None = None,
    dbscan_metric: str = "euclidean",
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
    topN: int = 10,
    agg_for_heatmap: str = "mean",
    save_representatives: bool = True,
    random_state: int = 42,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
    out_dir = OUT_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    # 0) 데이터 로드
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # Youden 임계값 & df_pred 보정
    ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
    if ythr is None:
        ythr = _compute_youden_from_preds(
            np.asarray(y_true).astype(int), np.asarray(y_prob)
        )
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

    # 1) 표준화 & UMAP(고차원; 군집용)
    # V = _standardize(vals) # 표준화
    V = vals
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # 2) eps 초기값 (k-distance) & 곡선 저장
    if eps is None:
        eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.9)  # 0.85
        eps = eps0
        save_kdist_plot(
            kth,
            k=min_samples,
            out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
        )

    # 3) DBSCAN 적응형 루프
    trial = 0
    labels = None
    info_hist = []
    cur_min_samples = int(min_samples)
    cur_eps = float(eps)

    while trial < max_trials:
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))

        info_hist.append(
            dict(
                trial=trial,
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        if noise_ratio > max_noise_ratio:
            cur_eps *= 1.20
            if cur_min_samples > 3:
                cur_min_samples -= 1
        elif n_clusters < target_min_clusters:
            cur_eps *= 0.88
        else:
            cur_eps *= 1.05

        trial += 1

    # all-noise 대비: 마지막 완화 시도 1회
    if (labels is None) or np.all(labels == -1):
        cur_eps *= 1.30
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial="final_relax",
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

    # 4) 시각화용 2D UMAP (시각화 전용)
    if umap_n_components == 2:
        Z2 = X_hi
    else:
        um_2d = UMAP.UMAP(
            n_components=2,
            n_neighbors=max(umap_n_neighbors, 15),
            min_dist=max(umap_min_dist, 0.05),
            metric=umap_metric,
            random_state=random_state,
        )
        Z2 = um_2d.fit_transform(V)

    # 5) 라벨 매핑 저장(노이즈 포함)
    m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # 6) 2D 플롯 (클러스터 색)
    uniq = sorted(np.unique(labels))
    cmap = plt.get_cmap("tab10")  # tab20
    color_map = {c: ("#bbbbbb" if c == -1 else cmap(c % 20)) for c in uniq}
    colors = [color_map[c] for c in labels]
    plt.figure(figsize=(7.2, 6.2))
    plt.scatter(Z2[:, 0], Z2[:, 1], c=colors, s=18, alpha=0.9)
    title = f"UMAP + DBSCAN ({umap_n_components}D)"
    plt.title(f"{title} — {len(set(labels)-{-1})} clusters, noise={(labels==-1).sum()}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
    )

    # 6-2) 2D 플롯 (위험도 색)
    plt.figure(figsize=(7.2, 6.2))
    sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"UMAP(2D) (colored by risk) — {label}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
    )

    # 7) 클러스터 통계/요약 + 히트맵
    df_stats = add_dbscan_stats(
        out_dir,
        label,
        y_true=np.asarray(y_true),
        y_prob=np.asarray(y_prob),
        labels=np.asarray(labels),
    )
    ret = plot_cluster_heatmap(
        vals,
        feat_names,
        labels,
        topN=topN,
        out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
        cmap="magma_r",  # rocket_r, viridis_r
        agg=agg_for_heatmap,
    )

    # === 수치 저장(요청 파일명 규칙) ===
    if ret is not None:
        # 모든 피처의 원시 |SHAP| (클러스터 × 전체 피처)
        ret["raw_all"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__ALL_FEATURES.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 원시 |SHAP|
        ret["raw_top"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_RAW.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 행 정규화(상대값)
        ret["norm_top"].to_csv(
            out_dir
            / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_ROWNORM.csv",
            index=True,
        )

    # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
    if save_representatives:
        rep_dir = out_dir / "representatives_dbscan"
        # mean/max/min
        reps = pick_representatives(y_prob, np.asarray(labels))
        for rep in reps:
            c = rep["cluster"]
            for tag in ["mean_idx", "max_idx", "min_idx"]:
                i = rep[tag]
                name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                save_local_plots(
                    label,
                    expl,
                    i,
                    rep_dir / f"cluster_{c}",
                    feat_names=feat_names,
                    max_display=15,
                    df_pred=df_pred,
                    youden_thr=ythr,
                    name_tag=name_tag,
                )
        # FP/FN
        err_reps = pick_error_representatives(df_pred, np.asarray(labels))
        for rep in err_reps:
            c = rep["cluster"]
            for tag in ["fp_idx", "fn_idx"]:
                if tag in rep:
                    i = rep[tag]
                    name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                    save_local_plots(
                        label,
                        expl,
                        i,
                        rep_dir / f"cluster_{c}",
                        feat_names=feat_names,
                        max_display=15,
                        df_pred=df_pred,
                        youden_thr=ythr,
                        name_tag=name_tag,
                    )

    # 9) 메타(JSON)
    meta = dict(
        timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
        algo="dbscan_tuned",
        random_state=random_state,
        umap=dict(
            n_components=umap_n_components,
            n_neighbors=umap_n_neighbors,
            min_dist=umap_min_dist,
            metric=umap_metric,
        ),
        final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
        trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
        n_clusters=int(len(set(labels) - {-1})),
        noise_ratio=float(np.mean(np.asarray(labels) == -1)),
        history=info_hist,
    )
    with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"-> saved: {out_dir}")
    return df_stats, labels

In [ ]:
def plot_umap_for_ppt(
    Z2: np.ndarray,
    labels: np.ndarray,
    y_prob: np.ndarray,
    y_true: np.ndarray,  # [NEW] 실제 정답 레이블 추가
    out_path: Path,
    title: str,
):

    # plt.rcParams.update({'font.size': 14})

    # --- [설정] Risk 판단 기준 (Event Rate 기준) ---
    def get_risk_style(event_rate):
        # Event Rate는 보통 확률보다 낮으므로 기준을 조금 조정할 수도 있음 (현행 유지)
        if event_rate < 0.1:
            return "Low", "#4575b4"
        elif event_rate < 0.3:
            return "Moderate", "#7b3294"
        elif event_rate < 0.5:
            return "High", "#f1a340"
        else:
            return "Very High", "#d73027"

    uniq = sorted(np.unique(labels))
    clusters = [c for c in uniq if c != -1]
    n_clusters = len(clusters)

    global_center = np.mean(Z2, axis=0)
    cluster_cmap = plt.get_cmap("tab20" if n_clusters > 10 else "tab10")

    centroids = {}
    for c in clusters:
        centroids[c] = np.mean(Z2[labels == c], axis=0)

    fig, ax = plt.subplots(figsize=(12, 10))
    ax.scatter(
        Z2[:, 0], Z2[:, 1], c="#e0e0e0", s=20, alpha=0.4, edgecolors="none", zorder=0
    )

    label_items = []
    legend_handles = []

    for i, c in enumerate(clusters):
        mask = labels == c
        points = Z2[mask]

        # [변경] 실제 Event Rate 계산
        # y_true가 0/1로 되어 있다고 가정 (1=Event)
        true_vals = y_true[mask]
        event_rate = np.mean(true_vals == 1)

        risk_name, risk_color = get_risk_style(event_rate)  # Event Rate 기준 스타일
        cluster_color = cluster_cmap(i % 20)

        handle = mpatches.Patch(color=cluster_color, label=f"Cluster {c}")
        legend_handles.append(handle)

        if len(points) < 3:
            ax.scatter(
                points[:, 0], points[:, 1], c=cluster_color, s=20, alpha=0.8, zorder=2
            )
            continue

        ax.scatter(
            points[:, 0],
            points[:, 1],
            c=[cluster_color],
            s=20,
            alpha=0.8,
            edgecolors="white",
            linewidth=0.2,
            zorder=2,
        )
        hull = ConvexHull(points)
        hull_coords = points[np.append(hull.vertices, hull.vertices[0])]
        ax.plot(
            hull_coords[:, 0],
            hull_coords[:, 1],
            color=cluster_color,
            linestyle="--",
            linewidth=1.2,
            alpha=0.8,
            zorder=3,
        )
        ax.add_patch(
            mpatches.Polygon(
                hull_coords,
                closed=True,
                fc=cluster_color,
                ec=None,
                alpha=0.08,
                zorder=1,
            )
        )

        # --- 위치 계산 ---
        current_centroid = centroids[c]
        vec_radial = current_centroid - global_center
        vec_repulsion = np.array([0.0, 0.0])
        crowding = 0

        for other_c, other_cent in centroids.items():
            if c == other_c:
                continue
            diff = current_centroid - other_cent
            dist = np.linalg.norm(diff)
            if dist > 1e-4:
                w = 1.0 / (dist**2.0)
                vec_repulsion += (diff / dist) * w
                crowding += w

        final_vec = vec_radial + (vec_repulsion * 4.0)
        if crowding > 0.5 or np.linalg.norm(vec_radial) < 1.0:
            final_vec[1] += (1.0 if i % 2 == 0 else -1.0) * 3.0

        norm = np.linalg.norm(final_vec)
        unit_vec = final_vec / norm if norm > 1e-4 else np.array([0, 1])

        hull_points = points[hull.vertices]
        anchor_point = hull_points[np.argmax(np.dot(hull_points, unit_vec))]

        offset = 2.5 + min(crowding, 3.5)
        text_pos = anchor_point + (unit_vec * offset)

        label_items.append(
            {
                "text_pos": text_pos,
                "anchor_point": anchor_point,
                # [변경] 라벨 텍스트에 Event Rate 표시
                "text_str": f"{risk_name}\n(Event:{event_rate:.2f})",
                "color": risk_color,
            }
        )

    # --- 물리 엔진 ---
    iterations = 50
    min_dist = 3.0
    for _ in range(iterations):
        for i in range(len(label_items)):
            for j in range(i + 1, len(label_items)):
                p1 = label_items[i]["text_pos"]
                p2 = label_items[j]["text_pos"]
                diff = p1 - p2
                dist = np.linalg.norm(diff)
                if dist < min_dist:
                    if dist < 1e-4:
                        force = np.array([0.1, 0.1])
                    else:
                        force = (diff / dist) * (min_dist - dist) * 0.5
                    label_items[i]["text_pos"] += force
                    label_items[j]["text_pos"] -= force

    # --- 축 확장 ---
    all_x = list(Z2[:, 0]) + [item["text_pos"][0] for item in label_items]
    all_y = list(Z2[:, 1]) + [item["text_pos"][1] for item in label_items]
    margin_x = (max(all_x) - min(all_x)) * 0.1
    margin_y = (max(all_y) - min(all_y)) * 0.1
    ax.set_xlim(min(all_x) - margin_x, max(all_x) + margin_x)
    ax.set_ylim(min(all_y) - margin_y, max(all_y) + margin_y)

    # # --- Annotation 그리기 ---
    # for item in label_items:
    #     bbox_props = dict(
    #         boxstyle="round,pad=0.2", fc="white", ec=item["color"], lw=1.5, alpha=0.95
    #     )

    #     ax.annotate(
    #         text=item["text_str"],
    #         xy=(item["anchor_point"][0], item["anchor_point"][1]),
    #         xytext=(item["text_pos"][0], item["text_pos"][1]),
    #         ha="center",
    #         va="center",
    #         fontsize=14,
    #         weight="bold",
    #         color=item["color"],
    #         bbox=bbox_props,
    #         arrowprops=dict(
    #             arrowstyle="-",
    #             color="black",
    #             linewidth=0.8,
    #             shrinkB=4,
    #             connectionstyle="arc3,rad=0.1",
    #         ),
    #         zorder=10,
    #     )

    # 범례 및 타이틀
    if -1 in uniq:
        legend_handles.insert(0, mpatches.Patch(color="#e0e0e0", label="Noise"))

    ax.legend(
        handles=legend_handles,
        title="Clusters",
        bbox_to_anchor=(1.02, 1),
        loc="upper left",
        borderaxespad=0,
        frameon=False,
        fontsize=14,
    )

    noise_pct = (labels == -1).sum() / len(labels) if len(labels) > 0 else 0
    full_title = f"{title}\n(Noise ratio={noise_pct:.1%})"

    ax.set_title(full_title, fontsize=14, weight="bold", pad=20)
    ax.set_xlabel("UMAP-1", fontsize=12, labelpad=10)
    ax.set_ylabel("UMAP-2", fontsize=12, labelpad=10)
    # ax.tick_params(axis='both', which='major', labelsize=12)

    ax.set_xticks([])
    ax.set_yticks([])

    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 유틸 ===============
def _standardize(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    X = np.nan_to_num(
        X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
    )
    return (X - X.mean(0)) / (X.std(0) + 1e-9)


def save_png_svg(figpath: Path, **savefig_kwargs):
    plt.savefig(figpath, **savefig_kwargs)
    try:
        plt.savefig(
            figpath.with_suffix(".svg"),
            bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
        )
    except Exception:
        pass
    plt.close()


# =============== Youden 임계값 ===============
from sklearn.metrics import roc_curve


def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


# =============== SHAP 로딩 ===============
def get_shap_matrix(label: str):
    """
    Returns:
      expl: shap.Explanation (n, p)
      vals: ndarray (n, p)
      feat_names: list[str] length p (cleaned)
      y_prob: ndarray (n,)
      y_true: ndarray (n,)
      X_te_t: ndarray (n, p_after_preproc)
      df_pred: DataFrame with y_prob[, y_true, y_pred]
    """
    # ⚠️ 기존 프로젝트의 함수 가정
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    vals = np.asarray(expl.values, dtype=float)
    vals = np.nan_to_num(
        vals,
        nan=0.0,
        posinf=np.finfo(float).max / 1e6,
        neginf=-np.finfo(float).max / 1e6,
    )

    if getattr(expl, "feature_names", None) is not None:
        raw_names = list(expl.feature_names)
    else:
        raw_names = [f"f{i}" for i in range(vals.shape[1])]
    feat_names = [clean_name(n) for n in raw_names]

    y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
    y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
    return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# =============== k-distance & eps 추정 ===============
def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps, kth


def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
    plt.figure(figsize=(4, 3))
    plt.plot(np.arange(len(kth_sorted)), kth_sorted)
    plt.xlabel("Points (sorted)")
    plt.ylabel(f"{k}-NN distance")
    plt.title(f"k-distance curve (k={k})")
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 히트맵 ===============
def _agg_abs(values: np.ndarray, how="mean"):
    return (
        np.abs(values).mean(axis=0)
        if how == "mean"
        else np.mean(np.abs(values), axis=0)
    )


def plot_cluster_heatmap(
    vals,
    feat_names,
    labels,
    topN,
    out_path: Path,
    cmap: str = "rocket",
    agg: str = "mean",
):
    """
    반환:
      {
        "raw_all":  DataFrame (clusters × all_features) : 클러스터별 원시 집계 |SHAP|
        "raw_top":  DataFrame (clusters × topN)         : 히트맵에 사용된 TopN의 원시 |SHAP|
        "norm_top": DataFrame (clusters × topN)         : TopN의 행 정규화(상대값)
        "feat_top": List[str]                           : TopN 피처명
        "clusters": np.ndarray                          : 사용된 클러스터 ID (노이즈 -1 제외)
        "fig_raw_path": Path                            : raw heatmap PNG 경로
        "fig_norm_path": Path                           : normalized heatmap PNG 경로(= out_path)
      }
    """
    from textwrap import fill

    # 노이즈(-1) 제외
    clusters = [c for c in sorted(np.unique(labels)) if c != -1]
    if not clusters:
        return None

    # --- (A) 클러스터별 원시 집계 |SHAP|
    rows = []
    for c in clusters:
        mask = labels == c
        rows.append(_agg_abs(vals[mask], how=agg))  # shape: (p,)
    M = np.vstack(rows)  # (n_clusters, n_features)

    # --- (B) 히트맵에서 쓸 TopN 선택 (클러스터 max 기준)
    global_imp = _agg_abs(vals, how=agg)  # 전반적 중요도 (median|SHAP| 또는 mean|SHAP|)
    hetero = M.max(axis=0) - M.min(axis=0)  # 간단한 이질성 점수 (max - min)

    topK_imp = max(topN * 2, 30)  # 후보 풀(전반적 중요도 상위 K)
    cand = np.argsort(global_imp)[::-1][:topK_imp]
    top_idx = cand[
        np.argsort(hetero[cand])[::-1][:topN]
    ]  # 후보 안에서 이질성 큰 순 TopN

    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    ###########################################
    # global_imp = _agg_abs(vals, how=agg)  # shape (p,)  # median(|SHAP|)
    # top_idx = np.argsort(global_imp)[::-1][:topN]

    # M_top = M[:, top_idx]
    # feat_top = [feat_names[i] for i in top_idx]
    ###########################################
    # top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    # M_top = M[:, top_idx]  # raw topN
    # feat_top = [feat_names[i] for i in top_idx]

    # --- (C) 행 정규화(클러스터 내부 상대값)
    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max  # normalized topN

    # --- (D) DataFrame 구성
    idx_names = [f"cluster_{c}" for c in clusters]
    df_raw_all = pd.DataFrame(M, index=idx_names, columns=feat_names)
    df_raw_top = pd.DataFrame(M_top, index=idx_names, columns=feat_top)
    df_norm_top = pd.DataFrame(M_disp, index=idx_names, columns=feat_top)

    # ---------- (E) Normalized Heatmap (기존 파일명 유지) ----------
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]
    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_norm_top.T.values,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",  #  – Row-normalized
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")

    # ---------- (F) Raw Heatmap (새 파일) ----------
    out_path_raw = out_path.with_name(out_path.stem + "__RAW.png")
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_raw_top.T.values,
        cmap=cmap,
        cbar_kws={"label": f"{agg} |SHAP|"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path_raw, dpi=300, bbox_inches="tight")

    return {
        "raw_all": df_raw_all,
        "raw_top": df_raw_top,
        "norm_top": df_norm_top,
        "feat_top": feat_top,
        "clusters": np.array(clusters),
        "fig_raw_path": out_path_raw,
        "fig_norm_path": out_path,
    }


# =============== 대표 케이스 ===============
def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:  # 노이즈 제외(원하면 포함 가능)
            continue
        idx = np.where(labels == c)[0]
        probs = y_prob[idx]
        med = np.mean(probs)
        med_idx = idx[np.argmin(np.abs(probs - med))]
        max_idx = idx[np.argmax(probs)]
        min_idx = idx[np.argmin(probs)]
        reps.append(
            {
                "cluster": int(c),
                "mean_idx": int(med_idx),
                "max_idx": int(max_idx),
                "min_idx": int(min_idx),
            }
        )
    return reps


def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:
            continue
        idx = np.where(labels == c)[0]
        d = df_pred.iloc[idx]
        rec = {"cluster": int(c)}
        fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
        fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
        if len(fp):
            rec["fp_idx"] = int(fp["y_prob"].idxmax())
        if len(fn):
            rec["fn_idx"] = int(fn["y_prob"].idxmin())
        reps.append(rec)
    return reps


def save_local_plots(
    label: str,
    expl: shap.Explanation,
    idx: int,
    out_dir: Path,
    feat_names=None,
    max_display=15,
    df_pred: pd.DataFrame | None = None,
    youden_thr: float | None = None,
    name_tag: str | None = None,
):
    out_dir.mkdir(parents=True, exist_ok=True)
    y_t = y_p = None
    y_prob = None
    thr_txt = ""
    if df_pred is not None:
        if "y_true" in df_pred.columns:
            y_t = int(df_pred.loc[idx, "y_true"])
        if "y_prob" in df_pred.columns:
            y_prob = float(df_pred.loc[idx, "y_prob"])
        if "y_pred" in df_pred.columns:
            y_p = int(df_pred.loc[idx, "y_pred"])
        else:
            thr = youden_thr if youden_thr is not None else 0.5
            y_p = (
                int((df_pred.loc[idx, "y_prob"]) >= thr)
                if "y_prob" in df_pred.columns
                else None
            )
        if youden_thr is not None:
            thr_txt = f", thr={youden_thr:.3f}"

    # waterfall
    plt.figure(figsize=(10, 8))
    try:
        shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
    except Exception:
        row = expl[idx]
        shap.plots.waterfall(row, show=False, max_display=max_display)
    title_bits = [f"{label} | id={idx}"]
    if y_t is not None:
        title_bits.append(f"y_true={y_t}")
    if y_p is not None:
        title_bits.append(f"y_pred={y_p}")
    if y_prob is not None:
        title_bits.append(f"y_prob={y_prob:.3f}")
    if name_tag:
        title_bits.append(f"[{name_tag}]")
    ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
    plt.title(ttl)
    plt.tight_layout()
    save_png_svg(
        out_dir
        / (
            f"{label}__waterfall_id{idx}"
            + (f"__{name_tag}" if name_tag else "")
            + ".png"
        ),
        dpi=300,
        bbox_inches="tight",
    )

    # force (옵션)
    try:
        plt.figure(figsize=(7, 2.8))
        shap.plots.force(
            expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
        )
        plt.tight_layout()
        save_png_svg(
            out_dir
            / (
                f"{label}__force_id{idx}"
                + (f"__{name_tag}" if name_tag else "")
                + ".png"
            ),
            dpi=300,
        )
    except Exception:
        plt.close()


# =============== DBSCAN 통계/요약 ===============
def add_dbscan_stats(
    out_dir: Path,
    label: str,
    y_true: np.ndarray,
    y_prob: np.ndarray,
    labels: np.ndarray,
):
    lab = np.asarray(labels)
    clusters = [c for c in np.unique(lab) if c != -1]
    rows = []
    for c in clusters:
        m = lab == c
        rows.append(
            dict(
                cluster=int(c),
                n=int(m.sum()),
                event_rate=float(np.mean(y_true[m] == 1)),
                mean_y_prob=float(np.mean(y_prob[m])),
            )
        )
    df = pd.DataFrame(rows).sort_values("n", ascending=False)

    chi_p = np.nan
    kw_p = np.nan
    if len(clusters) >= 2:
        table = [
            [
                int(np.sum((lab == c) & (y_true == 1))),
                int(np.sum((lab == c) & (y_true == 0))),
            ]
            for c in clusters
        ]
        chi_p = chi2_contingency(table)[1]
        groups = [y_prob[lab == c] for c in clusters]
        kw_p = kruskal(*groups).pvalue

    df["p_event_rate(chi2)"] = chi_p
    df["p_yprob(kruskal)"] = kw_p
    df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
    return df


# =============== 메인 루틴 ===============


def run_cluster_analysis_for_label(
    label: str,
    umap_n_components: int = 12,  # 군집용 UMAP 차원
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "euclidean",  # cosine
    min_samples: int = 6,  # DBSCAN
    eps: float | None = None,
    dbscan_metric: str = "euclidean",
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
    topN: int = 10,
    agg_for_heatmap: str = "mean",
    save_representatives: bool = True,
    random_state: int = 42,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
    out_dir = OUT_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    # 0) 데이터 로드
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # Youden 임계값 & df_pred 보정
    ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
    if ythr is None:
        ythr = _compute_youden_from_preds(
            np.asarray(y_true).astype(int), np.asarray(y_prob)
        )
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

    # 1) 표준화 & UMAP(고차원; 군집용)
    # V = _standardize(vals) # 표준화
    V = vals
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # 2) eps 초기값 (k-distance) & 곡선 저장
    if eps is None:
        eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.9)  # 0.85
        eps = eps0
        save_kdist_plot(
            kth,
            k=min_samples,
            out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
        )

    # 3) DBSCAN 적응형 루프
    trial = 0
    labels = None
    info_hist = []
    cur_min_samples = int(min_samples)
    cur_eps = float(eps)

    while trial < max_trials:
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))

        info_hist.append(
            dict(
                trial=trial,
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        if noise_ratio > max_noise_ratio:
            cur_eps *= 1.20
            if cur_min_samples > 3:
                cur_min_samples -= 1
        elif n_clusters < target_min_clusters:
            cur_eps *= 0.88
        else:
            cur_eps *= 1.05

        trial += 1

    # all-noise 대비: 마지막 완화 시도 1회
    if (labels is None) or np.all(labels == -1):
        cur_eps *= 1.30
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial="final_relax",
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

    # 4) 시각화용 2D UMAP (시각화 전용)
    if umap_n_components == 2:
        Z2 = X_hi
    else:
        um_2d = UMAP.UMAP(
            n_components=2,
            n_neighbors=max(umap_n_neighbors, 15),
            min_dist=max(umap_min_dist, 0.05),
            metric=umap_metric,
            random_state=random_state,
        )
        Z2 = um_2d.fit_transform(V)

    # [NEW] 위험도 순으로 라벨 재정렬 (이 부분 추가!)
    # Cluster 0 = Low Risk, Cluster N = High Risk
    if labels is not None:
        print("-> Reordering clusters by risk (0: Low -> N: High)...")
        labels = reorder_labels_by_risk(labels, np.asarray(y_true))

    # 5) 라벨 매핑 저장 (이제 정렬된 labels가 저장됨)
    m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # # 5) 라벨 매핑 저장(노이즈 포함)
    # m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    # m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # # ⭐️⭐️⭐️⭐️기존 코드 6) 2D 플롯 (클러스터 색) ⭐️⭐️⭐️⭐️
    # uniq = sorted(np.unique(labels))
    # # 클러스터 개수가 많을 수 있으므로 tab10 사용 권장
    # cmap = plt.get_cmap("tab10")

    # # 범례 공간 확보를 위해 가로 사이즈를 약간 늘림
    # plt.figure(figsize=(8.5, 6.0))

    # # 1. 노이즈(-1) 먼저 그리기 (배경으로 깔리게 처리)
    # if -1 in uniq:
    #     mask = labels == -1
    #     plt.scatter(
    #         Z2[mask, 0],
    #         Z2[mask, 1],
    #         c="#e0e0e0",  # 아주 연한 회색
    #         edgecolor="#bbbbbb",  # 테두리는 조금 진하게
    #         linewidth=0.1,
    #         s=15,  # 크기는 작게
    #         alpha=0.4,  # 투명하게
    #         label="Noise",  # 범례 이름
    #         zorder=1,  # 가장 뒤쪽에 배치
    #     )

    # # 2. 실제 클러스터 그리기 Loop
    # for c in uniq:
    #     if c == -1:
    #         continue
    #     mask = labels == c
    #     color = cmap(c % 20)  # 색상 순환 적용

    #     plt.scatter(
    #         Z2[mask, 0],
    #         Z2[mask, 1],
    #         c=[color],  # 단일 색상 적용
    #         s=22,  # 클러스터 포인트는 조금 더 크게
    #         alpha=0.9,  # 진하게
    #         label=f"Cluster {c}",
    #         edgecolor="white",  # 포인트 구분감
    #         linewidth=0.3,
    #         zorder=2,  # 노이즈보다 앞쪽에 배치
    #     )

    # title = f"UMAP + DBSCAN ({umap_n_components}D)"
    # # 제목에 전체 대비 노이즈 비율 표기
    # n_noise = np.sum(labels == -1)
    # n_total = len(labels)
    # plt.title(
    #     f"{title}\nTotal: {n_total}, Clusters: {len(set(labels)-{-1})}, Noise: {n_noise} ({n_noise/n_total:.1%})",
    #     fontsize=11,
    # )
    # plt.xlabel("UMAP-1")
    # plt.ylabel("UMAP-2")

    # # 범례 설정 (그래프 영역 밖 우측 상단에 배치)
    # plt.legend(
    #     bbox_to_anchor=(1.02, 1),
    #     loc="upper left",
    #     borderaxespad=0,
    #     frameon=False,
    #     fontsize=9,
    #     markerscale=1.5,  # 범례의 점 크기 키우기
    # )

    # plt.tight_layout()
    # save_png_svg(
    #     out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
    # )

    # 6) 2D 플롯 (Hull + Annotation 적용 버전)
    # print("-> Plotting UMAP with Threshold-based Risk Labels...")
    # plot_umap_with_hulls_and_labels(
    #     Z2=Z2,
    #     labels=labels,
    #     y_prob=y_prob,  # [중요] 여기에 y_prob를 반드시 넘겨주세요!
    #     out_path=out_dir / f"{label}__UMAP2D_DBSCAN_annotated.png",
    #     title=f"Risk Patterns ({label})",
    # )

    print("-> Plotting UMAP for PPT (Event Rate based)...")
    plot_umap_for_ppt(
        Z2=Z2,
        labels=labels,
        y_prob=y_prob,  # 기존 인자
        y_true=y_true,  # [NEW] 실제 정답값 전달
        out_path=out_dir / f"{label}__UMAP2D_DBSCAN_.png", # annotated_ppt
        title=f"Risk Patterns ({label})",
    )

    # 6-2) 2D 플롯 (위험도 색)
    plt.figure(figsize=(7.2, 6.2))
    sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"UMAP(2D) (colored by risk) — {label}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
    )

    # 7) 클러스터 통계/요약 + 히트맵
    df_stats = add_dbscan_stats(
        out_dir,
        label,
        y_true=np.asarray(y_true),
        y_prob=np.asarray(y_prob),
        labels=np.asarray(labels),
    )
    ret = plot_cluster_heatmap(
        vals,
        feat_names,
        labels,
        topN=topN,
        out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
        cmap="magma_r",  # rocket_r, viridis_r
        agg=agg_for_heatmap,
    )

    # === 수치 저장(요청 파일명 규칙) ===
    if ret is not None:
        # 모든 피처의 원시 |SHAP| (클러스터 × 전체 피처)
        ret["raw_all"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__ALL_FEATURES.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 원시 |SHAP|
        ret["raw_top"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_RAW.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 행 정규화(상대값)
        ret["norm_top"].to_csv(
            out_dir
            / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_ROWNORM.csv",
            index=True,
        )

    # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
    if save_representatives:
        rep_dir = out_dir / "representatives_dbscan"
        # mean/max/min
        reps = pick_representatives(y_prob, np.asarray(labels))
        for rep in reps:
            c = rep["cluster"]
            for tag in ["mean_idx", "max_idx", "min_idx"]:
                i = rep[tag]
                name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                save_local_plots(
                    label,
                    expl,
                    i,
                    rep_dir / f"cluster_{c}",
                    feat_names=feat_names,
                    max_display=15,
                    df_pred=df_pred,
                    youden_thr=ythr,
                    name_tag=name_tag,
                )
        # FP/FN
        err_reps = pick_error_representatives(df_pred, np.asarray(labels))
        for rep in err_reps:
            c = rep["cluster"]
            for tag in ["fp_idx", "fn_idx"]:
                if tag in rep:
                    i = rep[tag]
                    name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                    save_local_plots(
                        label,
                        expl,
                        i,
                        rep_dir / f"cluster_{c}",
                        feat_names=feat_names,
                        max_display=15,
                        df_pred=df_pred,
                        youden_thr=ythr,
                        name_tag=name_tag,
                    )

    # 9) 메타(JSON)
    meta = dict(
        timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
        algo="dbscan_tuned",
        random_state=random_state,
        umap=dict(
            n_components=umap_n_components,
            n_neighbors=umap_n_neighbors,
            min_dist=umap_min_dist,
            metric=umap_metric,
        ),
        final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
        trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
        n_clusters=int(len(set(labels) - {-1})),
        noise_ratio=float(np.mean(np.asarray(labels) == -1)),
        history=info_hist,
    )
    with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"-> saved: {out_dir}")
    return df_stats, labels

In [ ]:
# =============== 전체 라벨 반복 실행 ===============
all_cluster_summaries = []
for lb in LABELS:
    try:
        df_sum, labels = run_cluster_analysis_for_label(
            lb,
            umap_n_components=2,
            umap_n_neighbors=45,  # 40
            umap_min_dist=0.0,  # 0.05, 0.01
            umap_metric="euclidean",  # cosine
            min_samples=17,  # 20
            eps=None,
            dbscan_metric="euclidean",
            target_min_clusters=3,
            max_noise_ratio=0.45,  # 0.45
            max_trials=5,
            topN=10,
            agg_for_heatmap="mean",
            save_representatives=True,
            random_state=42,
        )
        df_sum.insert(0, "label", lb)
        all_cluster_summaries.append(df_sum)
    except Exception as e:
        print(f"[WARN] {lb}: {e}")

if all_cluster_summaries:
    all_summ = pd.concat(all_cluster_summaries, ignore_index=True)
    out_csv = OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.csv"
    all_summ.to_csv(out_csv, index=False)
    print("✅ saved:", out_csv)
    # Excel(라벨별 시트) 옵션
    try:
        with pd.ExcelWriter(
            OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx"
        ) as xw:
            for lb, df_ in all_summ.groupby("label"):
                df_.to_excel(xw, sheet_name=lb, index=False)
        print("✅ saved:", OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx")
    except Exception as e:
        print("[WARN] Excel save:", e)

In [ ]:
# ---------- 경로 설정 ----------
csv_path = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters/ALL_labels__cluster_summary_dbscan_tuned.csv")
)
out_path = csv_path.with_name("Table3_Cluster_Summary.xlsx")

# ---------- 데이터 로드 ----------
df = pd.read_csv(csv_path)

# ---------- 전처리 ----------
# (1) Δ 계산: 예측확률 - 실제 event rate 차이
df["Delta(abs)"] = (df["mean_y_prob"] - df["event_rate"]).abs().round(3)
df["Direction"] = df.apply(
    lambda x: (
        "Over (Pred>Event)"
        if x["mean_y_prob"] > x["event_rate"]
        else "Under (Pred<Event)"
    ),
    axis=1,
)

# (2) 도메인/해석 컬럼 자리 만들기 (수동 입력용)
df["Dominant domain"] = ""
df["Key top features"] = ""
df["Interpretation summary"] = ""

# (3) 정렬 및 보기 좋은 라벨링
df = df.rename(
    columns={
        "label": "Outcome (label)",
        "cluster": "Cluster",
        "n": "N",
        "event_rate": "Event rate",
        "mean_y_prob": "Mean predicted prob",
        "p_event_rate(chi2)": "p (event rate, χ²)",
        "p_yprob(kruskal)": "p (pred prob, KW)",
    }
)
df = df[
    [
        "Outcome (label)",
        "Cluster",
        "N",
        "Event rate",
        "Mean predicted prob",
        "Delta(abs)",
        "Direction",
        "Dominant domain",
        "Key top features",
        "Interpretation summary",
        "p (event rate, χ²)",
        "p (pred prob, KW)",
    ]
]

# ---------- Excel 저장 ----------
with pd.ExcelWriter(out_path) as writer:
    for lb, dsub in df.groupby("Outcome (label)"):
        dsub.to_excel(writer, sheet_name=lb, index=False)
    # 전체 통합 시트
    df.to_excel(writer, sheet_name="All_labels_combined", index=False)

print(f"✅ Saved Table 3 summary Excel → {out_path}")

In [ ]:
# ===== 경로 설정 =====
base = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)


# ===== Domain 분류 함수 =====
def infer_domain(feat: str) -> str:
    t = feat.lower()
    if any(k in t for k in ["aggression", "support", "social", "interpersonal", "llm"]):
        return "SDoMH"
    if any(k in t for k in ["er visit", "≥2 er", "≥3 er", "admission", "admit"]):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "k(",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "alc",
            "lymphocyte",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trinf",
            "psychometric",
            "audit",
            "hcl",
            "asrs",
            "ham",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "treatment",
            "medic",
            "benzodiazepine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# ===== 초기화 =====
labels = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]
records_summary = []  # Table 4
etable_dict = {}  # eTable S4 outcome별 sheet 저장

# ===== outcome별 반복 =====
for lb in labels:
    fpath = base / lb / f"{lb}__heatmap_values_mean__TOP10_RAW.csv"
    print("using:", fpath)
    df = pd.read_csv(fpath, index_col=0)

    # --- eTable S4용 wide-format 저장
    df_wide = df.copy()
    df_wide["Feature"] = df_wide.index
    etable_dict[lb] = df_wide.T  # transpose해서 Feature가 column으로 가도록 저장

    # --- Table 4용 계산
    for col in df.columns:
        vals = df[col].values
        # Kruskal–Wallis 검정 (클러스터 간 이질성 유의성)
        try:
            stat, pval = kruskal(*[df[col][df.index == c] for c in df.index])
        except Exception:
            pval = None

        records_summary.append(
            {
                "Outcome": lb,
                "Feature": col,
                "Mean(|SHAP|)": vals.mean(),
                "SD": vals.std(),
                "Range": vals.max() - vals.min(),
                "Δ(Top−Bottom)": vals.max() - vals.min(),
                "Max cluster": df.index[vals.argmax()],
                "Min cluster": df.index[vals.argmin()],
                "Top vs Bottom": f"{df.index[vals.argmax()]} > {df.index[vals.argmin()]}",
                "p(KW)": pval,
            }
        )

# ===== Table 4 (요약) =====
df_table4 = pd.DataFrame(records_summary)
df_table4["Domain"] = df_table4["Feature"].apply(infer_domain)
df_table4 = df_table4.sort_values(["Outcome", "SD"], ascending=[True, False])

# ===== 저장 =====
out_summary = base / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
out_wide = base / "eTableS4_SHAP_ClusterMatrix.xlsx"

with pd.ExcelWriter(out_summary) as writer:
    df_table4.to_excel(writer, sheet_name="All_outcomes", index=False)
    for lb, sub in df_table4.groupby("Outcome"):
        sub.to_excel(writer, sheet_name=lb, index=False)

with pd.ExcelWriter(out_wide) as writer:
    for lb, dfw in etable_dict.items():
        dfw.to_excel(writer, sheet_name=lb)

print(f"✅ Saved Table 4 summary → {out_summary}")
print(f"✅ Saved eTable S4 matrix → {out_wide}")

In [ ]:
# ===== 경로 =====
base = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)

# Table4 요약 엑셀
f_table4 = base / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
df = pd.read_excel(f_table4, sheet_name="All_outcomes")

sns.set(style="whitegrid", font_scale=1.0)
palette = sns.color_palette("Set2")

# ===== (A) Feature별 Boxplot (x=cluster, y=mean |SHAP|) =====
for outcome in ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]:
    target_outcome = outcome
    fpath = (
        base / target_outcome / f"{target_outcome}__heatmap_values_mean__TOP10_RAW.csv"
    )
    df_feat = pd.read_csv(fpath, index_col=0)
    df_feat.index = [
        c.replace("cluster_", "C") for c in df_feat.index
    ]  # cluster_0 → C0

    # ===== (1) Dotplot (by Feature 색상) =====
    df_melt = df_feat.T.reset_index().melt(
        id_vars="index", var_name="Cluster", value_name="Mean |SHAP|"
    )

    plt.figure(figsize=(8, 5))
    ax = sns.stripplot(
        x="Cluster",
        y="Mean |SHAP|",
        hue="index",
        data=df_melt,
        dodge=True,
        size=6,
        jitter=False,
        palette="tab10",
        alpha=0.9,
    )

    # ➊ 카테고리 순서(좌→우) 얻기
    cats = [t.get_text() for t in ax.get_xticklabels()]

    # ➋ 각 카테고리 경계(정수 + 0.5 위치)에 점선 추가
    for i in range(len(cats) - 1):
        ax.axvline(i + 0.5, ls="--", lw=0.8, color="gray", alpha=0.5, zorder=0)

    # ➌ 레이아웃/레이블
    ax.set_title(
        f"{target_outcome} — Cluster mean |SHAP| per Feature",
        fontsize=11,
        weight="bold",
    )
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Mean |SHAP|")
    plt.xticks()  # rotation=30
    plt.legend(title="Feature", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(base / f"Fig5A_{target_outcome}_dotplot_byFeature_guides.png", dpi=300)

    # ===== (2) Top N feature subplot 버전 =====
    top_feats = df_feat.columns[:6]  # 상위 6개 feature 예시
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)

    for ax, feat in zip(axes.flat, top_feats):
        sns.lineplot(x=df_feat.index, y=df_feat[feat], marker="o", color="black", ax=ax)
        ax.set_title(feat, fontsize=10)
        ax.set_xlabel("Cluster")
        ax.set_ylabel("Mean |SHAP|")
        ax.set_xticklabels(ax.get_xticklabels(), ha="right")  # rotation=30

    plt.suptitle(
        f"{target_outcome} — Cluster mean |SHAP| (Top 6 features)",
        fontsize=13,
        weight="bold",
    )
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(base / f"Fig5A_{target_outcome}_TopFeatures_clean.png", dpi=300)

# ===== (B) Δ(Top−Bottom) Barplot (feature별 이질성 순위) =====
topN = 20  # 상위 20개 표시 (필요시 조절)
df_rank = df.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)

plt.figure(figsize=(7.5, 6.5))
sns.barplot(
    y="Feature",
    x="Δ(Top−Bottom)",
    hue="Domain",
    data=df_rank,
    palette=palette,
    dodge=False,
)
plt.title(
    "Top features by SHAP heterogeneity (Δ Top−Bottom)", fontsize=12, weight="bold"
)
plt.xlabel("Δ (max−min mean |SHAP| across clusters)")
plt.ylabel("Feature")
plt.legend(title="Domain", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(base / "Fig5B_SHAP_Heterogeneity_Rank.png", dpi=300)

# ===== (C) Domain별 평균 SD 막대그래프 =====
df_dom = (
    df.groupby("Domain", as_index=False)["SD"].mean().sort_values("SD", ascending=False)
)

plt.figure(figsize=(6.5, 4.2))
sns.barplot(y="Domain", x="SD", data=df_dom, palette=palette)
plt.title("Average SHAP heterogeneity (SD) by Domain", fontsize=12, weight="bold")
plt.xlabel("Mean SD of mean |SHAP| across clusters")
plt.ylabel("")
plt.tight_layout()
plt.savefig(base / "Fig5C_Domain_SHAP_Heterogeneity.png", dpi=300)

print("✅ Figures saved:")
print(" -", base / f"Fig5A_{target_outcome}_boxplot.png")
print(" -", base / "Fig5B_SHAP_Heterogeneity_Rank.png")

print(" -", base / "Fig5C_Domain_SHAP_Heterogeneity.png")

# Raw SHAP 뽑기

In [ ]:
# === 샘플별 SHAP 추출 & 저장 ===
OUT_ROOT = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters/SHAP")
)


def export_sample_shap(label: str, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) SHAP/예측 로드 (이미 있는 함수들 활용)
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # 2) 메타(정답/확률/예측) 정리
    meta = pd.DataFrame(
        {
            "sample_id": np.arange(vals.shape[0], dtype=int),
            "y_true": np.asarray(y_true, dtype=int),
            "y_prob": np.asarray(y_prob, dtype=float),
        }
    )
    if "y_pred" in df_pred.columns:
        meta["y_pred"] = df_pred["y_pred"].astype(int)
    else:
        # 필요 시 Youden threshold로 예측치 생성
        thr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
        if thr is None:
            thr = _compute_youden_from_preds(
                meta["y_true"].values, meta["y_prob"].values
            )
        meta["y_pred"] = (meta["y_prob"] >= thr).astype(int)

    # 3) 샘플×피처 SHAP (wide)
    df_shap_wide = pd.DataFrame(vals, columns=feat_names)
    df_wide = pd.concat([meta, df_shap_wide], axis=1)

    # 4) (선택) 클러스터 라벨 붙이기: 이미 DBSCAN 돌렸다면 라벨 파일을 자동 병합
    lab_path = OUT_ROOT / label / f"{label}__dbscan_labels.csv"
    if lab_path.exists():
        lab = pd.read_csv(lab_path)  # columns: idx, cluster
        lab = lab.rename(columns={"idx": "sample_id"})
        df_wide = df_wide.merge(lab, on="sample_id", how="left")

    # 5) 저장 (wide & long)
    out_wide_csv = OUT_ROOT / label / f"{label}__sample_SHAP_wide.csv"
    out_wide_parq = OUT_ROOT / label / f"{label}__sample_SHAP_wide.parquet"
    df_wide.to_csv(out_wide_csv, index=False)
    try:
        df_wide.to_parquet(out_wide_parq, index=False)
    except Exception:
        pass

    # long 포맷 (원하면 주석 해제; 용량 커질 수 있음)
    # df_long = df_wide.melt(
    #     id_vars=[c for c in ["sample_id","y_true","y_prob","y_pred","cluster"] if c in df_wide.columns],
    #     var_name="feature",
    #     value_name="shap_value"
    # )
    # df_long.to_parquet(OUT_ROOT / label / f"{label}__sample_SHAP_long.parquet", index=False)

    print(f"✅ saved: {out_wide_csv}")
    return df_wide


for label in ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]:
    export_sample_shap(label, OUT_ROOT / label)

# Cluster 유의성 비교

In [ ]:
from pathlib import Path
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.stats import kruskal, mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [ ]:
# --------------------------
# 경로
# --------------------------
ROOT = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)
SHAP_ROOT = ROOT / "SHAP"
DBSCAN_ROOT = ROOT
SIG_ROOT = DBSCAN_ROOT / "cluster_sig"
SIG_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# --------------------------
# 유틸
# --------------------------
META_COLS = {"sample_id", "y_true", "y_prob", "y_pred", "cluster"}


def _feature_columns(df: pd.DataFrame):
    return [c for c in df.columns if c not in META_COLS]


def _all_identical(groups):
    cat = np.concatenate([g.ravel() for g in groups if len(g) > 0])
    return cat.size == 0 or np.nanstd(cat) == 0.0


def _safe_mw(a, b):
    if len(a) < 2 or len(b) < 2:
        return np.nan, np.nan
    if _all_identical([a, b]):
        return 1.0, 0.0
    p = mannwhitneyu(a, b, alternative="two-sided").pvalue
    # Cliff's delta
    gt = sum(x > y for x in a for y in b)
    lt = sum(x < y for x in a for y in b)
    d = (gt - lt) / (len(a) * len(b))
    return p, d


# --------------------------
# 1) SHAP × Cluster 병합
# --------------------------
def attach_cluster_fast(label: str) -> Path:
    """
    sample_id == idx 가정 하에 DBSCAN 라벨을 병합.
    병합된 CSV를 SHAP_ROOT/<label>/에 저장하고 그 경로를 반환.
    """
    shap_dir = SHAP_ROOT / label
    dbscan_dir = DBSCAN_ROOT / label
    shap_dir.mkdir(parents=True, exist_ok=True)

    f_shap = shap_dir / f"{label}__sample_SHAP_wide.csv"
    f_lab = dbscan_dir / f"{label}__dbscan_labels.csv"
    assert f_shap.exists(), f"Not found: {f_shap}"
    assert f_lab.exists(), f"Not found: {f_lab}"

    df = pd.read_csv(f_shap)
    lab = pd.read_csv(f_lab)  # columns: idx, cluster

    merged = df.merge(
        lab.rename(columns={"idx": "sample_id"}), on="sample_id", how="left"
    )
    cov = merged["cluster"].notna().mean()
    print(f"[{label}] coverage by sample_id==idx: {cov*100:.1f}%")

    out = shap_dir / f"{label}__sample_SHAP_wide_with_cluster.csv"
    merged.to_csv(out, index=False)
    print(f"✅ saved (merged): {out}")
    return out


# --------------------------
# 2) 유의성 검정 (Kruskal + 모든 pairwise)
#    - noise(-1) 제외 기본값
#    - pairwise FDR은 '피처별'로 보정
# --------------------------
def run_significance_full(
    label: str, use_abs: bool = True, include_noise: bool = False
) -> Path:
    f_merged = SHAP_ROOT / label / f"{label}__sample_SHAP_wide_with_cluster.csv"
    df = pd.read_csv(f_merged)
    if not include_noise:
        df = df[df["cluster"] != -1].copy()
    print(f"[{label}] Used {len(df)} samples (exclude noise={not include_noise})")

    feat_cols = _feature_columns(df)
    clusters = sorted(df["cluster"].dropna().unique())

    # ---- (A) Kruskal ----
    rows_kw = []
    for feat in feat_cols:
        vals = np.abs(df[feat].values) if use_abs else df[feat].values
        groups = [vals[df["cluster"].values == c] for c in clusters]
        if len(groups) < 2 or any(len(g) < 2 for g in groups) or _all_identical(groups):
            p_kw = np.nan
        else:
            try:
                p_kw = kruskal(*groups).pvalue
            except Exception:
                p_kw = np.nan
        rows_kw.append({"Feature": feat, "p_kw": p_kw})
    kw = pd.DataFrame(rows_kw)
    kw["q_kw"] = multipletests(kw["p_kw"].fillna(1.0), method="fdr_bh")[1]

    # 보조 요약(중앙값 기반 Top/Bottom 및 범위)
    if len(clusters) >= 1:
        med = df.groupby("cluster")[feat_cols].mean()
        rng = med.max(0) - med.min(0)
        top_c = med.idxmax()
        bot_c = med.idxmin()
        kw["Δ(Top−Bottom)"] = kw["Feature"].map(rng.to_dict())
        kw["Top cluster"] = kw["Feature"].map(top_c.to_dict())
        kw["Bottom cluster"] = kw["Feature"].map(bot_c.to_dict())
        kw["Top vs Bottom"] = (
            kw["Top cluster"].astype(str) + " > " + kw["Bottom cluster"].astype(str)
        )

    # ---- (B) Pairwise 모든 클러스터 쌍 (피처별 FDR) ----
    pair_tables = []
    for feat in feat_cols:
        vals = np.abs(df[feat].values) if use_abs else df[feat].values
        rows = []
        for c1, c2 in combinations(clusters, 2):
            a, b = vals[df["cluster"] == c1], vals[df["cluster"] == c2]
            p, d = _safe_mw(a, b)
            rows.append(
                {
                    "Feature": feat,
                    "Cluster1": c1,
                    "Cluster2": c2,
                    "p_raw": p,
                    "cliffs_delta": d,
                    "n1": len(a),
                    "n2": len(b),
                }
            )
        pw_feat = pd.DataFrame(rows)
        if len(pw_feat) > 0:
            pw_feat["q_fdr"] = multipletests(
                pw_feat["p_raw"].fillna(1.0), method="fdr_bh"
            )[1]
        pair_tables.append(pw_feat)
    pw = (
        pd.concat(pair_tables, ignore_index=True)
        if pair_tables
        else pd.DataFrame(
            columns=[
                "Feature",
                "Cluster1",
                "Cluster2",
                "p_raw",
                "cliffs_delta",
                "n1",
                "n2",
                "q_fdr",
            ]
        )
    )

    # ---- (C) 저장 ----
    out_xlsx = SIG_ROOT / f"{label}__SHAP_cluster_significance_full.xlsx"
    with pd.ExcelWriter(out_xlsx) as xw:
        kw.to_excel(xw, sheet_name="Kruskal", index=False)
        pw.to_excel(xw, sheet_name="Pairwise_All", index=False)
    print(f"✅ saved full pairwise significance: {out_xlsx}")
    return out_xlsx


# --------------------------
# 3) 전체 outcome 일괄 실행
# --------------------------
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]


def run_all(labels=LABELS):
    for lb in labels:
        try:
            attach_cluster_fast(lb)
            run_significance_full(
                lb, use_abs=True, include_noise=False
            )  # |SHAP| 기준, 노이즈 제외
        except Exception as e:
            print(f"[WARN] {lb}: {e}")

In [ ]:
# 실행
if __name__ == "__main__":
    run_all()

## Table 및 시각화 반영

In [ ]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import kruskal  # 기존 코드에서 사용

sns.set(style="whitegrid", font_scale=1.0)

In [ ]:
# ----- 경로 설정 -----
RAWNORM_BASE = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")
)
SIG_ROOT = RAWNORM_BASE / "cluster_sig"
FIGTAB_ROOT = RAWNORM_BASE / "cluster_figure_table"
FIGTAB_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# ----- 유틸 -----
def stars_from_q(q):
    if pd.isna(q):
        return ""
    if q < 0.001:
        return "***"
    if q < 0.01:
        return "**"
    if q < 0.05:
        return "*"
    return ""


def parse_cluster_str(s):
    # "cluster_3" -> 3 / "C3" -> 3
    if pd.isna(s):
        return None
    m = re.search(r"(-?\d+)", str(s))
    return int(m.group(1)) if m else None


# ===== Domain 분류 함수 (기존과 동일) =====
# ===== Domain 분류 함수 =====
def infer_domain(feat: str) -> str:
    t = feat.lower()
    if any(k in t for k in ["aggression", "support", "social", "interpersonal", "llm"]):
        return "SDoMH"
    if any(k in t for k in ["er visit", "≥2 er", "≥3 er", "admission", "admit"]):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "k(",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "alc",
            "lymphocyte",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trinf",
            "psychometric",
            "audit",
            "hcl",
            "asrs",
            "ham",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "treatment",
            "medic",
            "benzodiazepine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# ===== 1) 기존 Table4 요약 읽기 =====
f_table4 = RAWNORM_BASE / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
df_all = pd.read_excel(f_table4, sheet_name="All_outcomes")
# 안전하게 Domain 재계산(혹시 비어있을 경우)
if "Domain" not in df_all.columns:
    df_all["Domain"] = df_all["Feature"].apply(infer_domain)

# ===== 2) 유의성 결과(각 label)와 결합 → 메인 테이블 확장 =====
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# 결과 컬럼 초기화
for col in ["p_kw", "q_kw", "p_tb", "q_tb", "cliffs_delta", "sig_kw", "sig_tb"]:
    if col not in df_all.columns:
        df_all[col] = np.nan

# label별로 KW + Pairwise에서 Top-Bottom p를 주입
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if not sig_path.exists():
        print(f"[WARN] missing significance file: {sig_path}")
        continue
    df_kw = pd.read_excel(sig_path, sheet_name="Kruskal")
    df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")

    # 매핑(dict) 준비
    qkw_map = dict(zip(df_kw["Feature"], df_kw["q_kw"]))
    pkw_map = dict(zip(df_kw["Feature"], df_kw["p_kw"]))

    # df_all 중 해당 outcome만 추림
    m_out = df_all["Outcome"] == lb
    sub = df_all.loc[m_out].copy()

    # 각 row에서 Top/Bottom 클러스터 번호 파싱 후 pairwise에서 해당 쌍의 p/q/효과크기 찾기
    p_tb_list, q_tb_list, d_tb_list, sig_tb_list = [], [], [], []
    p_kw_list, q_kw_list, sig_kw_list = [], [], []
    for _, row in sub.iterrows():
        feat = row["Feature"]
        # KW
        pkw = pkw_map.get(feat, np.nan)
        qkw = qkw_map.get(feat, np.nan)
        sig_kw = stars_from_q(qkw)

        # Top/Bottom 클러스터
        topc = parse_cluster_str(row.get("Max cluster") or row.get("Top cluster") or "")
        botc = parse_cluster_str(
            row.get("Min cluster") or row.get("Bottom cluster") or ""
        )

        # pairwise에서 두 방향(Cluster1,Cluster2) 모두 확인
        df_feat = df_pw[df_pw["Feature"] == feat]
        hit = df_feat[
            ((df_feat["Cluster1"] == topc) & (df_feat["Cluster2"] == botc))
            | ((df_feat["Cluster1"] == botc) & (df_feat["Cluster2"] == topc))
        ]
        if len(hit):
            p_tb = float(hit["p_raw"].iloc[0])
            q_tb = float(hit["q_fdr"].iloc[0])
            d_tb = float(hit["cliffs_delta"].iloc[0])
            sig_tb = stars_from_q(q_tb)
        else:
            p_tb = q_tb = d_tb = np.nan
            sig_tb = ""

        p_kw_list.append(pkw)
        q_kw_list.append(qkw)
        sig_kw_list.append(sig_kw)
        p_tb_list.append(p_tb)
        q_tb_list.append(q_tb)
        d_tb_list.append(d_tb)
        sig_tb_list.append(sig_tb)

    # 주입
    df_all.loc[m_out, "p_kw"] = p_kw_list
    df_all.loc[m_out, "q_kw"] = q_kw_list
    df_all.loc[m_out, "sig_kw"] = sig_kw_list
    df_all.loc[m_out, "p_tb"] = p_tb_list
    df_all.loc[m_out, "q_tb"] = q_tb_list
    df_all.loc[m_out, "cliffs_delta"] = d_tb_list
    df_all.loc[m_out, "sig_tb"] = sig_tb_list

# 저장(메인 테이블)
out_table_main = FIGTAB_ROOT / "Table4_with_significance.xlsx"
with pd.ExcelWriter(out_table_main) as xw:
    df_all.to_excel(xw, sheet_name="All_outcomes", index=False)
    for lb, sub in df_all.groupby("Outcome"):
        sub.to_excel(xw, sheet_name=lb, index=False)
print("✅ Saved main table with significance →", out_table_main)

# ===== 3) 서플: 모든 쌍 pairwise를 그대로 outcome별로 CSV 저장 =====
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if sig_path.exists():
        df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")
        df_pw.to_csv(FIGTAB_ROOT / f"Supp_Pairwise_All_{lb}.csv", index=False)

# ===== 4) 그림에 별표 추가 (KW 기준) =====
palette = sns.color_palette("Set2")

# (A) Feature별 Dotplot: 범례(Feature 이름)에 별표 붙이기 (q_kw 기준)
for outcome in LABELS:
    # 히트맵 수치 파일 로드
    fpath = RAWNORM_BASE / outcome / f"{outcome}__heatmap_values_mean__TOP10_RAW.csv"
    if not fpath.exists():
        print(f"[WARN] missing heatmap CSV: {fpath}")
        continue
    df_feat = pd.read_csv(fpath, index_col=0)
    # C 라벨로 보기 좋게
    df_feat.index = [c.replace("cluster_", "C") for c in df_feat.index]

    # 해당 outcome의 q_kw 맵(Feature→별표)
    sub_main = df_all[df_all["Outcome"] == outcome]
    qkw_map = dict(zip(sub_main["Feature"], sub_main["q_kw"]))
    star_map = {f: stars_from_q(q) for f, q in qkw_map.items()}

    # legend용 라벨 교체를 위해 컬럼명을 "원래+별"로 임시 재라벨링
    feat_renamed = {
        col: (f"{col}{star_map.get(col,'')}" if col in star_map else col)
        for col in df_feat.columns
    }
    df_feat_renamed = df_feat.rename(columns=feat_renamed)

    # Dotplot
    df_melt = df_feat_renamed.T.reset_index().melt(
        id_vars="index", var_name="Cluster", value_name="Mean |SHAP|"
    )
    plt.figure(figsize=(10, 5))
    ax = sns.stripplot(
        x="Cluster",
        y="Mean |SHAP|",
        hue="index",
        data=df_melt,
        dodge=True,
        size=6,
        jitter=False,
        palette="tab10",
        alpha=0.9,
    )
    # 경계선 가이드
    cats = [t.get_text() for t in ax.get_xticklabels()]
    for i in range(len(cats) - 1):
        ax.axvline(i + 0.5, ls="--", lw=0.8, color="gray", alpha=0.5, zorder=0)
    ax.set_title(
        f"{outcome} — Cluster mean |SHAP| per Feature", fontsize=11, weight="bold"
    )
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Mean |SHAP|")
    plt.legend(title="Feature", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
    plt.tight_layout()
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_dotplot_byFeature_guides_withStars.png", dpi=300
    )
    plt.close()

    # (B) Top N feature subplot: 제목에 별표 붙이기 (KW 기준)
    top_feats = df_feat.columns[:6]
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
    for ax, feat in zip(axes.flat, top_feats):
        star = star_map.get(feat, "")
        sns.lineplot(x=df_feat.index, y=df_feat[feat], marker="o", color="black", ax=ax)
        ax.set_title(f"{feat}{star}", fontsize=10)
        ax.set_xlabel("Cluster")
        ax.set_ylabel("Mean |SHAP|")
        ax.set_xticklabels(ax.get_xticklabels(), ha="right")
    plt.suptitle(
        f"{outcome} — Cluster mean |SHAP| (Top 6 features)",
        fontsize=13,
        weight="bold",
    )
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_TopFeatures_clean_withStars.png", dpi=300
    )
    plt.close()

# (B) Δ(Top−Bottom) barplot & (C) Domain barplot은 기존 그림 유지
#    (원하면 bar label에 별표 추가 가능: df_all에서 원하는 조건으로 star를 조합)
# 그대로 복제해서 저장 위치만 figure_table로 변경:

# Δ(Top−Bottom) TopN
topN = 20
df_rank = df_all.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)
plt.figure(figsize=(7.5, 6.5))
sns.barplot(
    y="Feature",
    x="Δ(Top−Bottom)",
    hue="Domain",
    data=df_rank,
    palette=palette,
    dodge=False,
)
plt.title(
    "Top features by SHAP heterogeneity (Δ Top−Bottom)", fontsize=12, weight="bold"
)
plt.xlabel("Δ (max−min mean |SHAP| across clusters)")
plt.ylabel("Feature")
plt.legend(title="Domain", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5B_SHAP_Heterogeneity_Rank.png", dpi=300)
plt.close()

# Domain 별 평균 SD
df_dom = (
    df_all.groupby("Domain", as_index=False)["SD"]
    .mean()
    .sort_values("SD", ascending=False)
)
plt.figure(figsize=(6.5, 4.2))
sns.barplot(y="Domain", x="SD", data=df_dom, palette=palette)
plt.title("Average SHAP heterogeneity (SD) by Domain", fontsize=12, weight="bold")
plt.xlabel("Mean SD of mean |SHAP| across clusters")
plt.ylabel("")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5C_Domain_SHAP_Heterogeneity.png", dpi=300)
plt.close()

print("✅ Outputs saved to:", FIGTAB_ROOT)

- ER, AD 제외

In [ ]:
FIGTAB_ROOT = RAWNORM_BASE / "cluster_figure_table/no_util"

In [ ]:
# ----- 유틸 -----
def stars_from_q(q):
    if pd.isna(q):
        return ""
    if q < 0.001:
        return "***"
    if q < 0.01:
        return "**"
    if q < 0.05:
        return "*"
    return ""


def parse_cluster_str(s):
    # "cluster_3" -> 3 / "C3" -> 3
    if pd.isna(s):
        return None
    m = re.search(r"(-?\d+)", str(s))
    return int(m.group(1)) if m else None


# ===== Domain 분류 함수 (기존과 동일) =====
# ===== Domain 분류 함수 =====
def infer_domain(feat: str) -> str:
    t = feat.lower()
    if any(k in t for k in ["aggression", "support", "social", "interpersonal", "llm"]):
        return "Aggression / Social function (LLM)"
    if any(k in t for k in ["er visit", "≥2 er", "≥3 er", "admission", "admit"]):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "k(",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "alc",
            "lymphocyte",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trinf",
            "psychometric",
            "audit",
            "hcl",
            "asrs",
            "ham",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "treatment",
            "medic",
            "benzodiazepine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# ===== 1) 기존 Table4 요약 읽기 =====
f_table4 = RAWNORM_BASE / "Table4_CrossCluster_SHAP_Heterogeneity.xlsx"
df_all = pd.read_excel(f_table4, sheet_name="All_outcomes")
# 안전하게 Domain 재계산(혹시 비어있을 경우)
if "Domain" not in df_all.columns:
    df_all["Domain"] = df_all["Feature"].apply(infer_domain)

# ===== 2) 유의성 결과(각 label)와 결합 → 메인 테이블 확장 =====
LABELS = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# 결과 컬럼 초기화
for col in ["p_kw", "q_kw", "p_tb", "q_tb", "cliffs_delta", "sig_kw", "sig_tb"]:
    if col not in df_all.columns:
        df_all[col] = np.nan

# label별로 KW + Pairwise에서 Top-Bottom p를 주입
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if not sig_path.exists():
        print(f"[WARN] missing significance file: {sig_path}")
        continue
    df_kw = pd.read_excel(sig_path, sheet_name="Kruskal")
    df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")

    # 매핑(dict) 준비
    qkw_map = dict(zip(df_kw["Feature"], df_kw["q_kw"]))
    pkw_map = dict(zip(df_kw["Feature"], df_kw["p_kw"]))

    # df_all 중 해당 outcome만 추림
    m_out = df_all["Outcome"] == lb
    sub = df_all.loc[m_out].copy()

    # 각 row에서 Top/Bottom 클러스터 번호 파싱 후 pairwise에서 해당 쌍의 p/q/효과크기 찾기
    p_tb_list, q_tb_list, d_tb_list, sig_tb_list = [], [], [], []
    p_kw_list, q_kw_list, sig_kw_list = [], [], []
    for _, row in sub.iterrows():
        feat = row["Feature"]
        # KW
        pkw = pkw_map.get(feat, np.nan)
        qkw = qkw_map.get(feat, np.nan)
        sig_kw = stars_from_q(qkw)

        # Top/Bottom 클러스터
        topc = parse_cluster_str(row.get("Max cluster") or row.get("Top cluster") or "")
        botc = parse_cluster_str(
            row.get("Min cluster") or row.get("Bottom cluster") or ""
        )

        # pairwise에서 두 방향(Cluster1,Cluster2) 모두 확인
        df_feat = df_pw[df_pw["Feature"] == feat]
        hit = df_feat[
            ((df_feat["Cluster1"] == topc) & (df_feat["Cluster2"] == botc))
            | ((df_feat["Cluster1"] == botc) & (df_feat["Cluster2"] == topc))
        ]
        if len(hit):
            p_tb = float(hit["p_raw"].iloc[0])
            q_tb = float(hit["q_fdr"].iloc[0])
            d_tb = float(hit["cliffs_delta"].iloc[0])
            sig_tb = stars_from_q(q_tb)
        else:
            p_tb = q_tb = d_tb = np.nan
            sig_tb = ""

        p_kw_list.append(pkw)
        q_kw_list.append(qkw)
        sig_kw_list.append(sig_kw)
        p_tb_list.append(p_tb)
        q_tb_list.append(q_tb)
        d_tb_list.append(d_tb)
        sig_tb_list.append(sig_tb)

    # 주입
    df_all.loc[m_out, "p_kw"] = p_kw_list
    df_all.loc[m_out, "q_kw"] = q_kw_list
    df_all.loc[m_out, "sig_kw"] = sig_kw_list
    df_all.loc[m_out, "p_tb"] = p_tb_list
    df_all.loc[m_out, "q_tb"] = q_tb_list
    df_all.loc[m_out, "cliffs_delta"] = d_tb_list
    df_all.loc[m_out, "sig_tb"] = sig_tb_list

# 저장(메인 테이블)
out_table_main = FIGTAB_ROOT / "Table4_with_significance.xlsx"
with pd.ExcelWriter(out_table_main) as xw:
    df_all.to_excel(xw, sheet_name="All_outcomes", index=False)
    for lb, sub in df_all.groupby("Outcome"):
        sub.to_excel(xw, sheet_name=lb, index=False)
print("✅ Saved main table with significance →", out_table_main)

# ===== 3) 서플: 모든 쌍 pairwise를 그대로 outcome별로 CSV 저장 =====
for lb in LABELS:
    sig_path = SIG_ROOT / f"{lb}__SHAP_cluster_significance_full.xlsx"
    if sig_path.exists():
        df_pw = pd.read_excel(sig_path, sheet_name="Pairwise_All")
        df_pw.to_csv(FIGTAB_ROOT / f"Supp_Pairwise_All_{lb}.csv", index=False)

# ===== 4) 그림에 별표 추가 (KW 기준) =====
palette = sns.color_palette("Set2")

# (A) Feature별 Dotplot: 범례(Feature 이름)에 별표 붙이기 (q_kw 기준)
for outcome in LABELS:
    # 히트맵 수치 파일 로드
    fpath = RAWNORM_BASE / outcome / f"{outcome}__heatmap_values_mean__TOP10_RAW.csv"
    if not fpath.exists():
        print(f"[WARN] missing heatmap CSV: {fpath}")
        continue
    df_feat = pd.read_csv(fpath, index_col=0)

    # 🌟🌟🌟 이 위치에 필터링 코드 추가 🌟🌟🌟
    EXCLUDE_VARS = ["≥2 ER Visits", "≥3 Admissions"]
    cols_to_keep = [col for col in df_feat.columns if col not in EXCLUDE_VARS]
    df_feat = df_feat[cols_to_keep]
    # 🌟🌟🌟 필터링 코드 끝 🌟🌟🌟

    # C 라벨로 보기 좋게
    df_feat.index = [c.replace("cluster_", "C") for c in df_feat.index]

    # 해당 outcome의 q_kw 맵(Feature→별표)
    sub_main = df_all[df_all["Outcome"] == outcome]
    qkw_map = dict(zip(sub_main["Feature"], sub_main["q_kw"]))
    star_map = {f: stars_from_q(q) for f, q in qkw_map.items()}

    # legend용 라벨 교체를 위해 컬럼명을 "원래+별"로 임시 재라벨링
    feat_renamed = {
        col: (f"{col}{star_map.get(col,'')}" if col in star_map else col)
        for col in df_feat.columns
    }
    df_feat_renamed = df_feat.rename(columns=feat_renamed)

    # Dotplot
    df_melt = df_feat_renamed.T.reset_index().melt(
        id_vars="index", var_name="Cluster", value_name="Mean |SHAP|"
    )
    plt.figure(figsize=(10, 5))
    ax = sns.stripplot(
        x="Cluster",
        y="Mean |SHAP|",
        hue="index",
        data=df_melt,
        dodge=True,
        size=6,
        jitter=False,
        palette="tab10",
        alpha=0.9,
    )
    # 경계선 가이드
    cats = [t.get_text() for t in ax.get_xticklabels()]
    for i in range(len(cats) - 1):
        ax.axvline(i + 0.5, ls="--", lw=0.8, color="gray", alpha=0.5, zorder=0)
    ax.set_title(
        f"{outcome} — Cluster mean |SHAP| per Feature", fontsize=11, weight="bold"
    )
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Mean |SHAP|")
    plt.legend(title="Feature", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
    plt.tight_layout()
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_dotplot_byFeature_guides_withStars.png", dpi=300
    )
    plt.close()

    # (B) Top N feature subplot: 제목에 별표 붙이기 (KW 기준)
    top_feats = df_feat.columns[:6]
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
    for ax, feat in zip(axes.flat, top_feats):
        star = star_map.get(feat, "")
        sns.lineplot(x=df_feat.index, y=df_feat[feat], marker="o", color="black", ax=ax)
        ax.set_title(f"{feat}{star}", fontsize=10)
        ax.set_xlabel("Cluster")
        ax.set_ylabel("Mean |SHAP|")
        ax.set_xticklabels(ax.get_xticklabels(), ha="right")
    plt.suptitle(
        f"{outcome} — Cluster mean |SHAP| (Top 6 features)",
        fontsize=13,
        weight="bold",
    )
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(
        FIGTAB_ROOT / f"Fig5A_{outcome}_TopFeatures_clean_withStars.png", dpi=300
    )
    plt.close()

# (B) Δ(Top−Bottom) barplot & (C) Domain barplot은 기존 그림 유지
#    (원하면 bar label에 별표 추가 가능: df_all에서 원하는 조건으로 star를 조합)
# 그대로 복제해서 저장 위치만 figure_table로 변경:

# Δ(Top−Bottom) TopN

# 🌟🌟🌟 이 위치에 필터링 코드 추가 🌟🌟🌟
EXCLUDE_VARS = ["AD_more_three", "ER_more_two"]
df_all_filtered = df_all[~df_all["Feature"].isin(EXCLUDE_VARS)].copy()
# 🌟🌟🌟 필터링 코드 끝 🌟🌟🌟

topN = 20
# df_rank = df_all.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)
df_rank = df_all_filtered.sort_values("Δ(Top−Bottom)", ascending=False).head(topN)
plt.figure(figsize=(7.5, 6.5))
sns.barplot(
    y="Feature",
    x="Δ(Top−Bottom)",
    hue="Domain",
    data=df_rank,
    palette=palette,
    dodge=False,
)
plt.title(
    "Top features by SHAP heterogeneity (Δ Top−Bottom)", fontsize=12, weight="bold"
)
plt.xlabel("Δ (max−min mean |SHAP| across clusters)")
plt.ylabel("Feature")
plt.legend(title="Domain", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5B_SHAP_Heterogeneity_Rank.png", dpi=300)
plt.close()

# Domain 별 평균 SD
# df_dom = (
#     df_all.groupby("Domain", as_index=False)["SD"]
#     .mean()
#     .sort_values("SD", ascending=False)
# )
df_all_viz = df_all[~df_all["Feature"].isin(EXCLUDE_VARS)]  # 🌟 필터링된 데이터 사용

df_dom = (
    df_all_viz.groupby("Domain", as_index=False)["SD"]  # 🌟 df_all_viz 사용
    .mean()
    .sort_values("SD", ascending=False)
)
plt.figure(figsize=(6.5, 4.2))
sns.barplot(y="Domain", x="SD", data=df_dom, palette=palette)
plt.title("Average SHAP heterogeneity (SD) by Domain", fontsize=12, weight="bold")
plt.xlabel("Mean SD of mean |SHAP| across clusters")
plt.ylabel("")
plt.tight_layout()
plt.savefig(FIGTAB_ROOT / "Fig5C_Domain_SHAP_Heterogeneity.png", dpi=300)
plt.close()

print("✅ Outputs saved to:", FIGTAB_ROOT)

# Waterfall plot 정리

In [ ]:
import os, re, math, glob, textwrap
from pathlib import Path
from typing import List, Tuple, Dict
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np

# ========= 경로 설정 (당신 환경에 맞게 확인) =========
ROOT = Path(
    str(PROJECT_ROOT / "results/step2_modeling/251127_Final_modeling_globalunion25/figures/clinic_interpretation")
)
PWF = ROOT / "personal_waterfall_figures"  # 개인 워터폴
CLSR = ROOT / "PatientClusters"  # 클러스터 산출물
OUT = ROOT / "personal_cluster_water_sum"  # 출력 폴더 (이미 사용 중이면 그대로 씀)
OUT.mkdir(parents=True, exist_ok=True)


# ========= 공통 유틸 =========
def _font(size=18):
    try:
        # mac 기본 폰트 후보
        return ImageFont.truetype("/Library/Fonts/Arial.ttf", size)
    except Exception:
        return ImageFont.load_default()


def read_topk_txt_if_any(png_path: Path, fallback_csv_top: int = 3) -> str:
    """
    representatives_dbscan 쪽은 __top*.txt가 없을 수 있으니
    옆의 __top20.csv에서 상위 k개로 간단히 카드 문자열 구성.
    """
    txt_path = (
        png_path.with_suffix("").with_suffix("").with_name(png_path.stem + "__top3.txt")
    )
    if txt_path.exists():
        try:
            return txt_path.read_text().strip()
        except Exception:
            pass
    # fallback: 같은 이름의 __top20.csv 이용
    csv_guess = png_path.with_name(png_path.stem + "__top20.csv")
    if csv_guess.exists():
        try:
            d = pd.read_csv(csv_guess)
            d = d.sort_values("rank").head(fallback_csv_top)
            bits = [
                f"{r.feature}({'+' if r.shap_value>=0 else '−'})"
                for r in d.itertuples(index=False)
            ]
            return "; ".join(bits)
        except Exception:
            return ""
    return ""


def montage(
    images: List[Image.Image],
    captions: List[str],
    ncols=3,
    pad=28,
    cap_h=42,
    bgcolor=(255, 255, 255),
    font_size=18,
) -> Image.Image:
    """이미지들을 grid로 배치하고 각 패널 하단에 한 줄 캡션"""
    if not images:
        raise ValueError("No images to montage.")
    W = max(im.width for im in images)
    H = max(im.height for im in images)
    font = _font(font_size)
    n = len(images)
    ncols = max(1, ncols)
    nrows = math.ceil(n / ncols)
    canvas_w = pad + ncols * (W + pad)
    canvas_h = pad + nrows * (H + cap_h + pad)
    canvas = Image.new("RGB", (canvas_w, canvas_h), bgcolor)
    draw = ImageDraw.Draw(canvas)

    for i, (im, cap) in enumerate(zip(images, captions)):
        r, c = divmod(i, ncols)
        x = pad + c * (W + pad)
        y = pad + r * (H + cap_h + pad)
        canvas.paste(im, (x, y))
        # 한 줄로 너무 길면 줄임표
        cap = cap.strip().replace("\n", " ")
        if len(cap) > 120:
            cap = cap[:117] + "..."
        draw.text((x, y + H + 4), cap, fill=(25, 25, 25), font=font)
    return canvas


def save_multipage_pdf(png_paths: List[Path], out_pdf: Path):
    """여러 PNG를 여러 페이지 PDF로 저장"""
    imgs = [Image.open(p).convert("RGB") for p in png_paths]
    if not imgs:
        return
    first, rest = imgs[0], imgs[1:]
    first.save(out_pdf, save_all=True, append_images=rest)


def parse_cluster_from_name(name: str) -> str:
    m = re.search(r"cluster[_\-]?(\d+)", name)
    return m.group(1) if m else "?"


def caption_from_filename(p: Path, extra: str = "") -> str:
    """파일명에서 핵심 메타 추출 (id, tag 등) + 보조 텍스트"""
    stem = p.stem
    # label_xxx__waterfall_id123__cluster2__median
    m_id = re.search(r"id(\d+)", stem)
    m_tag = re.search(r"__(mean|max|min|fp|fn)", stem)
    s_id = m_id.group(1) if m_id else ""
    s_tag = m_tag.group(1).upper() if m_tag else ""
    s_clu = parse_cluster_from_name(stem)
    bits = []
    if s_clu != "?":
        bits.append(f"C{s_clu}")
    if s_tag:
        bits.append(s_tag)
    if s_id:
        bits.append(f"id={s_id}")
    if extra:
        bits.append(extra)
    return " · ".join(bits)


# ========= Figure 8: Cluster-level representatives =========
def build_figure8_for_label(
    label: str,
    max_clusters: int = 4,
    per_cluster_modes: Tuple[str, ...] = ("median", "max", "min"),
    select_by: str = "largest",  # or "event_rate"
    ncols: int = 3,
):
    """
    representatives_dbscan/cluster_k/ 에 있는 median/max/min png를 한 장으로.
    max_clusters 개 클러스터만 선별(본문용 축약).
    """
    lab_dir = CLSR / label
    rep_root = lab_dir / "representatives_dbscan"
    stat_csv = lab_dir / f"{label}__dbscan_cluster_stats.csv"
    if not rep_root.exists():
        print(f"[{label}] no representatives_dbscan.")
        return None

    # 클러스터 선택: 크기순 or 사건율순
    clusters = []
    if stat_csv.exists():
        d = pd.read_csv(stat_csv)
        if select_by == "event_rate" and "event_rate" in d.columns:
            d = d.sort_values(["event_rate", "n"], ascending=[False, False])
        else:
            d = d.sort_values(["n", "event_rate"], ascending=[False, False])
        clusters = [int(x) for x in d["cluster"].tolist()]
    else:
        # 폴더 스캔
        clusters = sorted(
            [int(parse_cluster_from_name(p.name)) for p in rep_root.glob("cluster_*")]
        )

    clusters = clusters[:max_clusters]
    panel_imgs, panel_caps = [], []
    for c in clusters:
        cdir = rep_root / f"cluster_{c}"
        for mode in per_cluster_modes:
            # label__waterfall_id###__cluster{c}__{mode}.png
            cand = sorted(cdir.glob(f"{label}__waterfall_id*__cluster{c}__{mode}.png"))
            if not cand:
                continue
            p = cand[0]
            cap_extra = read_topk_txt_if_any(p)
            cap = caption_from_filename(p) + (f" · {cap_extra}" if cap_extra else "")
            panel_imgs.append(Image.open(p).convert("RGB"))
            panel_caps.append(cap)

    if not panel_imgs:
        print(f"[{label}] no panels for Figure8.")
        return None

    fig = montage(panel_imgs, panel_caps, ncols=ncols, pad=28, cap_h=46, font_size=18)
    out_path = OUT / f"Figure8_{label}_representatives.png"
    fig.save(out_path, quality=95)
    print(f"[OK] Figure8 saved → {out_path}")
    return out_path


def build_figure8_all_labels(labels: List[str]):
    outs = []
    for lb in labels:
        p = build_figure8_for_label(
            lb,
            max_clusters=4,
            per_cluster_modes=("mean", "max", "min"),
            select_by="largest",
            ncols=3,
        )
        if p:
            outs.append(p)
    # 멀티페이지 PDF로도 저장
    if outs:
        pdf = OUT / "Figure8_ClusterRepresentatives_ALL.pdf"
        save_multipage_pdf(outs, pdf)
        print(f"[OK] multi-page PDF → {pdf}")


# ========= Figure 9: Decision boundary (borderline vs TP) =========
def build_figure9_for_label(label: str, pairs: int = 3, ncols: int = 2):
    """
    personal_waterfall_figures/label_xx/{borderline,true_positive}
    요에서 같은 outcome의 borderline K개와 TP K개를 좌/우 비교 그리드로.
    """
    base = PWF / label
    bdir = base / "borderline"
    tdir = base / "true_positive"
    if not (bdir.exists() and tdir.exists()):
        print(f"[{label}] missing borderline/true_positive dirs.")
        return None

    bsum = sorted(bdir.glob(f"summary__{label}__borderline_k.csv"))
    tsum = sorted(tdir.glob(f"summary__{label}__tp.csv"))
    if not (bsum and tsum):
        print(f"[{label}] missing summary csvs.")
        return None

    db = pd.read_csv(bsum[0])
    dt = pd.read_csv(tsum[0])
    # 상위 probability 기준 정렬(이미 정렬되어 있을 가능성 큼)
    db = db.sort_values("y_prob", ascending=False).head(pairs)
    dt = dt.sort_values("y_prob", ascending=False).head(pairs)

    imgs, caps = [], []
    for (_, rb), (_, rt) in zip(db.iterrows(), dt.iterrows()):
        # 아이디 기준 이미지 찾기
        bid = int(rb["idx"])
        tid = int(rt["idx"])
        bp = bdir / f"waterfall_patient_{bid}.png"
        tp = tdir / f"waterfall_patient_{tid}.png"
        if not (bp.exists() and tp.exists()):
            continue
        for pth, meta in [(bp, rb), (tp, rt)]:
            cap_extra = read_topk_txt_if_any(pth)
            thr = (
                f", thr={meta['youden_thr']:.3f}"
                if not pd.isna(meta.get("youden_thr", np.nan))
                else ""
            )
            cap = f"{'BORDER' if pth==bp else 'TP'} · id={int(meta['idx'])} · y_true={int(meta['y_true'])}, y_pred={int(meta['y_pred'])}, y_prob={meta['y_prob']:.3f}{thr}"
            if cap_extra:
                cap += f" · {cap_extra}"
            imgs.append(Image.open(pth).convert("RGB"))
            caps.append(cap)

    if not imgs:
        print(f"[{label}] no panels for Figure9.")
        return None

    # 2열(좌: Borderline, 우: TP) 고정
    fig = montage(imgs, caps, ncols=ncols, pad=32, cap_h=48, font_size=18)
    out_path = OUT / f"Figure9_{label}_decision_boundary.png"
    fig.save(out_path, quality=95)
    print(f"[OK] Figure9 saved → {out_path}")
    return out_path


def build_figure9_all_labels(labels: List[str]):
    outs = []
    for lb in labels:
        p = build_figure9_for_label(lb, pairs=3, ncols=2)
        if p:
            outs.append(p)
    if outs:
        pdf = OUT / "Figure9_DecisionBoundary_ALL.pdf"
        save_multipage_pdf(outs, pdf)
        print(f"[OK] multi-page PDF → {pdf}")


# ========= Supplement eFigure S6: Cluster gallery (모든 클러스터) =========
def build_supp_cluster_gallery(label: str, ncols=3):
    lab_dir = CLSR / label / "representatives_dbscan"
    if not lab_dir.exists():
        print(f"[{label}] no representatives_dbscan.")
        return None
    pages = []
    for cdir in sorted(
        lab_dir.glob("cluster_*"), key=lambda p: int(parse_cluster_from_name(p.name))
    ):
        pngs = sorted(
            cdir.glob(f"{label}__waterfall_id*__cluster*__(mean|max|min|fp|fn).png")
        )
        if not pngs:
            continue
        imgs = [Image.open(p).convert("RGB") for p in pngs]
        caps = [caption_from_filename(p, extra=read_topk_txt_if_any(p)) for p in pngs]
        page = montage(imgs, caps, ncols=ncols, pad=28, cap_h=46, font_size=18)
        tmp = OUT / f"_tmp_{label}_cluster{parse_cluster_from_name(cdir.name)}.png"
        page.save(tmp, quality=95)
        pages.append(tmp)
    if pages:
        out_pdf = OUT / f"eFigureS6_{label}_ClusterGallery.pdf"
        save_multipage_pdf(pages, out_pdf)
        print(f"[OK] eFigure S6 saved → {out_pdf}")
        # 임시 PNG 삭제(원하면 유지 가능)
        for p in pages:
            try:
                p.unlink()
            except:
                pass


# ========= Supplement eFigure S7: Personal waterfalls (TP/FN galleries) =========
def build_supp_personal_gallery(
    label: str, ncols=3, which=("true_positive", "false_negative", "borderline")
):
    base = PWF / label
    pages = []
    for mode in which:
        mdir = base / mode
        if not mdir.exists():
            continue
        pngs = sorted(mdir.glob("waterfall_patient_*.png"))
        if not pngs:
            continue
        # 여러 페이지로 나눔 (1페이지 당 ncols*rows = 9 가정)
        per_page = ncols * 3
        for i in range(0, len(pngs), per_page):
            chunk = pngs[i : i + per_page]
            imgs = [Image.open(p).convert("RGB") for p in chunk]
            caps = []
            # summary csv 로 메타 읽기 (optional)
            scsv = list(mdir.glob("summary__*.csv"))
            meta = None
            if scsv:
                try:
                    meta = pd.read_csv(scsv[0]).set_index("idx")
                except Exception:
                    meta = None
            for p in chunk:
                m = re.search(r"waterfall_patient_(\d+)", p.stem)
                cap_extra = read_topk_txt_if_any(p)
                if meta is not None and m:
                    iidx = int(m.group(1))
                    if iidx in meta.index:
                        r = meta.loc[iidx]
                        thr = (
                            f", thr={r['youden_thr']:.3f}"
                            if not pd.isna(r.get("youden_thr", np.nan))
                            else ""
                        )
                        cap = f"{mode.upper()} · id={iidx} · y_true={int(r['y_true'])}, y_pred={int(r['y_pred'])}, y_prob={r['y_prob']:.3f}{thr}"
                    else:
                        cap = f"{mode.upper()} · id={iidx}"
                else:
                    cap = mode.upper()
                if cap_extra:
                    cap += f" · {cap_extra}"
                caps.append(cap)
            page = montage(imgs, caps, ncols=ncols, pad=28, cap_h=46, font_size=18)
            tmp = OUT / f"_tmp_{label}_{mode}_{i//per_page+1}.png"
            page.save(tmp, quality=95)
            pages.append(tmp)
    if pages:
        out_pdf = OUT / f"eFigureS7_{label}_PersonalWaterfalls.pdf"
        save_multipage_pdf(pages, out_pdf)
        print(f"[OK] eFigure S7 saved → {out_pdf}")
        for p in pages:
            try:
                p.unlink()
            except:
                pass


# ========= eTable S6: Case cards (CSV/XLSX) =========
def build_case_cards(labels: List[str]):
    rows = []
    # representatives_dbscan
    for lb in labels:
        rep_root = CLSR / lb / "representatives_dbscan"
        for p in sorted(rep_root.glob("cluster_*/*.png")):
            kind = "cluster_rep"
            m_id = re.search(r"id(\d+)", p.stem)
            idx = int(m_id.group(1)) if m_id else None
            cluster = parse_cluster_from_name(p.stem)
            tag = re.search(r"__(mean|max|min|fp|fn)", p.stem)
            tag = tag.group(1) if tag else ""
            top = read_topk_txt_if_any(p)
            rows.append(
                dict(
                    source=kind,
                    outcome=lb,
                    cluster=cluster,
                    case_id=idx,
                    tag=tag,
                    path=str(p),
                    top3=top,
                )
            )

    # personal_waterfall_figures
    for lb in labels:
        for mode in ("borderline", "true_positive", "false_negative"):
            base = PWF / lb / mode
            scsv = list(base.glob("summary__*.csv"))
            meta = pd.read_csv(scsv[0]).set_index("idx") if scsv else None
            for p in sorted(base.glob("waterfall_patient_*.png")):
                kind = "personal"
                m_id = re.search(r"waterfall_patient_(\d+)", p.stem)
                idx = int(m_id.group(1)) if m_id else None
                top = read_topk_txt_if_any(p)
                y_true = y_pred = y_prob = thr = None
                if meta is not None and idx in meta.index:
                    r = meta.loc[idx]
                    y_true = int(r["y_true"])
                    y_pred = int(r["y_pred"])
                    y_prob = float(r["y_prob"])
                    thr = (
                        float(r["youden_thr"])
                        if "youden_thr" in r and not pd.isna(r["youden_thr"])
                        else None
                    )
                rows.append(
                    dict(
                        source=kind,
                        outcome=lb,
                        cluster="",
                        case_id=idx,
                        tag=mode,
                        path=str(p),
                        y_true=y_true,
                        y_pred=y_pred,
                        y_prob=y_prob,
                        youden_thr=thr,
                        top3=top,
                    )
                )
    df = pd.DataFrame(rows)
    out_csv = OUT / "eTableS6_CaseCards.csv"
    df.to_csv(out_csv, index=False)
    try:
        xlsx = OUT / "eTableS6_CaseCards.xlsx"
        with pd.ExcelWriter(xlsx) as w:
            for g, sub in df.groupby("outcome"):
                sub.to_excel(w, sheet_name=g, index=False)
            df.to_excel(w, sheet_name="All", index=False)
        print(f"[OK] eTable S6 → {xlsx}")
    except Exception as e:
        print("[WARN] Excel write:", e)
    print(f"[OK] eTable S6 → {out_csv}")


# ========= 실행 =========
if __name__ == "__main__":
    # 프로젝트에서 쓰는 outcome 라벨
    labels = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

    # Figure 8 (본문): 각 outcome당 대표 클러스터 4개 × (median/max/min)
    build_figure8_all_labels(labels)

    # Figure 9 (본문): 각 outcome당 borderline vs TP 페어 3쌍
    build_figure9_all_labels(labels)

    # Supplement: 클러스터 갤러리(eFigure S6) & 개인 워터폴(eFigure S7)
    for lb in labels:
        build_supp_cluster_gallery(lb, ncols=3)
        build_supp_personal_gallery(
            lb, ncols=3, which=("true_positive", "false_negative", "borderline")
        )

    # eTable S6: 케이스 카드(이미지 경로/메타/Top3)
    build_case_cards(labels)

## Top 2 제외 heatmap

In [ ]:
# =============== 경로/전역 ===============
MODEL_DIR = str(PROJECT_ROOT / "results/step2_modeling/251127_Final_modeling_globalunion25")
OUT_ROOT = (
    Path(MODEL_DIR) / "figures/clinic_interpretation/PatientClusters/cluster_no_top2"
)
OUT_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# # =============== 유틸 ===============
# def _standardize(X: np.ndarray) -> np.ndarray:
#     X = np.asarray(X, dtype=float)
#     X = np.nan_to_num(
#         X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
#     )
#     return (X - X.mean(0)) / (X.std(0) + 1e-9)


# def save_png_svg(figpath: Path, **savefig_kwargs):
#     plt.savefig(figpath, **savefig_kwargs)
#     try:
#         plt.savefig(
#             figpath.with_suffix(".svg"),
#             bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
#         )
#     except Exception:
#         pass
#     plt.close()


# # =============== Youden 임계값 ===============
# from sklearn.metrics import roc_curve


# def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
#     for sep in ["__", "___"]:
#         f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
#         if f.exists():
#             try:
#                 dfy = pd.read_csv(f)
#                 if "threshold" in dfy.columns:
#                     return float(dfy.loc[0, "threshold"])
#             except Exception:
#                 pass
#     return None


# def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
#     fpr, tpr, thr = roc_curve(y_true, y_prob)
#     J = tpr - fpr
#     return float(thr[int(np.argmax(J))])


# # =============== SHAP 로딩 ===============
# def get_shap_matrix(label: str):
#     """
#     Returns:
#       expl: shap.Explanation (n, p)
#       vals: ndarray (n, p)
#       feat_names: list[str] length p (cleaned)
#       y_prob: ndarray (n,)
#       y_true: ndarray (n,)
#       X_te_t: ndarray (n, p_after_preproc)
#       df_pred: DataFrame with y_prob[, y_true, y_pred]
#     """
#     # ⚠️ 기존 프로젝트의 함수 가정
#     expl, X_te_t, y_true, df_pred = shap_on_test(label)

#     vals = np.asarray(expl.values, dtype=float)
#     vals = np.nan_to_num(
#         vals,
#         nan=0.0,
#         posinf=np.finfo(float).max / 1e6,
#         neginf=-np.finfo(float).max / 1e6,
#     )

#     if getattr(expl, "feature_names", None) is not None:
#         raw_names = list(expl.feature_names)
#     else:
#         raw_names = [f"f{i}" for i in range(vals.shape[1])]
#     feat_names = [clean_name(n) for n in raw_names]

#     y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
#     y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
#     return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# # =============== k-distance & eps 추정 ===============
# def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
#     nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
#     nn.fit(X_latent)
#     dists, _ = nn.kneighbors(X_latent)
#     kth = np.sort(dists[:, -1])
#     base_eps = float(np.quantile(kth, pct))
#     return base_eps, kth


# def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
#     plt.figure(figsize=(4, 3))
#     plt.plot(np.arange(len(kth_sorted)), kth_sorted)
#     plt.xlabel("Points (sorted)")
#     plt.ylabel(f"{k}-NN distance")
#     plt.title(f"k-distance curve (k={k})")
#     plt.tight_layout()
#     save_png_svg(out_path, dpi=300, bbox_inches="tight")


# # =============== 히트맵 ===============
# def _agg_abs(values: np.ndarray, how="mean"):
#     return (
#         np.abs(values).mean(axis=0)
#         if how == "mean"
#         else np.mean(np.abs(values), axis=0)
#     )


# def plot_cluster_heatmap(
#     vals,
#     feat_names,
#     labels,
#     topN,
#     out_path: Path,
#     cmap: str = "rocket",
#     agg: str = "mean",
#     skip_top: int = 0,
# ):
#     """
#     반환:
#       {
#         "raw_all":  DataFrame (clusters × all_features) : 클러스터별 원시 집계 |SHAP|
#         "raw_top":  DataFrame (clusters × topN)         : 히트맵에 사용된 TopN의 원시 |SHAP|
#         "norm_top": DataFrame (clusters × topN)         : TopN의 행 정규화(상대값)
#         "feat_top": List[str]                           : TopN 피처명
#         "clusters": np.ndarray                          : 사용된 클러스터 ID (노이즈 -1 제외)
#         "fig_raw_path": Path                            : raw heatmap PNG 경로
#         "fig_norm_path": Path                           : normalized heatmap PNG 경로(= out_path)
#       }
#     """
#     from textwrap import fill

#     # 노이즈(-1) 제외
#     clusters = [c for c in sorted(np.unique(labels)) if c != -1]
#     if not clusters:
#         return None

#     # --- (A) 클러스터별 원시 집계 |SHAP|
#     rows = []
#     for c in clusters:
#         mask = labels == c
#         rows.append(_agg_abs(vals[mask], how=agg))  # shape: (p,)
#     M = np.vstack(rows)  # (n_clusters, n_features)

#     # --- (B) 히트맵에서 쓸 TopN 선택 (클러스터 max 기준)
#     global_imp = _agg_abs(vals, how=agg)  # 전반적 중요도 (median|SHAP| 또는 mean|SHAP|)
#     hetero = M.max(axis=0) - M.min(axis=0)  # 간단한 이질성 점수 (max - min)

#     topK_imp = max(topN * 2, 30)  # 후보 풀(전반적 중요도 상위 K)
#     cand = np.argsort(global_imp)[::-1][:topK_imp]

#     # [수정] 이질성(hetero) 순으로 정렬된 인덱스에서 앞부분(skip_top)을 건너뛰고 선택
#     sorted_candidates = cand[np.argsort(hetero[cand])[::-1]]
#     top_idx = sorted_candidates[skip_top : skip_top + topN]

#     M_top = M[:, top_idx]
#     feat_top = [feat_names[i] for i in top_idx]

#     # --- (C) 행 정규화(클러스터 내부 상대값)
#     row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
#     M_disp = M_top / row_max  # normalized topN

#     # --- (D) DataFrame 구성
#     idx_names = [f"cluster_{c}" for c in clusters]
#     df_raw_all = pd.DataFrame(M, index=idx_names, columns=feat_names)
#     df_raw_top = pd.DataFrame(M_top, index=idx_names, columns=feat_top)
#     df_norm_top = pd.DataFrame(M_disp, index=idx_names, columns=feat_top)

#     # ---------- (E) Normalized Heatmap (기존 파일명 유지) ----------
#     xticks = [f"Cluster {c}" for c in clusters]
#     yticks = [fill(f, width=28) for f in feat_top]
#     fig_w = max(5, 0.8 * len(clusters))
#     fig_h = max(4, 0.35 * len(yticks))

#     fig, ax = plt.subplots(figsize=(fig_w, fig_h))
#     sns.heatmap(
#         df_norm_top.T.values,
#         cmap=cmap,
#         cbar_kws={"label": "Relative importance"},
#         xticklabels=xticks,
#         yticklabels=yticks,
#         linewidths=0.3,
#         linecolor="white",
#         ax=ax,
#     )
#     ax.set_title(
#         f"Cluster × Feature importance ({agg} |SHAP|)",  #  – Row-normalized
#         fontsize=12,
#         weight="bold",
#         pad=12,
#     )
#     ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
#     ax.set_ylabel("Top features", fontsize=10, labelpad=6)
#     ax.tick_params(
#         axis="x",
#         labelrotation=30,
#         labelsize=9,
#         bottom=True,
#         length=4,
#         width=1,
#         color="black",
#     )
#     ax.tick_params(
#         axis="y", labelsize=8, pad=4, left=True, length=4, width=1, color="black"
#     )
#     ax.collections[0].colorbar.ax.tick_params(labelsize=9)
#     plt.tight_layout()
#     save_png_svg(out_path, dpi=300, bbox_inches="tight")

#     # ---------- (F) Raw Heatmap (새 파일) ----------
#     out_path_raw = out_path.with_name(out_path.stem + "__RAW.png")
#     fig, ax = plt.subplots(figsize=(fig_w, fig_h))
#     sns.heatmap(
#         df_raw_top.T.values,
#         cmap=cmap,
#         cbar_kws={"label": f"{agg} |SHAP| (raw)"},
#         xticklabels=xticks,
#         yticklabels=yticks,
#         linewidths=0.3,
#         linecolor="white",
#         ax=ax,
#     )
#     ax.set_title(
#         f"Cluster × Feature importance ({agg} |SHAP|) – Raw",
#         fontsize=12,
#         weight="bold",
#         pad=12,
#     )
#     ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
#     ax.set_ylabel("Top features", fontsize=10, labelpad=6)
#     ax.tick_params(
#         axis="x",
#         labelrotation=30,
#         labelsize=9,
#         bottom=True,
#         length=4,
#         width=1,
#         color="black",
#     )
#     ax.tick_params(
#         axis="y", labelsize=8, pad=4, left=True, length=4, width=1, color="black"
#     )
#     ax.collections[0].colorbar.ax.tick_params(labelsize=9)
#     plt.tight_layout()
#     save_png_svg(out_path_raw, dpi=300, bbox_inches="tight")

#     return {
#         "raw_all": df_raw_all,
#         "raw_top": df_raw_top,
#         "norm_top": df_norm_top,
#         "feat_top": feat_top,
#         "clusters": np.array(clusters),
#         "fig_raw_path": out_path_raw,
#         "fig_norm_path": out_path,
#     }


# # =============== 대표 케이스 ===============
# def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
#     reps = []
#     for c in np.unique(labels):
#         if c == -1:  # 노이즈 제외(원하면 포함 가능)
#             continue
#         idx = np.where(labels == c)[0]
#         probs = y_prob[idx]
#         med = np.mean(probs)
#         med_idx = idx[np.argmin(np.abs(probs - med))]
#         max_idx = idx[np.argmax(probs)]
#         min_idx = idx[np.argmin(probs)]
#         reps.append(
#             {
#                 "cluster": int(c),
#                 "mean_idx": int(med_idx),
#                 "max_idx": int(max_idx),
#                 "min_idx": int(min_idx),
#             }
#         )
#     return reps


# def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
#     reps = []
#     for c in np.unique(labels):
#         if c == -1:
#             continue
#         idx = np.where(labels == c)[0]
#         d = df_pred.iloc[idx]
#         rec = {"cluster": int(c)}
#         fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
#         fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
#         if len(fp):
#             rec["fp_idx"] = int(fp["y_prob"].idxmax())
#         if len(fn):
#             rec["fn_idx"] = int(fn["y_prob"].idxmin())
#         reps.append(rec)
#     return reps


# def save_local_plots(
#     label: str,
#     expl: shap.Explanation,
#     idx: int,
#     out_dir: Path,
#     feat_names=None,
#     max_display=15,
#     df_pred: pd.DataFrame | None = None,
#     youden_thr: float | None = None,
#     name_tag: str | None = None,
# ):
#     out_dir.mkdir(parents=True, exist_ok=True)
#     y_t = y_p = None
#     y_prob = None
#     thr_txt = ""
#     if df_pred is not None:
#         if "y_true" in df_pred.columns:
#             y_t = int(df_pred.loc[idx, "y_true"])
#         if "y_prob" in df_pred.columns:
#             y_prob = float(df_pred.loc[idx, "y_prob"])
#         if "y_pred" in df_pred.columns:
#             y_p = int(df_pred.loc[idx, "y_pred"])
#         else:
#             thr = youden_thr if youden_thr is not None else 0.5
#             y_p = (
#                 int((df_pred.loc[idx, "y_prob"]) >= thr)
#                 if "y_prob" in df_pred.columns
#                 else None
#             )
#         if youden_thr is not None:
#             thr_txt = f", thr={youden_thr:.3f}"

#     # waterfall
#     plt.figure(figsize=(10, 8))
#     try:
#         shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
#     except Exception:
#         row = expl[idx]
#         shap.plots.waterfall(row, show=False, max_display=max_display)
#     title_bits = [f"{label} | id={idx}"]
#     if y_t is not None:
#         title_bits.append(f"y_true={y_t}")
#     if y_p is not None:
#         title_bits.append(f"y_pred={y_p}")
#     if y_prob is not None:
#         title_bits.append(f"y_prob={y_prob:.3f}")
#     if name_tag:
#         title_bits.append(f"[{name_tag}]")
#     ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
#     plt.title(ttl)
#     plt.tight_layout()
#     save_png_svg(
#         out_dir
#         / (
#             f"{label}__waterfall_id{idx}"
#             + (f"__{name_tag}" if name_tag else "")
#             + ".png"
#         ),
#         dpi=300,
#         bbox_inches="tight",
#     )

#     # force (옵션)
#     try:
#         plt.figure(figsize=(7, 2.8))
#         shap.plots.force(
#             expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
#         )
#         plt.tight_layout()
#         save_png_svg(
#             out_dir
#             / (
#                 f"{label}__force_id{idx}"
#                 + (f"__{name_tag}" if name_tag else "")
#                 + ".png"
#             ),
#             dpi=300,
#         )
#     except Exception:
#         plt.close()


# # =============== DBSCAN 통계/요약 ===============
# def add_dbscan_stats(
#     out_dir: Path,
#     label: str,
#     y_true: np.ndarray,
#     y_prob: np.ndarray,
#     labels: np.ndarray,
# ):
#     lab = np.asarray(labels)
#     clusters = [c for c in np.unique(lab) if c != -1]
#     rows = []
#     for c in clusters:
#         m = lab == c
#         rows.append(
#             dict(
#                 cluster=int(c),
#                 n=int(m.sum()),
#                 event_rate=float(np.mean(y_true[m] == 1)),
#                 mean_y_prob=float(np.mean(y_prob[m])),
#             )
#         )
#     df = pd.DataFrame(rows).sort_values("n", ascending=False)

#     chi_p = np.nan
#     kw_p = np.nan
#     if len(clusters) >= 2:
#         table = [
#             [
#                 int(np.sum((lab == c) & (y_true == 1))),
#                 int(np.sum((lab == c) & (y_true == 0))),
#             ]
#             for c in clusters
#         ]
#         chi_p = chi2_contingency(table)[1]
#         groups = [y_prob[lab == c] for c in clusters]
#         kw_p = kruskal(*groups).pvalue

#     df["p_event_rate(chi2)"] = chi_p
#     df["p_yprob(kruskal)"] = kw_p
#     df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
#     return df


# # =============== 메인 루틴 ===============
# def run_cluster_analysis_for_label(
#     label: str,
#     umap_n_components: int = 12,  # 군집용 UMAP 차원
#     umap_n_neighbors: int = 12,
#     umap_min_dist: float = 0.01,
#     umap_metric: str = "euclidean",  # cosine
#     min_samples: int = 6,  # DBSCAN
#     eps: float | None = None,
#     dbscan_metric: str = "euclidean",
#     target_min_clusters: int = 2,
#     max_noise_ratio: float = 0.45,
#     max_trials: int = 5,
#     topN: int = 10,
#     agg_for_heatmap: str = "mean",
#     skip_top: int = 0,
#     save_representatives: bool = True,
#     random_state: int = 42,
# ):
#     assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
#     print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
#     out_dir = OUT_ROOT / label
#     out_dir.mkdir(parents=True, exist_ok=True)

#     # 0) 데이터 로드
#     expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

#     # Youden 임계값 & df_pred 보정
#     ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
#     if ythr is None:
#         ythr = _compute_youden_from_preds(
#             np.asarray(y_true).astype(int), np.asarray(y_prob)
#         )
#     if "y_true" not in df_pred.columns:
#         df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
#     if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
#         df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

#     # 1) 표준화 & UMAP(고차원; 군집용)
#     # V = _standardize(vals) # 표준화
#     V = vals
#     um_hi = UMAP.UMAP(
#         n_components=umap_n_components,
#         n_neighbors=umap_n_neighbors,
#         min_dist=umap_min_dist,
#         metric=umap_metric,
#         random_state=random_state,
#     )
#     X_hi = um_hi.fit_transform(V)

#     # 2) eps 초기값 (k-distance) & 곡선 저장
#     if eps is None:
#         eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.9)  # 0.85
#         eps = eps0
#         save_kdist_plot(
#             kth,
#             k=min_samples,
#             out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
#         )

#     # 3) DBSCAN 적응형 루프
#     trial = 0
#     labels = None
#     info_hist = []
#     cur_min_samples = int(min_samples)
#     cur_eps = float(eps)

#     while trial < max_trials:
#         db = DBSCAN(
#             eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
#         )
#         labels = db.fit_predict(X_hi)  # X_hi
#         noise_ratio = float(np.mean(labels == -1))
#         n_clusters = int(len(set(labels) - {-1}))

#         info_hist.append(
#             dict(
#                 trial=trial,
#                 eps=cur_eps,
#                 min_samples=cur_min_samples,
#                 n_clusters=n_clusters,
#                 noise_ratio=noise_ratio,
#             )
#         )

#         if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
#             break

#         # 조정 규칙
#         if noise_ratio > max_noise_ratio:
#             cur_eps *= 1.20
#             if cur_min_samples > 3:
#                 cur_min_samples -= 1
#         elif n_clusters < target_min_clusters:
#             cur_eps *= 0.88
#         else:
#             cur_eps *= 1.05

#         trial += 1

#     # all-noise 대비: 마지막 완화 시도 1회
#     if (labels is None) or np.all(labels == -1):
#         cur_eps *= 1.30
#         db = DBSCAN(
#             eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
#         )
#         labels = db.fit_predict(X_hi)  # X_hi
#         noise_ratio = float(np.mean(labels == -1))
#         n_clusters = int(len(set(labels) - {-1}))
#         info_hist.append(
#             dict(
#                 trial="final_relax",
#                 eps=cur_eps,
#                 min_samples=cur_min_samples,
#                 n_clusters=n_clusters,
#                 noise_ratio=noise_ratio,
#             )
#         )

#     # 4) 시각화용 2D UMAP (시각화 전용)
#     if umap_n_components == 2:
#         Z2 = X_hi
#     else:
#         um_2d = UMAP.UMAP(
#             n_components=2,
#             n_neighbors=max(umap_n_neighbors, 15),
#             min_dist=max(umap_min_dist, 0.05),
#             metric=umap_metric,
#             random_state=random_state,
#         )
#         Z2 = um_2d.fit_transform(V)

#     # 5) 라벨 매핑 저장(노이즈 포함)
#     m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
#     m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

#     # 6) 2D 플롯 (클러스터 색)
#     uniq = sorted(np.unique(labels))
#     # 클러스터 개수가 많을 수 있으므로 tab10 사용 권장
#     cmap = plt.get_cmap("tab10")

#     # 범례 공간 확보를 위해 가로 사이즈를 약간 늘림
#     plt.figure(figsize=(8.5, 6.0))

#     # 1. 노이즈(-1) 먼저 그리기 (배경으로 깔리게 처리)
#     if -1 in uniq:
#         mask = labels == -1
#         plt.scatter(
#             Z2[mask, 0],
#             Z2[mask, 1],
#             c="#e0e0e0",  # 아주 연한 회색
#             edgecolor="#bbbbbb",  # 테두리는 조금 진하게
#             linewidth=0.1,
#             s=15,  # 크기는 작게
#             alpha=0.4,  # 투명하게
#             label="Noise",  # 범례 이름
#             zorder=1,  # 가장 뒤쪽에 배치
#         )

#     # 2. 실제 클러스터 그리기 Loop
#     for c in uniq:
#         if c == -1:
#             continue
#         mask = labels == c
#         color = cmap(c % 20)  # 색상 순환 적용

#         plt.scatter(
#             Z2[mask, 0],
#             Z2[mask, 1],
#             c=[color],  # 단일 색상 적용
#             s=22,  # 클러스터 포인트는 조금 더 크게
#             alpha=0.9,  # 진하게
#             label=f"Cluster {c}",
#             edgecolor="white",  # 포인트 구분감
#             linewidth=0.3,
#             zorder=2,  # 노이즈보다 앞쪽에 배치
#         )

#     title = f"UMAP + DBSCAN ({umap_n_components}D)"
#     # 제목에 전체 대비 노이즈 비율 표기
#     n_noise = np.sum(labels == -1)
#     n_total = len(labels)
#     plt.title(
#         f"{title}\nTotal: {n_total}, Clusters: {len(set(labels)-{-1})}, Noise: {n_noise} ({n_noise/n_total:.1%})",
#         fontsize=11,
#     )
#     plt.xlabel("UMAP-1")
#     plt.ylabel("UMAP-2")

#     # 범례 설정 (그래프 영역 밖 우측 상단에 배치)
#     plt.legend(
#         bbox_to_anchor=(1.02, 1),
#         loc="upper left",
#         borderaxespad=0,
#         frameon=False,
#         fontsize=9,
#         markerscale=1.5,  # 범례의 점 크기 키우기
#     )

#     plt.tight_layout()
#     save_png_svg(
#         out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
#     )

#     # 6-2) 2D 플롯 (위험도 색)
#     plt.figure(figsize=(7.2, 6.2))
#     sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
#     plt.colorbar(sc, label="Predicted risk (prob.)")
#     plt.title(f"UMAP(2D) (colored by risk) — {label}")
#     plt.xlabel("UMAP-1")
#     plt.ylabel("UMAP-2")
#     plt.margins(0.02)
#     save_png_svg(
#         out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
#     )

#     # 7) 클러스터 통계/요약 + 히트맵
#     df_stats = add_dbscan_stats(
#         out_dir,
#         label,
#         y_true=np.asarray(y_true),
#         y_prob=np.asarray(y_prob),
#         labels=np.asarray(labels),
#     )
#     ret = plot_cluster_heatmap(
#         vals,
#         feat_names,
#         labels,
#         topN=topN,
#         out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
#         cmap="magma_r",  # rocket_r, viridis_r
#         agg=agg_for_heatmap,
#         skip_top=skip_top,
#     )

#     # === 수치 저장(요청 파일명 규칙) ===
#     if ret is not None:
#         # 모든 피처의 원시 |SHAP| (클러스터 × 전체 피처)
#         ret["raw_all"].to_csv(
#             out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__ALL_FEATURES.csv",
#             index=True,
#         )
#         # 히트맵에 쓰인 TopN의 원시 |SHAP|
#         ret["raw_top"].to_csv(
#             out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_RAW.csv",
#             index=True,
#         )
#         # 히트맵에 쓰인 TopN의 행 정규화(상대값)
#         ret["norm_top"].to_csv(
#             out_dir
#             / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_ROWNORM.csv",
#             index=True,
#         )

#     # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
#     if save_representatives:
#         rep_dir = out_dir / "representatives_dbscan"
#         # mean/max/min
#         reps = pick_representatives(y_prob, np.asarray(labels))
#         for rep in reps:
#             c = rep["cluster"]
#             for tag in ["mean_idx", "max_idx", "min_idx"]:
#                 i = rep[tag]
#                 name_tag = f"cluster{c}__{tag.replace('_idx','')}"
#                 save_local_plots(
#                     label,
#                     expl,
#                     i,
#                     rep_dir / f"cluster_{c}",
#                     feat_names=feat_names,
#                     max_display=15,
#                     df_pred=df_pred,
#                     youden_thr=ythr,
#                     name_tag=name_tag,
#                 )
#         # FP/FN
#         err_reps = pick_error_representatives(df_pred, np.asarray(labels))
#         for rep in err_reps:
#             c = rep["cluster"]
#             for tag in ["fp_idx", "fn_idx"]:
#                 if tag in rep:
#                     i = rep[tag]
#                     name_tag = f"cluster{c}__{tag.replace('_idx','')}"
#                     save_local_plots(
#                         label,
#                         expl,
#                         i,
#                         rep_dir / f"cluster_{c}",
#                         feat_names=feat_names,
#                         max_display=15,
#                         df_pred=df_pred,
#                         youden_thr=ythr,
#                         name_tag=name_tag,
#                     )

#     # 9) 메타(JSON)
#     meta = dict(
#         timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
#         algo="dbscan_tuned",
#         random_state=random_state,
#         umap=dict(
#             n_components=umap_n_components,
#             n_neighbors=umap_n_neighbors,
#             min_dist=umap_min_dist,
#             metric=umap_metric,
#         ),
#         final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
#         trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
#         n_clusters=int(len(set(labels) - {-1})),
#         noise_ratio=float(np.mean(np.asarray(labels) == -1)),
#         history=info_hist,
#     )
#     with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
#         json.dump(meta, f, indent=2)

#     print(f"-> saved: {out_dir}")
#     return df_stats, labels

In [ ]:
def plot_umap_for_ppt(
    Z2: np.ndarray,
    labels: np.ndarray,
    y_prob: np.ndarray,
    y_true: np.ndarray,  # [NEW] 실제 정답 레이블 추가
    out_path: Path,
    title: str,
):

    # plt.rcParams.update({'font.size': 14})

    # --- [설정] Risk 판단 기준 (Event Rate 기준) ---
    def get_risk_style(event_rate):
        # Event Rate는 보통 확률보다 낮으므로 기준을 조금 조정할 수도 있음 (현행 유지)
        if event_rate < 0.1:
            return "Low", "#4575b4"
        elif event_rate < 0.3:
            return "Moderate", "#7b3294"
        elif event_rate < 0.5:
            return "High", "#f1a340"
        else:
            return "Very High", "#d73027"

    uniq = sorted(np.unique(labels))
    clusters = [c for c in uniq if c != -1]
    n_clusters = len(clusters)

    global_center = np.mean(Z2, axis=0)
    cluster_cmap = plt.get_cmap("tab20" if n_clusters > 10 else "tab10")

    centroids = {}
    for c in clusters:
        centroids[c] = np.mean(Z2[labels == c], axis=0)

    fig, ax = plt.subplots(figsize=(12, 10))
    ax.scatter(
        Z2[:, 0], Z2[:, 1], c="#e0e0e0", s=20, alpha=0.4, edgecolors="none", zorder=0
    )

    label_items = []
    legend_handles = []

    for i, c in enumerate(clusters):
        mask = labels == c
        points = Z2[mask]

        # [변경] 실제 Event Rate 계산
        # y_true가 0/1로 되어 있다고 가정 (1=Event)
        true_vals = y_true[mask]
        event_rate = np.mean(true_vals == 1)

        risk_name, risk_color = get_risk_style(event_rate)  # Event Rate 기준 스타일
        cluster_color = cluster_cmap(i % 20)

        handle = mpatches.Patch(color=cluster_color, label=f"Cluster {c}")
        legend_handles.append(handle)

        if len(points) < 3:
            ax.scatter(
                points[:, 0], points[:, 1], c=cluster_color, s=20, alpha=0.8, zorder=2
            )
            continue

        ax.scatter(
            points[:, 0],
            points[:, 1],
            c=[cluster_color],
            s=20,
            alpha=0.8,
            edgecolors="white",
            linewidth=0.2,
            zorder=2,
        )
        hull = ConvexHull(points)
        hull_coords = points[np.append(hull.vertices, hull.vertices[0])]
        ax.plot(
            hull_coords[:, 0],
            hull_coords[:, 1],
            color=cluster_color,
            linestyle="--",
            linewidth=1.2,
            alpha=0.8,
            zorder=3,
        )
        ax.add_patch(
            mpatches.Polygon(
                hull_coords,
                closed=True,
                fc=cluster_color,
                ec=None,
                alpha=0.08,
                zorder=1,
            )
        )

        # --- 위치 계산 ---
        current_centroid = centroids[c]
        vec_radial = current_centroid - global_center
        vec_repulsion = np.array([0.0, 0.0])
        crowding = 0

        for other_c, other_cent in centroids.items():
            if c == other_c:
                continue
            diff = current_centroid - other_cent
            dist = np.linalg.norm(diff)
            if dist > 1e-4:
                w = 1.0 / (dist**2.0)
                vec_repulsion += (diff / dist) * w
                crowding += w

        final_vec = vec_radial + (vec_repulsion * 4.0)
        if crowding > 0.5 or np.linalg.norm(vec_radial) < 1.0:
            final_vec[1] += (1.0 if i % 2 == 0 else -1.0) * 3.0

        norm = np.linalg.norm(final_vec)
        unit_vec = final_vec / norm if norm > 1e-4 else np.array([0, 1])

        hull_points = points[hull.vertices]
        anchor_point = hull_points[np.argmax(np.dot(hull_points, unit_vec))]

        offset = 2.5 + min(crowding, 3.5)
        text_pos = anchor_point + (unit_vec * offset)

        label_items.append(
            {
                "text_pos": text_pos,
                "anchor_point": anchor_point,
                # [변경] 라벨 텍스트에 Event Rate 표시
                "text_str": f"{risk_name}\n(Event:{event_rate:.2f})",
                "color": risk_color,
            }
        )

    # --- 물리 엔진 ---
    iterations = 50
    min_dist = 3.0
    for _ in range(iterations):
        for i in range(len(label_items)):
            for j in range(i + 1, len(label_items)):
                p1 = label_items[i]["text_pos"]
                p2 = label_items[j]["text_pos"]
                diff = p1 - p2
                dist = np.linalg.norm(diff)
                if dist < min_dist:
                    if dist < 1e-4:
                        force = np.array([0.1, 0.1])
                    else:
                        force = (diff / dist) * (min_dist - dist) * 0.5
                    label_items[i]["text_pos"] += force
                    label_items[j]["text_pos"] -= force

    # --- 축 확장 ---
    all_x = list(Z2[:, 0]) + [item["text_pos"][0] for item in label_items]
    all_y = list(Z2[:, 1]) + [item["text_pos"][1] for item in label_items]
    margin_x = (max(all_x) - min(all_x)) * 0.1
    margin_y = (max(all_y) - min(all_y)) * 0.1
    ax.set_xlim(min(all_x) - margin_x, max(all_x) + margin_x)
    ax.set_ylim(min(all_y) - margin_y, max(all_y) + margin_y)

    # --- Annotation 그리기 ---
    for item in label_items:
        bbox_props = dict(
            boxstyle="round,pad=0.2", fc="white", ec=item["color"], lw=1.5, alpha=0.95
        )

        ax.annotate(
            text=item["text_str"],
            xy=(item["anchor_point"][0], item["anchor_point"][1]),
            xytext=(item["text_pos"][0], item["text_pos"][1]),
            ha="center",
            va="center",
            fontsize=14,
            weight="bold",
            color=item["color"],
            bbox=bbox_props,
            arrowprops=dict(
                arrowstyle="-",
                color="black",
                linewidth=0.8,
                shrinkB=4,
                connectionstyle="arc3,rad=0.1",
            ),
            zorder=10,
        )

    # 범례 및 타이틀
    if -1 in uniq:
        legend_handles.insert(0, mpatches.Patch(color="#e0e0e0", label="Noise"))

    ax.legend(
        handles=legend_handles,
        title="Clusters",
        bbox_to_anchor=(1.02, 1),
        loc="upper left",
        borderaxespad=0,
        frameon=False,
        fontsize=14,
    )

    noise_pct = (labels == -1).sum() / len(labels) if len(labels) > 0 else 0
    full_title = f"{title}\n(Noise ratio={noise_pct:.1%})"

    ax.set_title(full_title, fontsize=14, weight="bold", pad=20)
    ax.set_xlabel("UMAP-1", fontsize=12, labelpad=10)
    ax.set_ylabel("UMAP-2", fontsize=12, labelpad=10)
    # ax.tick_params(axis='both', which='major', labelsize=12)

    ax.set_xticks([])
    ax.set_yticks([])

    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# def plot_umap_with_hulls_and_labels(
#     Z2: np.ndarray,
#     labels: np.ndarray,
#     y_prob: np.ndarray,
#     out_path: Path,
#     title: str,
# ):
#     """
#     [Ultimate Version with Legend]
#     - 우측 상단에 클러스터 범례(Cluster 0, 1...)와 Noise 범례 추가
#     - 물리 엔진(Simulation) 기반 라벨 겹침 방지
#     - 축 자동 확장 및 스마트 배치
#     """
#     import matplotlib.cm as cm
#     import matplotlib.patches as mpatches
#     from scipy.spatial import ConvexHull

#     # --- 설정 ---
#     def get_risk_style(mean_prob):
#         if mean_prob < 0.1: return "Low", "#4575b4"
#         elif mean_prob < 0.3: return "Moderate", "#7b3294"
#         elif mean_prob < 0.5: return "High", "#f1a340"
#         else: return "Very High", "#d73027"

#     uniq = sorted(np.unique(labels))
#     clusters = [c for c in uniq if c != -1]
#     n_clusters = len(clusters)

#     global_center = np.mean(Z2, axis=0)
#     cluster_cmap = plt.get_cmap("tab20" if n_clusters > 10 else "tab10")

#     centroids = {}
#     for c in clusters:
#         centroids[c] = np.mean(Z2[labels == c], axis=0)

#     # 1. 캔버스 설정 (범례 공간 확보를 위해 가로폭 넉넉하게)
#     fig, ax = plt.subplots(figsize=(12, 10))

#     # 배경 산점도
#     ax.scatter(Z2[:, 0], Z2[:, 1], c='#e0e0e0', s=20, alpha=0.4, edgecolors='none', zorder=0)

#     # -------------------------------------------------------
#     # [Step 1] 데이터 수집, 초기 위치, 범례 핸들 생성
#     # -------------------------------------------------------
#     label_items = []
#     legend_handles = [] # 범례용 핸들 리스트

#     # 노이즈가 있으면 범례에 먼저 추가
#     if -1 in uniq:
#         legend_handles.append(mpatches.Patch(color='#e0e0e0', label='Noise'))

#     for i, c in enumerate(clusters):
#         mask = (labels == c)
#         points = Z2[mask]
#         probs = y_prob[mask]

#         mean_p = np.mean(probs)
#         risk_name, risk_color = get_risk_style(mean_p)
#         cluster_color = cluster_cmap(i % 20)

#         # 범례에 추가 (Cluster ID와 색상 매핑)
#         legend_handles.append(mpatches.Patch(color=cluster_color, label=f'Cluster {c}'))

#         if len(points) < 3:
#             ax.scatter(points[:, 0], points[:, 1], c=cluster_color, s=20, alpha=0.8, zorder=2)
#             continue

#         # Hull 그리기
#         ax.scatter(points[:, 0], points[:, 1], c=[cluster_color], s=20, alpha=0.8, edgecolors='white', linewidth=0.2, zorder=2)
#         hull = ConvexHull(points)
#         hull_coords = points[np.append(hull.vertices, hull.vertices[0])]
#         ax.plot(hull_coords[:, 0], hull_coords[:, 1], color=cluster_color, linestyle='--', linewidth=1.2, alpha=0.8, zorder=3)
#         ax.add_patch(mpatches.Polygon(hull_coords, closed=True, fc=cluster_color, ec=None, alpha=0.08, zorder=1))

#         # --- 초기 위치 계산 ---
#         current_centroid = centroids[c]

#         vec_radial = current_centroid - global_center
#         vec_repulsion = np.array([0.0, 0.0])
#         crowding = 0

#         for other_c, other_cent in centroids.items():
#             if c == other_c: continue
#             diff = current_centroid - other_cent
#             dist = np.linalg.norm(diff)
#             if dist > 1e-4:
#                 w = 1.0 / (dist**2.0)
#                 vec_repulsion += (diff / dist) * w
#                 crowding += w

#         final_vec = vec_radial + (vec_repulsion * 4.0)
#         if crowding > 0.5 or np.linalg.norm(vec_radial) < 1.0:
#             final_vec[1] += (1.0 if i % 2 == 0 else -1.0) * 3.0

#         norm = np.linalg.norm(final_vec)
#         unit_vec = final_vec / norm if norm > 1e-4 else np.array([0, 1])

#         hull_points = points[hull.vertices]
#         anchor_point = hull_points[np.argmax(np.dot(hull_points, unit_vec))]

#         offset = 2.5 + min(crowding, 3.5) # 2.0, 3.0
#         text_pos = anchor_point + (unit_vec * offset)

#         label_items.append({
#             'text_pos': text_pos,
#             'anchor_point': anchor_point,
#             'text_str': f"{risk_name}\n(Risk:{mean_p:.2f})",
#             'color': risk_color,
#         })

#     # -------------------------------------------------------
#     # [Step 2] 물리 엔진: 라벨 겹침 해결
#     # -------------------------------------------------------
#     iterations = 50
#     min_dist = 3.0 # 2.5

#     for _ in range(iterations):
#         for i in range(len(label_items)):
#             for j in range(i + 1, len(label_items)):
#                 p1 = label_items[i]['text_pos']
#                 p2 = label_items[j]['text_pos']

#                 diff = p1 - p2
#                 dist = np.linalg.norm(diff)

#                 if dist < min_dist:
#                     if dist < 1e-4:
#                         force = np.array([0.1, 0.1])
#                     else:
#                         force = (diff / dist) * (min_dist - dist) * 0.5

#                     label_items[i]['text_pos'] += force
#                     label_items[j]['text_pos'] -= force

#     # -------------------------------------------------------
#     # [Step 3] 최종 그리기 & 축 확장
#     # -------------------------------------------------------
#     all_x = list(Z2[:, 0]) + [item['text_pos'][0] for item in label_items]
#     all_y = list(Z2[:, 1]) + [item['text_pos'][1] for item in label_items]

#     margin_x = (max(all_x) - min(all_x)) * 0.1
#     margin_y = (max(all_y) - min(all_y)) * 0.1

#     ax.set_xlim(min(all_x) - margin_x, max(all_x) + margin_x)
#     ax.set_ylim(min(all_y) - margin_y, max(all_y) + margin_y)

#     # Annotation 그리기
#     for item in label_items:
#         bbox_props = dict(boxstyle="round,pad=0.2", fc="white", ec=item['color'], lw=1.5, alpha=0.95)

#         ax.annotate(
#             text=item['text_str'],
#             xy=(item['anchor_point'][0], item['anchor_point'][1]),
#             xytext=(item['text_pos'][0], item['text_pos'][1]),
#             ha='center', va='center',
#             fontsize=14, weight='bold',
#             color=item['color'],
#             bbox=bbox_props,
#             arrowprops=dict(
#                 arrowstyle="-", color="black", linewidth=0.8, shrinkB=4,
#                 connectionstyle="arc3,rad=0.1"
#             ),
#             zorder=10
#         )

#     # [NEW] 범례 표시 (Legend)
#     # bbox_to_anchor로 그래프 영역 바깥 우측 상단에 배치
#     ax.legend(
#         handles=legend_handles,
#         title="Clusters",
#         bbox_to_anchor=(1.02, 1),
#         loc='upper left',
#         borderaxespad=0,
#         frameon=False,
#         fontsize=11,
#         title_fontsize=12
#     )

#     # 마무리
#     noise_pct = (labels == -1).sum() / len(labels) if len(labels) > 0 else 0
#     full_title = f"{title}\n(Noise ratio={noise_pct:.1%})"

#     ax.set_title(full_title, fontsize=14, weight='bold', pad=20)
#     ax.set_xlabel("UMAP-1")
#     ax.set_ylabel("UMAP-2")
#     ax.set_xticks([])
#     ax.set_yticks([])

#     plt.tight_layout()
#     save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 유틸 ===============
def _standardize(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    X = np.nan_to_num(
        X, nan=0.0, posinf=np.finfo(float).max / 1e6, neginf=-np.finfo(float).max / 1e6
    )
    return (X - X.mean(0)) / (X.std(0) + 1e-9)


def save_png_svg(figpath: Path, **savefig_kwargs):
    plt.savefig(figpath, **savefig_kwargs)
    try:
        plt.savefig(
            figpath.with_suffix(".svg"),
            bbox_inches=savefig_kwargs.get("bbox_inches", "tight"),
        )
    except Exception:
        pass
    plt.close()


# =============== Youden 임계값 ===============
from sklearn.metrics import roc_curve


def _try_read_youden_threshold(label: str, base_dir: Path = OUT_ROOT.parent):
    for sep in ["__", "___"]:
        f = base_dir / f"{label}{sep}youden_metrics_Test.csv"
        if f.exists():
            try:
                dfy = pd.read_csv(f)
                if "threshold" in dfy.columns:
                    return float(dfy.loc[0, "threshold"])
            except Exception:
                pass
    return None


def _compute_youden_from_preds(y_true: np.ndarray, y_prob: np.ndarray) -> float:
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    J = tpr - fpr
    return float(thr[int(np.argmax(J))])


# =============== SHAP 로딩 ===============
def get_shap_matrix(label: str):
    """
    Returns:
      expl: shap.Explanation (n, p)
      vals: ndarray (n, p)
      feat_names: list[str] length p (cleaned)
      y_prob: ndarray (n,)
      y_true: ndarray (n,)
      X_te_t: ndarray (n, p_after_preproc)
      df_pred: DataFrame with y_prob[, y_true, y_pred]
    """
    # ⚠️ 기존 프로젝트의 함수 가정
    expl, X_te_t, y_true, df_pred = shap_on_test(label)

    vals = np.asarray(expl.values, dtype=float)
    vals = np.nan_to_num(
        vals,
        nan=0.0,
        posinf=np.finfo(float).max / 1e6,
        neginf=-np.finfo(float).max / 1e6,
    )

    if getattr(expl, "feature_names", None) is not None:
        raw_names = list(expl.feature_names)
    else:
        raw_names = [f"f{i}" for i in range(vals.shape[1])]
    feat_names = [clean_name(n) for n in raw_names]

    y_prob = np.asarray(df_pred["y_prob"].values, dtype=float)
    y_true = np.asarray(getattr(y_true, "values", y_true)).astype(int)
    return expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred


# =============== k-distance & eps 추정 ===============
def _kdist_eps_heuristic(X_latent: np.ndarray, k: int = 5, pct: float = 0.85) -> float:
    nn = NearestNeighbors(n_neighbors=max(k, 2), metric="euclidean")
    nn.fit(X_latent)
    dists, _ = nn.kneighbors(X_latent)
    kth = np.sort(dists[:, -1])
    base_eps = float(np.quantile(kth, pct))
    return base_eps, kth


def save_kdist_plot(kth_sorted: np.ndarray, k: int, out_path: Path):
    plt.figure(figsize=(4, 3))
    plt.plot(np.arange(len(kth_sorted)), kth_sorted)
    plt.xlabel("Points (sorted)")
    plt.ylabel(f"{k}-NN distance")
    plt.title(f"k-distance curve (k={k})")
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")


# =============== 히트맵 ===============
def _agg_abs(values: np.ndarray, how="mean"):
    return (
        np.abs(values).mean(axis=0)
        if how == "mean"
        else np.mean(np.abs(values), axis=0)
    )


def plot_cluster_heatmap(
    vals,
    feat_names,
    labels,
    topN,
    out_path: Path,
    cmap: str = "rocket",
    agg: str = "mean",
):
    """
    반환:
      {
        "raw_all":  DataFrame (clusters × all_features) : 클러스터별 원시 집계 |SHAP|
        "raw_top":  DataFrame (clusters × topN)         : 히트맵에 사용된 TopN의 원시 |SHAP|
        "norm_top": DataFrame (clusters × topN)         : TopN의 행 정규화(상대값)
        "feat_top": List[str]                           : TopN 피처명
        "clusters": np.ndarray                          : 사용된 클러스터 ID (노이즈 -1 제외)
        "fig_raw_path": Path                            : raw heatmap PNG 경로
        "fig_norm_path": Path                           : normalized heatmap PNG 경로(= out_path)
      }
    """
    from textwrap import fill

    # 노이즈(-1) 제외
    clusters = [c for c in sorted(np.unique(labels)) if c != -1]
    if not clusters:
        return None

    # --- (A) 클러스터별 원시 집계 |SHAP|
    rows = []
    for c in clusters:
        mask = labels == c
        rows.append(_agg_abs(vals[mask], how=agg))  # shape: (p,)
    M = np.vstack(rows)  # (n_clusters, n_features)

    # --- (B) 히트맵에서 쓸 TopN 선택 (클러스터 max 기준)
    global_imp = _agg_abs(vals, how=agg)  # 전반적 중요도 (median|SHAP| 또는 mean|SHAP|)
    hetero = M.max(axis=0) - M.min(axis=0)  # 간단한 이질성 점수 (max - min)

    topK_imp = max(topN * 2, 30)  # 후보 풀(전반적 중요도 상위 K)
    cand = np.argsort(global_imp)[::-1][:topK_imp]
    top_idx = cand[
        np.argsort(hetero[cand])[::-1][:topN]
    ]  # 후보 안에서 이질성 큰 순 TopN

    M_top = M[:, top_idx]
    feat_top = [feat_names[i] for i in top_idx]

    ###########################################
    # global_imp = _agg_abs(vals, how=agg)  # shape (p,)  # median(|SHAP|)
    # top_idx = np.argsort(global_imp)[::-1][:topN]

    # M_top = M[:, top_idx]
    # feat_top = [feat_names[i] for i in top_idx]
    ###########################################
    # top_idx = np.argsort(M.max(axis=0))[::-1][:topN]
    # M_top = M[:, top_idx]  # raw topN
    # feat_top = [feat_names[i] for i in top_idx]

    # --- (C) 행 정규화(클러스터 내부 상대값)
    row_max = np.maximum(M_top.max(axis=1, keepdims=True), 1e-9)
    M_disp = M_top / row_max  # normalized topN

    # --- (D) DataFrame 구성
    idx_names = [f"cluster_{c}" for c in clusters]
    df_raw_all = pd.DataFrame(M, index=idx_names, columns=feat_names)
    df_raw_top = pd.DataFrame(M_top, index=idx_names, columns=feat_top)
    df_norm_top = pd.DataFrame(M_disp, index=idx_names, columns=feat_top)

    # ---------- (E) Normalized Heatmap (기존 파일명 유지) ----------
    xticks = [f"Cluster {c}" for c in clusters]
    yticks = [fill(f, width=28) for f in feat_top]
    fig_w = max(5, 0.8 * len(clusters))
    fig_h = max(4, 0.35 * len(yticks))

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_norm_top.T.values,
        cmap=cmap,
        cbar_kws={"label": "Relative importance"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",  #  – Row-normalized
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path, dpi=300, bbox_inches="tight")

    # ---------- (F) Raw Heatmap (새 파일) ----------
    out_path_raw = out_path.with_name(out_path.stem + "__RAW.png")
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    sns.heatmap(
        df_raw_top.T.values,
        cmap=cmap,
        cbar_kws={"label": f"{agg} |SHAP|"},
        xticklabels=xticks,
        yticklabels=yticks,
        linewidths=0.3,
        linecolor="white",
        ax=ax,
    )
    ax.set_title(
        f"Cluster × Feature importance ({agg} |SHAP|)",
        fontsize=12,
        weight="bold",
        pad=12,
    )
    ax.set_xlabel("Cluster", fontsize=10, labelpad=6)
    ax.set_ylabel("Top features", fontsize=10, labelpad=6)
    ax.tick_params(axis="x", labelrotation=30, labelsize=9)
    ax.tick_params(axis="y", labelsize=8, pad=4)
    ax.collections[0].colorbar.ax.tick_params(labelsize=9)
    plt.tight_layout()
    save_png_svg(out_path_raw, dpi=300, bbox_inches="tight")

    return {
        "raw_all": df_raw_all,
        "raw_top": df_raw_top,
        "norm_top": df_norm_top,
        "feat_top": feat_top,
        "clusters": np.array(clusters),
        "fig_raw_path": out_path_raw,
        "fig_norm_path": out_path,
    }


# =============== 대표 케이스 ===============
def pick_representatives(y_prob: np.ndarray, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:  # 노이즈 제외(원하면 포함 가능)
            continue
        idx = np.where(labels == c)[0]
        probs = y_prob[idx]
        med = np.mean(probs)
        med_idx = idx[np.argmin(np.abs(probs - med))]
        max_idx = idx[np.argmax(probs)]
        min_idx = idx[np.argmin(probs)]
        reps.append(
            {
                "cluster": int(c),
                "mean_idx": int(med_idx),
                "max_idx": int(max_idx),
                "min_idx": int(min_idx),
            }
        )
    return reps


def pick_error_representatives(df_pred: pd.DataFrame, labels: np.ndarray):
    reps = []
    for c in np.unique(labels):
        if c == -1:
            continue
        idx = np.where(labels == c)[0]
        d = df_pred.iloc[idx]
        rec = {"cluster": int(c)}
        fp = d[(d["y_true"] == 0) & (d["y_pred"] == 1)]
        fn = d[(d["y_true"] == 1) & (d["y_pred"] == 0)]
        if len(fp):
            rec["fp_idx"] = int(fp["y_prob"].idxmax())
        if len(fn):
            rec["fn_idx"] = int(fn["y_prob"].idxmin())
        reps.append(rec)
    return reps


def save_local_plots(
    label: str,
    expl: shap.Explanation,
    idx: int,
    out_dir: Path,
    feat_names=None,
    max_display=15,
    df_pred: pd.DataFrame | None = None,
    youden_thr: float | None = None,
    name_tag: str | None = None,
):
    out_dir.mkdir(parents=True, exist_ok=True)
    y_t = y_p = None
    y_prob = None
    thr_txt = ""
    if df_pred is not None:
        if "y_true" in df_pred.columns:
            y_t = int(df_pred.loc[idx, "y_true"])
        if "y_prob" in df_pred.columns:
            y_prob = float(df_pred.loc[idx, "y_prob"])
        if "y_pred" in df_pred.columns:
            y_p = int(df_pred.loc[idx, "y_pred"])
        else:
            thr = youden_thr if youden_thr is not None else 0.5
            y_p = (
                int((df_pred.loc[idx, "y_prob"]) >= thr)
                if "y_prob" in df_pred.columns
                else None
            )
        if youden_thr is not None:
            thr_txt = f", thr={youden_thr:.3f}"

    # waterfall
    plt.figure(figsize=(10, 8))
    try:
        shap.plots.waterfall(expl[idx], show=False, max_display=max_display)
    except Exception:
        row = expl[idx]
        shap.plots.waterfall(row, show=False, max_display=max_display)
    title_bits = [f"{label} | id={idx}"]
    if y_t is not None:
        title_bits.append(f"y_true={y_t}")
    if y_p is not None:
        title_bits.append(f"y_pred={y_p}")
    if y_prob is not None:
        title_bits.append(f"y_prob={y_prob:.3f}")
    if name_tag:
        title_bits.append(f"[{name_tag}]")
    ttl = " , ".join(title_bits) + thr_txt + " (Youden thr on TEST)"
    plt.title(ttl)
    plt.tight_layout()
    save_png_svg(
        out_dir
        / (
            f"{label}__waterfall_id{idx}"
            + (f"__{name_tag}" if name_tag else "")
            + ".png"
        ),
        dpi=300,
        bbox_inches="tight",
    )

    # force (옵션)
    try:
        plt.figure(figsize=(7, 2.8))
        shap.plots.force(
            expl[idx].base_values, expl[idx].values, matplotlib=True, show=False
        )
        plt.tight_layout()
        save_png_svg(
            out_dir
            / (
                f"{label}__force_id{idx}"
                + (f"__{name_tag}" if name_tag else "")
                + ".png"
            ),
            dpi=300,
        )
    except Exception:
        plt.close()


# =============== DBSCAN 통계/요약 ===============
def add_dbscan_stats(
    out_dir: Path,
    label: str,
    y_true: np.ndarray,
    y_prob: np.ndarray,
    labels: np.ndarray,
):
    lab = np.asarray(labels)
    clusters = [c for c in np.unique(lab) if c != -1]
    rows = []
    for c in clusters:
        m = lab == c
        rows.append(
            dict(
                cluster=int(c),
                n=int(m.sum()),
                event_rate=float(np.mean(y_true[m] == 1)),
                mean_y_prob=float(np.mean(y_prob[m])),
            )
        )
    df = pd.DataFrame(rows).sort_values("n", ascending=False)

    chi_p = np.nan
    kw_p = np.nan
    if len(clusters) >= 2:
        table = [
            [
                int(np.sum((lab == c) & (y_true == 1))),
                int(np.sum((lab == c) & (y_true == 0))),
            ]
            for c in clusters
        ]
        chi_p = chi2_contingency(table)[1]
        groups = [y_prob[lab == c] for c in clusters]
        kw_p = kruskal(*groups).pvalue

    df["p_event_rate(chi2)"] = chi_p
    df["p_yprob(kruskal)"] = kw_p
    df.to_csv(out_dir / f"{label}__dbscan_cluster_stats.csv", index=False)
    return df


# =============== 메인 루틴 ===============


def run_cluster_analysis_for_label(
    label: str,
    umap_n_components: int = 12,  # 군집용 UMAP 차원
    umap_n_neighbors: int = 12,
    umap_min_dist: float = 0.01,
    umap_metric: str = "euclidean",  # cosine
    min_samples: int = 6,  # DBSCAN
    eps: float | None = None,
    dbscan_metric: str = "euclidean",
    target_min_clusters: int = 2,
    max_noise_ratio: float = 0.45,
    max_trials: int = 5,
    topN: int = 10,
    agg_for_heatmap: str = "mean",
    save_representatives: bool = True,
    random_state: int = 42,
    drop_top_n_features: int = 0,
):
    assert HAS_UMAP, "umap-learn 패키지가 필요합니다."
    print(f"\n=== {label}: UMAP+DBSCAN (adaptive) ===")
    out_dir = OUT_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    # 0) 데이터 로드
    expl, vals, feat_names, y_prob, y_true, X_te_t, df_pred = get_shap_matrix(label)

    # [NEW] Top N Feature 제거 로직
    if drop_top_n_features > 0:
        print(f"-> Filtering out Top {drop_top_n_features} SHAP features...")

        # 1. 각 변수의 절대값 평균(|SHAP|) 계산
        mean_abs_shap = np.mean(np.abs(vals), axis=0)

        # 2. 중요도가 높은 순서대로 인덱스 정렬
        top_indices = np.argsort(mean_abs_shap)[::-1][:drop_top_n_features]

        # 3. 제거될 변수 이름 출력 (확인용)
        dropped_names = [feat_names[i] for i in top_indices]
        print(f"   Dropped: {dropped_names}")

        # 4. 해당 변수들을 vals와 feat_names에서 제거
        # (남길 인덱스: 전체 인덱스 집합 - Top 인덱스 집합)
        all_indices = np.arange(vals.shape[1])
        keep_indices = np.setdiff1d(all_indices, top_indices)

        # 원래 순서 유지하면서 필터링
        keep_indices = np.sort(keep_indices)

        vals = vals[:, keep_indices]  # SHAP 값 행렬 업데이트
        feat_names = [feat_names[i] for i in keep_indices]  # 변수명 리스트 업데이트

        print(f"   Remaining features: {vals.shape[1]}")

    # Youden 임계값 & df_pred 보정
    ythr = _try_read_youden_threshold(label, base_dir=OUT_ROOT.parent)
    if ythr is None:
        ythr = _compute_youden_from_preds(
            np.asarray(y_true).astype(int), np.asarray(y_prob)
        )
    if "y_true" not in df_pred.columns:
        df_pred = df_pred.assign(y_true=np.asarray(y_true).astype(int))
    if "y_pred" not in df_pred.columns and "y_prob" in df_pred.columns:
        df_pred = df_pred.assign(y_pred=(df_pred["y_prob"] >= ythr).astype(int))

    # 1) 표준화 & UMAP(고차원; 군집용)
    # V = _standardize(vals) # 표준화
    V = vals
    um_hi = UMAP.UMAP(
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=random_state,
    )
    X_hi = um_hi.fit_transform(V)

    # 2) eps 초기값 (k-distance) & 곡선 저장
    if eps is None:
        eps0, kth = _kdist_eps_heuristic(X_hi, k=min_samples, pct=0.9)  # 0.85
        eps = eps0
        save_kdist_plot(
            kth,
            k=min_samples,
            out_path=(out_dir / f"{label}__kdist_k{min_samples}.png"),
        )

    # 3) DBSCAN 적응형 루프
    trial = 0
    labels = None
    info_hist = []
    cur_min_samples = int(min_samples)
    cur_eps = float(eps)

    while trial < max_trials:
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))

        info_hist.append(
            dict(
                trial=trial,
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

        if (n_clusters >= target_min_clusters) and (noise_ratio <= max_noise_ratio):
            break

        # 조정 규칙
        if noise_ratio > max_noise_ratio:
            cur_eps *= 1.20
            if cur_min_samples > 3:
                cur_min_samples -= 1
        elif n_clusters < target_min_clusters:
            cur_eps *= 0.88
        else:
            cur_eps *= 1.05

        trial += 1

    # all-noise 대비: 마지막 완화 시도 1회
    if (labels is None) or np.all(labels == -1):
        cur_eps *= 1.30
        db = DBSCAN(
            eps=cur_eps, min_samples=cur_min_samples, metric=dbscan_metric, n_jobs=-1
        )
        labels = db.fit_predict(X_hi)  # X_hi
        noise_ratio = float(np.mean(labels == -1))
        n_clusters = int(len(set(labels) - {-1}))
        info_hist.append(
            dict(
                trial="final_relax",
                eps=cur_eps,
                min_samples=cur_min_samples,
                n_clusters=n_clusters,
                noise_ratio=noise_ratio,
            )
        )

    # 4) 시각화용 2D UMAP (시각화 전용)
    if umap_n_components == 2:
        Z2 = X_hi
    else:
        um_2d = UMAP.UMAP(
            n_components=2,
            n_neighbors=max(umap_n_neighbors, 15),
            min_dist=max(umap_min_dist, 0.05),
            metric=umap_metric,
            random_state=random_state,
        )
        Z2 = um_2d.fit_transform(V)

    # [NEW] 위험도 순으로 라벨 재정렬 (이 부분 추가!)
    # Cluster 0 = Low Risk, Cluster N = High Risk
    if labels is not None:
        print("-> Reordering clusters by risk (0: Low -> N: High)...")
        labels = reorder_labels_by_risk(labels, np.asarray(y_true))

    # 5) 라벨 매핑 저장 (이제 정렬된 labels가 저장됨)
    m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # # 5) 라벨 매핑 저장(노이즈 포함)
    # m = pd.DataFrame({"idx": np.arange(len(labels)), "cluster": labels})
    # m.to_csv(out_dir / f"{label}__dbscan_labels.csv", index=False)

    # # ⭐️⭐️⭐️⭐️기존 코드 6) 2D 플롯 (클러스터 색) ⭐️⭐️⭐️⭐️
    # uniq = sorted(np.unique(labels))
    # # 클러스터 개수가 많을 수 있으므로 tab10 사용 권장
    # cmap = plt.get_cmap("tab10")

    # # 범례 공간 확보를 위해 가로 사이즈를 약간 늘림
    # plt.figure(figsize=(8.5, 6.0))

    # # 1. 노이즈(-1) 먼저 그리기 (배경으로 깔리게 처리)
    # if -1 in uniq:
    #     mask = labels == -1
    #     plt.scatter(
    #         Z2[mask, 0],
    #         Z2[mask, 1],
    #         c="#e0e0e0",  # 아주 연한 회색
    #         edgecolor="#bbbbbb",  # 테두리는 조금 진하게
    #         linewidth=0.1,
    #         s=15,  # 크기는 작게
    #         alpha=0.4,  # 투명하게
    #         label="Noise",  # 범례 이름
    #         zorder=1,  # 가장 뒤쪽에 배치
    #     )

    # # 2. 실제 클러스터 그리기 Loop
    # for c in uniq:
    #     if c == -1:
    #         continue
    #     mask = labels == c
    #     color = cmap(c % 20)  # 색상 순환 적용

    #     plt.scatter(
    #         Z2[mask, 0],
    #         Z2[mask, 1],
    #         c=[color],  # 단일 색상 적용
    #         s=22,  # 클러스터 포인트는 조금 더 크게
    #         alpha=0.9,  # 진하게
    #         label=f"Cluster {c}",
    #         edgecolor="white",  # 포인트 구분감
    #         linewidth=0.3,
    #         zorder=2,  # 노이즈보다 앞쪽에 배치
    #     )

    # title = f"UMAP + DBSCAN ({umap_n_components}D)"
    # # 제목에 전체 대비 노이즈 비율 표기
    # n_noise = np.sum(labels == -1)
    # n_total = len(labels)
    # plt.title(
    #     f"{title}\nTotal: {n_total}, Clusters: {len(set(labels)-{-1})}, Noise: {n_noise} ({n_noise/n_total:.1%})",
    #     fontsize=11,
    # )
    # plt.xlabel("UMAP-1")
    # plt.ylabel("UMAP-2")

    # # 범례 설정 (그래프 영역 밖 우측 상단에 배치)
    # plt.legend(
    #     bbox_to_anchor=(1.02, 1),
    #     loc="upper left",
    #     borderaxespad=0,
    #     frameon=False,
    #     fontsize=9,
    #     markerscale=1.5,  # 범례의 점 크기 키우기
    # )

    # plt.tight_layout()
    # save_png_svg(
    #     out_dir / f"{label}__UMAP2D_DBSCAN_tuned.png", dpi=300, bbox_inches="tight"
    # )

    # 6) 2D 플롯 (Hull + Annotation 적용 버전)
    # print("-> Plotting UMAP with Threshold-based Risk Labels...")
    # plot_umap_with_hulls_and_labels(
    #     Z2=Z2,
    #     labels=labels,
    #     y_prob=y_prob,  # [중요] 여기에 y_prob를 반드시 넘겨주세요!
    #     out_path=out_dir / f"{label}__UMAP2D_DBSCAN_annotated.png",
    #     title=f"Risk Patterns ({label})",
    # )

    print("-> Plotting UMAP for PPT (Event Rate based)...")
    plot_umap_for_ppt(
        Z2=Z2,
        labels=labels,
        y_prob=y_prob,  # 기존 인자
        y_true=y_true,  # [NEW] 실제 정답값 전달
        out_path=out_dir / f"{label}__UMAP2D_DBSCAN_annotated_ppt.png",
        title=f"Risk Patterns ({label})",
    )

    # 6-2) 2D 플롯 (위험도 색)
    plt.figure(figsize=(7.2, 6.2))
    sc = plt.scatter(Z2[:, 0], Z2[:, 1], c=y_prob, s=18, alpha=0.9, cmap="plasma")
    plt.colorbar(sc, label="Predicted risk (prob.)")
    plt.title(f"UMAP(2D) (colored by risk) — {label}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.margins(0.02)
    save_png_svg(
        out_dir / f"{label}__UMAP2D_risk_DBSCAN.png", dpi=300, bbox_inches="tight"
    )

    # 7) 클러스터 통계/요약 + 히트맵
    df_stats = add_dbscan_stats(
        out_dir,
        label,
        y_true=np.asarray(y_true),
        y_prob=np.asarray(y_prob),
        labels=np.asarray(labels),
    )
    ret = plot_cluster_heatmap(
        vals,
        feat_names,
        labels,
        topN=topN,
        out_path=out_dir / f"{label}__heatmap_top{topN}_dbscan.png",
        cmap="magma_r",  # rocket_r, viridis_r
        agg=agg_for_heatmap,
    )

    # === 수치 저장(요청 파일명 규칙) ===
    if ret is not None:
        # 모든 피처의 원시 |SHAP| (클러스터 × 전체 피처)
        ret["raw_all"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__ALL_FEATURES.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 원시 |SHAP|
        ret["raw_top"].to_csv(
            out_dir / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_RAW.csv",
            index=True,
        )
        # 히트맵에 쓰인 TopN의 행 정규화(상대값)
        ret["norm_top"].to_csv(
            out_dir
            / f"{label}__heatmap_values_{agg_for_heatmap}__TOP{topN}_ROWNORM.csv",
            index=True,
        )

    # 8) 대표 케이스 저장 (mean/max/min + FP/FN)
    if save_representatives:
        rep_dir = out_dir / "representatives_dbscan"
        # mean/max/min
        reps = pick_representatives(y_prob, np.asarray(labels))
        for rep in reps:
            c = rep["cluster"]
            for tag in ["mean_idx", "max_idx", "min_idx"]:
                i = rep[tag]
                name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                save_local_plots(
                    label,
                    expl,
                    i,
                    rep_dir / f"cluster_{c}",
                    feat_names=feat_names,
                    max_display=15,
                    df_pred=df_pred,
                    youden_thr=ythr,
                    name_tag=name_tag,
                )
        # FP/FN
        err_reps = pick_error_representatives(df_pred, np.asarray(labels))
        for rep in err_reps:
            c = rep["cluster"]
            for tag in ["fp_idx", "fn_idx"]:
                if tag in rep:
                    i = rep[tag]
                    name_tag = f"cluster{c}__{tag.replace('_idx','')}"
                    save_local_plots(
                        label,
                        expl,
                        i,
                        rep_dir / f"cluster_{c}",
                        feat_names=feat_names,
                        max_display=15,
                        df_pred=df_pred,
                        youden_thr=ythr,
                        name_tag=name_tag,
                    )

    # 9) 메타(JSON)
    meta = dict(
        timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
        algo="dbscan_tuned",
        random_state=random_state,
        umap=dict(
            n_components=umap_n_components,
            n_neighbors=umap_n_neighbors,
            min_dist=umap_min_dist,
            metric=umap_metric,
        ),
        final=dict(eps=float(cur_eps), min_samples=int(cur_min_samples)),
        trials=len([h for h in info_hist if isinstance(h.get("trial"), int)]),
        n_clusters=int(len(set(labels) - {-1})),
        noise_ratio=float(np.mean(np.asarray(labels) == -1)),
        history=info_hist,
    )
    with open(out_dir / f"{label}__dbscan_meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"-> saved: {out_dir}")
    return df_stats, labels

In [ ]:
# =============== 전체 라벨 반복 실행 ===============
all_cluster_summaries = []
for lb in LABELS:
    try:
        df_sum, labels = run_cluster_analysis_for_label(
            lb,
            umap_n_components=2,
            umap_n_neighbors=45,  # 40
            umap_min_dist=0.0,  # 0.05, 0.01
            umap_metric="euclidean",  # cosine
            min_samples=17,  # 20
            eps=None,
            dbscan_metric="euclidean",
            target_min_clusters=3,
            max_noise_ratio=0.45,  # 0.45
            max_trials=5,
            topN=10,
            agg_for_heatmap="mean",
            save_representatives=True,
            random_state=42,
            drop_top_n_features=2,
        )
        df_sum.insert(0, "label", lb)
        all_cluster_summaries.append(df_sum)
    except Exception as e:
        print(f"[WARN] {lb}: {e}")

if all_cluster_summaries:
    all_summ = pd.concat(all_cluster_summaries, ignore_index=True)
    out_csv = OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.csv"
    all_summ.to_csv(out_csv, index=False)
    print("✅ saved:", out_csv)
    # Excel(라벨별 시트) 옵션
    try:
        with pd.ExcelWriter(
            OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx"
        ) as xw:
            for lb, df_ in all_summ.groupby("label"):
                df_.to_excel(xw, sheet_name=lb, index=False)
        print("✅ saved:", OUT_ROOT / "ALL_labels__cluster_summary_dbscan_tuned.xlsx")
    except Exception as e:
        print("[WARN] Excel save:", e)